# Dev sqlite window functions

In [ ]:
# sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/CA100/search_recs/search_recs.sqlite3'
# sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/CA3500/search_recs/search_recs.sqlite3'
sqlite_path = '/Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3'

from notebooks import *

db_url = f'sqlite:///{sqlite_path}'
eng = sqla.create_engine(db_url)

# For `%%sqla -Iplan`
def plan(sql):
    _plan(sql)
    return sql

def _plan(sql: str, **kwargs):
    print_sqlite_query_plan(_query(f'explain query plan {sql}', **kwargs))

# Normal python version of `%%sqla -Iplan`
def query(sql: str, **kwargs) -> DF:
    _plan(sql, **kwargs)
    return _query(sql, **kwargs)

def _query(sql: str, **kwargs) -> DF:
    return pd.read_sql(con=eng, sql=sql, **kwargs)

def print_sqlite_query_plan(df: DF) -> str:
    # Query plan structure
    root = dict(detail=None, children=[])
    by_id = {0: root}
    for row in df_rows(df):
        node = dict(detail=row.detail, children=[])
        by_id[row.id] = node
        by_id[row.parent]['children'].append(node)
    # Print (pretty)
    def f(plan):
        if not plan['detail']:
            return [f(x) for x in plan['children']]
        if not plan['children']:
            return plan['detail']
        else:
            return {plan['detail']: [f(x) for x in plan['children']]}
    print(yaml.safe_dump(default_flow_style=False, data=f(root)))

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO Example window
with
  A as (
    select *, f_preds_0 as d_pc, species as slp
    from search_recs
    where species not in ('SOSP', 'HOSP', 'HOFI', 'BEWR', 'PYNU')
  ),
  B as (
    select species, source_id, slp, d_pc, row_number() over (partition by species order by d_pc) as sp_d_pc_i
    from A
  )
select *
from B
where sp_d_pc_i <= 3
limit 20

- CO-ROUTINE 2:
  - CO-ROUTINE 4:
    - SCAN TABLE search_recs USING INDEX ix_search_recs__species__species_species_group__quality__source_id
    - USE TEMP B-TREE FOR RIGHT PART OF ORDER BY
  - SCAN SUBQUERY 4
- SCAN SUBQUERY 2


Running query...

[0.069s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,slp,d_pc,sp_d_pc_i
0,ABDU,xc:255449,ABDU,4.370e-03,1
1,ABDU,xc:255465,ABDU,7.090e-03,2
2,ABDU,xc:168801,ABDU,1.090e-02,3
3,ABTO,xc:111858,ABTO,8.580e-11,1
4,ABTO,xc:247614,ABTO,3.970e-10,2
5,ABTO,xc:111830,ABTO,6.370e-10,3
6,ACFL,xc:254311,ACFL,1.410e-09,1
7,ACFL,xc:20708,ACFL,1.760e-08,2
8,ACFL,xc:319271,ACFL,5.940e-08,3
9,ACWO,xc:323869,ACWO,1.110e-08,1


In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO (0/n): Baseline, copied from app logs
select
  S.*,
  coalesce(slp.slp, 1e38) as slp
--# Must select * from (...) else union complains about nested order by
from (select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'NOWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
) union all select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'WIWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
)) S
  left join (
    select column1 as species, column2 as slp from (values ('NOWA', 2.7278727231365987), ('WIWA', 2.826477946769426))
  ) slp on S.species = slp.species
order by
  slp asc,
  d_pc asc
limit 31

- CO-ROUTINE 6:
  - COMPOUND QUERY:
    - LEFT-MOST SUBQUERY:
      - CO-ROUTINE 2:
        - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
          (species=?)
        - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
        - USE TEMP B-TREE FOR ORDER BY
      - SCAN SUBQUERY 2
    - UNION ALL:
      - CO-ROUTINE 5:
        - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
          (species=?)
        - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
        - USE TEMP B-TREE FOR ORDER BY
      - SCAN SUBQUERY 5
- MATERIALIZE 8:
  - SCAN 2 CONSTANT ROWS
- SCAN SUBQUERY 6 AS S
- SCAN SUBQUERY 8
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.160s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,background,background_species,basename,bird_seen,channels,country,dataset,date,download,downloaded,duration_s,elevation,hour,id,lat,length,license,license_detail,license_type,lng,locality,month,month_day,n_background_species,number_of_notes,path,pitch,place,place_only,playback_used,quality,recordist,recs_for_sp,remarks,sample_rate,sample_width_bit,samples_mb,samples_n,shorthand,species,species_com_name,species_family,species_longhand,species_order,species_query,species_sci_name,species_species_group,species_taxon_id,species_taxon_order,speed,state,state_only,subspecies,taxon_id,taxon_order,time,type,url,variable,volume,xc_bitrate_of_mp3,xc_channels,xc_id,xc_sampling_rate,year,source_id,f_preds_0,f_preds_1,f_preds_2,f_preds_3,f_preds_4,f_preds_5,f_preds_6,f_preds_7,f_preds_8,f_preds_9,f_preds_10,f_preds_11,f_preds_12,f_preds_13,f_preds_14,f_preds_15,f_preds_16,f_preds_17,f_preds_18,f_preds_19,f_preds_20,f_preds_21,f_preds_22,f_preds_23,f_preds_24,f_preds_25,f_preds_26,f_preds_27,f_preds_28,f_preds_29,f_preds_30,f_preds_31,f_preds_32,f_preds_33,f_preds_34,f_preds_35,f_preds_36,f_preds_37,f_preds_38,f_preds_39,f_preds_40,f_preds_41,f_preds_42,f_preds_43,f_preds_44,f_preds_45,f_preds_46,f_preds_47,f_preds_48,f_preds_49,f_preds_50,f_preds_51,f_preds_52,f_preds_53,f_preds_54,f_preds_55,f_preds_56,f_preds_57,f_preds_58,f_preds_59,f_preds_60,f_preds_61,f_preds_62,f_preds_63,f_preds_64,f_preds_65,f_preds_66,f_preds_67,f_preds_68,f_preds_69,f_preds_70,f_preds_71,f_preds_72,f_preds_73,f_preds_74,f_preds_75,f_preds_76,f_preds_77,f_preds_78,f_preds_79,f_preds_80,f_preds_81,f_preds_82,f_preds_83,f_preds_84,f_preds_85,f_preds_86,f_preds_87,f_preds_88,f_preds_89,f_preds_90,f_preds_91,f_preds_92,f_preds_93,f_preds_94,f_preds_95,f_preds_96,f_preds_97,f_preds_98,f_preds_99,f_preds_100,f_preds_101,f_preds_102,f_preds_103,f_preds_104,f_preds_105,f_preds_106,f_preds_107,f_preds_108,f_preds_109,f_preds_110,f_preds_111,f_preds_112,f_preds_113,f_preds_114,f_preds_115,f_preds_116,f_preds_117,f_preds_118,f_preds_119,f_preds_120,f_preds_121,f_preds_122,f_preds_123,f_preds_124,f_preds_125,f_preds_126,f_preds_127,f_preds_128,f_preds_129,f_preds_130,f_preds_131,f_preds_132,f_preds_133,f_preds_134,f_preds_135,f_preds_136,f_preds_137,f_preds_138,f_preds_139,f_preds_140,f_preds_141,f_preds_142,f_preds_143,f_preds_144,f_preds_145,f_preds_146,f_preds_147,f_preds_148,f_preds_149,f_preds_150,f_preds_151,f_preds_152,f_preds_153,f_preds_154,f_preds_155,f_preds_156,f_preds_157,f_preds_158,f_preds_159,f_preds_160,f_preds_161,f_preds_162,f_preds_163,f_preds_164,f_preds_165,f_preds_166,f_preds_167,f_preds_168,f_preds_169,f_preds_170,f_preds_171,f_preds_172,f_preds_173,f_preds_174,f_preds_175,f_preds_176,f_preds_177,f_preds_178,f_preds_179,f_preds_180,f_preds_181,f_preds_182,f_preds_183,f_preds_184,f_preds_185,f_preds_186,f_preds_187,f_preds_188,f_preds_189,f_preds_190,f_preds_191,f_preds_192,f_preds_193,f_preds_194,f_preds_195,f_preds_196,f_preds_197,f_preds_198,f_preds_199,f_preds_200,f_preds_201,f_preds_202,f_preds_203,f_preds_204,f_preds_205,f_preds_206,f_preds_207,f_preds_208,f_preds_209,f_preds_210,f_preds_211,f_preds_212,f_preds_213,f_preds_214,f_preds_215,f_preds_216,f_preds_217,f_preds_218,f_preds_219,f_preds_220,f_preds_221,f_preds_222,f_preds_223,f_preds_224,f_preds_225,f_preds_226,f_preds_227,f_preds_228,f_preds_229,f_preds_230,f_preds_231,f_preds_232,f_preds_233,f_preds_234,f_preds_235,f_preds_236,f_preds_237,f_preds_238,f_preds_239,f_preds_240,f_preds_241,f_preds_242,f_preds_243,f_preds_244,f_preds_245,f_preds_246,f_preds_247,f_preds_248,f_preds_249,f_preds_250,f_preds_251,f_preds_252,f_preds_253,f_preds_254,f_preds_255,f_preds_256,f_preds_257,f_preds_258,f_preds_259,f_preds_260,f_preds_261,f_preds_262,f_preds_263,f_preds_264,f_preds_265,f_preds_266,f_preds_267,f_preds_268,f_preds_269,f_preds_270,f_preds_271,f_preds_272,f_preds_273,f_preds_274,f_preds_275,f_preds_276,f_preds_277,f_preds_278,f_preds_279,f_preds_280,f_preds_281,f_preds_282,f_preds_283,f_pred

In [ ]:
%%sqla -ceng -p3
--# TODO (1/n)
select
  --# S.*,
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  --# row_number() over (partition by species order by d_pc) as sp_d_pc_i,
  null
from (
  select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
    where true
      and S.species = 'NOWA'
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec from results
    order by
      d_pc asc
    limit 3
  ) union all select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
    where true
      and S.species = 'WIWA'
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec from results
    order by
      d_pc asc
    limit 3
  )
) S left join (
  select column1 as species, column2 as slp
  from (values ('NOWA', 2.7278727231365987), ('WIWA', 2.826477946769426))
) slp on S.species = slp.species
order by slp asc, d_pc asc
limit 31

Running query...

[0.108s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,slp,d_pc,null
0,NOWA,xc:387766,2.73,1.0,None
1,NOWA,xc:101312,2.73,1.0,None
2,NOWA,xc:232909,2.73,1.0,None
3,WIWA,xc:412807,2.83,1.0,None
4,WIWA,xc:243818,2.83,1.0,None
5,WIWA,xc:159099,2.83,1.0,None


In [ ]:
%%sqla -ceng -q
--# TODO (2/n)
select
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  --# row_number() over (partition by species order by d_pc) as sp_d_pc_i,
  null
from (
  select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true
    where true
      and S.species = 'NOWA'
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
    limit 3
  ) union all select * from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true
    where true
      and S.species = 'WIWA'
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
    limit 3
  )
) S
  left join (select column1 as species, column2 as slp from (values
    ('NOWA', 2.7278727231365987),
    ('WIWA', 2.826477946769426)
  )) slp on S.species = slp.species
order by slp asc, d_pc asc
limit 31

,species,source_id,slp,d_pc,null
0,NOWA,xc:387766,2.73,1.0,None
1,NOWA,xc:101312,2.73,1.0,None
2,NOWA,xc:232909,2.73,1.0,None
3,WIWA,xc:412807,2.83,1.0,None
4,WIWA,xc:243818,2.83,1.0,None
5,WIWA,xc:159099,2.83,1.0,None


In [ ]:
%%sqla -ceng -q
--# TODO (3/n)
select
  S.species,
  S.source_id,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc,
  sp_d_pc_i
from (
  select
    S.*,
    row_number() over (partition by S.species order by d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:417354') Q on true --# For dot(S.f_preds, Q.f_preds)
    where true
      and S.species in ('NOWA', 'WIWA') --# [topSlps.map(x => x.species)]
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
    order by d_pc asc
  ) S
) S
  --# TODO Perf: try doing both joins at once, above
  left join (select column1 as species, column2 as slp from (values
    ('NOWA', 2.7278727231365987), --# [topSlps (-> tableSlp)]
    ('WIWA', 2.826477946769426)
  )) slp on S.species = slp.species
where sp_d_pc_i <= 3 --# [n_per_sp]
order by slp asc, d_pc asc
limit 31 --# [n_recs]

,species,source_id,slp,d_pc,sp_d_pc_i
0,NOWA,xc:387766,2.73,1.0,1
1,NOWA,xc:101312,2.73,1.0,2
2,NOWA,xc:232909,2.73,1.0,3
3,WIWA,xc:412807,2.83,1.0,1
4,WIWA,xc:243818,2.83,1.0,2
5,WIWA,xc:159099,2.83,1.0,3


In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (4/n)
select
  species,
  source_id,
  coalesce(slp, 1e38) as slp,
  d_pc,
  sp_d_pc_i
from (
  select
    *,
    row_number() over (partition by species order by d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      slp.slp,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      join (select * from search_recs where source_id = 'xc:417354') Q on true --# For dot(S.f_preds, Q.f_preds)
      join (select column1 as species, column2 as slp from (values
        ('NOWA', 2.7278727231365987), --# [topSlps (-> tableSlp)]
        ('WIWA', 2.826477946769426)
      )) slp on S.species = slp.species
    where true
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:417354' --# Exclude query_rec
  )
)
where sp_d_pc_i <= 3 --# [n_per_sp]
order by slp asc, d_pc asc
limit 31 --# [n_recs]

- CO-ROUTINE 6:
  - CO-ROUTINE 8:
    - MATERIALIZE 3:
      - SCAN 2 CONSTANT ROWS
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - SCAN SUBQUERY 3
    - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
      (species=?)
    - USE TEMP B-TREE FOR ORDER BY
  - SCAN SUBQUERY 8
- SCAN SUBQUERY 6
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.078s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,slp,d_pc,sp_d_pc_i
0,NOWA,xc:387766,2.73,1.0,1
1,NOWA,xc:101312,2.73,1.0,2
2,NOWA,xc:232909,2.73,1.0,3
3,WIWA,xc:412807,2.83,1.0,1
4,WIWA,xc:243818,2.83,1.0,2
5,WIWA,xc:159099,2.83,1.0,3


In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (5/n): New baseline (copied from app logs)
select
  S.species, S.source_id,
  sp_d_pc_i,
  coalesce(slp.slp, 1e38) as slp,
  S.d_pc
  --# S.variable, S.bird_seen, S.speed, S.remarks, S.hour, S.species_com_name, S.taxon_order, S.time, S.species_species_group, S.quality, S.id, S.license, S.elevation, S.lng, S.species, S.background_species, S.background, S.license_detail, S.duration_s, S.samples_n, S.place, S.n_background_species, S.date, S.species_taxon_order, S.year, S.xc_channels, S.downloaded, S.samples_mb, S.volume, S.number_of_notes, S.license_type, S.xc_bitrate_of_mp3, S.species_longhand, S.species_sci_name, S.pitch, S.month_day, S.path, S.url, S.norm_f_preds, S.species_query, S.state, S.channels, S.basename, S.species_family, S.species_taxon_id, S.type, S.recs_for_sp, S.country, S.lat, S.species_order, S.sample_width_bit, S.subspecies, S.download, S.source_id, S.taxon_id, S.recordist, S.place_only, S.shorthand, S.length, S.xc_sampling_rate, S.month, S.xc_id, S.state_only, S.dataset, S.playback_used, S.sample_rate, S.locality
from (
  select
    S.*,
    row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      left join (select * from search_recs where source_id = 'xc:416410') Q on true --# 1 row, for dot(S,Q)
    where true
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:416410' --# Exclude query_rec from results
  ) S
) S
  left join (
    select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005), ('WIFL', 3.4793422717445033), ('ABTO', 3.496172099469417), ('GRVI', 3.5193633517510596), ('RCKI', 3.6404688761047668), ('LASP', 3.6512290161829797), ('GTGR', 3.9715303202185503), ('WEVI', 4.211134790180211), ('LOSH', 4.242274068412453), ('MAWA', 4.350559483605008), ('WIWR', 4.367698139109152), ('MOWA', 4.452379012375921), ('SWTH', 4.460531357394325), ('HUVI', 4.645053683966041), ('CBTH', 4.686734006282624), ('GCSP', 4.706664237639098), ('COYE', 4.738316347112597), ('SUTA', 4.768478476743198), ('CACW', 4.897283943714701), ('BTGN', 5.055958148609322), ('CITE', 5.13783401208553), ('PARE', 5.1870673093210415), ('SPTO', 5.289424823342004), ('HOWR', 5.313536619683756), ('VIRA', 5.359643554296269), ('NOMO', 5.435658780172663), ('STJA', 5.443618350266081), ('NOWA', 5.571608467440403), ('TOWA', 5.624281844181796), ('GTTO', 5.672155589635655), ('HOOR', 5.754228876946757), ('BRBL', 5.803098377199502), ('BUOR', 5.860230662760299), ('WETA', 5.870270129701547), ('SAVS', 5.9350801202552175), ('GRRO', 6.088533962280162), ('CASJ', 6.16486681026883), ('EUCD', 6.263018618968476), ('BHCO', 6.2676917297004735), ('GAQU', 6.277149956237753), ('PRAW', 6.322347171125688), ('MAWR', 6.394385748801781), ('BLTE', 6.495218547138002), ('LEBI', 6.50714035895642), ('HOGR', 6.519299953142001), ('GRHE', 6.594901668884092), ('RIRA', 6.778172676625919), ('CEDW', 6.792401744450868), ('BEVI', 6.8168670075358735), ('AMRE', 6.8434601571283), ('PUFI', 6.8800714610394955), ('TRES', 6.932283234725144), ('CAKI', 6.98838225278954), ('LBWO', 7.0568784274031495), ('MALL', 7.058447717494623), ('ISSJ', 7.094417567384517), ('GRCA', 7.104691391888663), ('DOWO', 7.233165856131655), ('RTHA', 7.320819893142206), ('REVI', 7.323189558427389), ('CANW', 7.340249438571233), ('FOSP', 7.341033665941331), ('BTSP', 7.379653544270154), ('CAGU', 7.4009806956460285), ('BCSP', 7.450540318791626), ('CLGR', 7.529309001424839), ('CALT', 7.665288650261056), ('AMBI', 7.670159206932208), ('WCSP', 7.677722963732932), ('CONI', 7.7165008502646755), ('WISA', 7.737650989254115), ('SCOR', 7.7756358010811), ('ROWR', 7.800027821612021), ('VEFL', 7.836716430514252), ('LUWA', 7.861275711075517), ('YGVI', 8.032603594833692), ('EAPH', 8.045889673331839), ('RECR', 8.054306272386695), ('PLVI', 8.0573258089993), ('VGSW', 8.150851135605016), ('DEJU', 8.177422672877068), ('WEKI', 8.192093961397726), ('AMRO', 8.207358641896416), ('COKI', 8.22691108048428), ('WOSJ', 8.23519637345769), ('TBKI', 8.248404895133326), ('BRSP', 8.260137874042956), ('GRAJ', 8.319454820372373), ('CLNU', 8.41210360706917), ('HEWA', 8.443646487477599), ('YBCU', 8.455966185801852), ('GBHE', 8.470547027359219), ('COBH', 8.473457680601122), ('WOTH', 8.47425251776894), ('CORA', 8.504145101647248), ('WFIB', 8.558195246726559), ('GCKI', 8.675845206307086), ('SNEG', 8.71489775217107), ('CAVI', 8.824804994737796), ('BCFL', 8.865598141201284), ('GADW', 8.881805163459138), ('PBGR', 8.883883139405508), ('OCWA', 8.91571997772636), ('BGGN', 8.979557632540418), ('PISI', 9.013475155286066), ('BHGR', 9.048014026311973), ('RNPH', 9.106074085564245), ('CAGN', 9.194456615444194), ('WESA', 9.253091637619606), ('HOSP', 9.329810940921154), ('YBMA', 9.331748009232133), ('HOFI', 9.352768888613367), ('WEGR', 9.36361691338387), ('ATFL', 9.395993001431028), ('OSFL', 9.43129582619472), ('BANO', 9.440415528979932), ('BARS', 9.468711682387298), ('NRWS', 9.483742792319145), ('DUFL', 9.494884209667863), ('BLGR', 9.506186466177965), ('WREN', 9.588832680404476), ('WHIM', 9.59023830590102), ('RCSP', 9.612597665765831), ('BCHU', 9.636988913398117), ('RNGR', 9.649324364699009), ('NAWA', 9.65689968046352), ('WEWP', 9.677391716081651), ('GIWO', 9.683855960858526), ('CAQU', 9.68931968940774), ('TRBL', 9.69457759244987), ('AMPI', 9.705954469663716), ('CATH', 9.756499339000824), ('FOTE', 9.786710739159583), ('INDO', 9.798281775522625), ('MGWA', 9.850680760487974), ('WAVI', 9.87826785453523), ('INBU', 9.973553500978726), ('BNST', 10.003575107070436), ('GWTE', 10.004672621664334), ('CATE', 10.024939059789673), ('COCK', 10.058411906242194), ('BRAN', 10.070341811872169), ('LBCU', 10.07268529924788), ('BEKI', 10.08892382643895), ('GREG', 10.12263126882629), ('GRFL', 10.225121670747903), ('MOCH', 10.288602435960087), ('AMAV', 10.289414125141299), ('NOHA', 10.296462703885892), ('VERD', 10.359538873224341), ('RWBU', 10.38093207690275), ('BTAH', 10.444779583769158), ('BOGU', 10.459158320788868), ('BTPI', 10.46969181032973), ('SATH', 10.494303412559143), ('EWPW', 10.499739379193759), ('BBWA', 10.528161986296945), ('AMCR', 10.534259580501928), ('ACWO', 10.556080342422899), ('LETE', 10.676353186733806), ('HETH', 10.703737887764852), ('YRWA', 10.740663773204677), ('BETH', 10.75306310082828), ('BESP', 10.775242600307786), ('JAWE', 10.79885071045656), ('LBDO', 10.881159712750236), ('CASP', 10.881791795833225), ('PIWA', 10.917633353380774), ('LENI', 11.001262318008), ('WIWA', 11.076173104920157), ('ELTE', 11.134156481436278), ('BTYW', 11.159490114984676), ('MEGU', 11.19561006888942), ('COGA', 11.234191400659466), ('SBMU', 11.272577125505444), ('GWFG', 11.283658145820533), ('HOLA', 11.289166873118502), ('YHPA', 11.30167898908479), ('BLPH', 11.328764066954324), ('COHA', 11.399349884857482), ('GRSP', 11.426447575635958), ('ROSA', 11.514579864617541), ('WHWO', 11.554579673748899), ('BCNH', 11.616288770600107), ('WEWA', 11.63805110813809), ('COGD', 11.660989309708581), ('WTSP', 11.705153274178675), ('YFGU', 11.710106064272063), ('RMPA', 11.745186647054975), ('WTKI', 11.745444015371856), ('DCFL', 11.775270191659672), ('SPSA', 11.8079276487243), ('WILL', 11.820715335632878), ('ANHU', 11.874078168204854), ('RBNU', 11.951802966228469), ('HAWO', 12.002295226387616), ('LEFL', 12.040031240389034), ('LAZB', 12.07708553255923), ('NAPA', 12.081497967472878), ('CSWA', 12.145381176599757), ('SOGR', 12.202756337377206), ('LAGO', 12.2319360538941), ('HAFL', 12.253908421142965), ('SOSA', 12.280515337039642), ('HOWA', 12.309078288331046), ('CAFI', 12.342163223335955), ('REDH', 12.342852696646961), ('CAHU', 12.34437175640791), ('PIGR', 12.370777588356646), ('AMCO', 12.371449846832995), ('CRTH', 12.40621727101908), ('CCSP', 12.418824267906743), ('WEME', 12.461447585447527), ('NUWO', 12.494546256185368), ('WBNU', 12.550259198156121), ('AMKE', 12.5879705692463), ('BFAL', 12.599010072947978), ('SUSC', 12.607163481842957), ('LEGO', 12.610194266715151), ('EGGO', 12.717279991381758), ('BTMJ', 12.734900484409916), ('LCPA', 12.785958480665037), ('SAPH', 12.85096814906219), ('BBWO', 12.853764320741432), ('TOSO', 12.887230564749972), ('PSFL', 12.887776107638214), ('BUSH', 12.89277208362413), ('WITU', 12.947646911113521), ('SNPL', 12.950877795143759), ('PAWR', 13.015895644703479), ('EVGR', 13.030762053934088), ('AMWI', 13.092302802697999), ('RBSA', 13.092911293379633), ('PTWH', 13.151428283688343), ('RBGR', 13.23211415043763), ('TRKI', 13.35721436984883), ('SEPL', 13.39987526822362), ('RCPA', 13.534886516815417), ('NOCA', 13.557554884405473), ('BLTU', 13.641568590587058), ('MAGO', 13.663751672726363), ('PHAI', 13.679443332396383), ('CANG', 13.712275984543163), ('PIJA', 13.779124742830192), ('WWDO', 13.93986752648307), ('NOFL', 13.949757055334633), ('NOPA', 13.967616353635064), ('FLOW', 14.046397653533175), ('AMGO', 14.09764953376981), ('MOUQ', 14.180958040833765), ('GRWA', 14.244986224414498), ('RTLO', 14.427145225017854), ('SACR', 14.49586095683052), ('LCTH', 14.535235838532195), ('RTPI', 14.566881452855002), ('COHU', 14.658086425044775), ('WTSW', 14.69155469601567), ('VATH', 14.760312386336937), ('CACG', 14.8905103260917), ('WEBL', 14.919877548339713), ('RRPA', 14.92159708600756), ('RSHA', 15.02440736450781), ('ROYT', 15.034523996218242), ('MWPW', 15.101850408480322), ('DUNL', 15.112934844811912), ('CLSW', 15.228457857956569), ('CHSP', 15.325009416549817), ('MERL', 15.36083794907528), ('SWHA', 15.483503064294434), ('SORA', 15.504577349038902), ('AFCD', 15.557755066107376), ('COLO', 15.571589072540275), ('PUMA', 15.580846337217782), ('GHOW', 15.592303703808632), ('LALO', 15.606447644743367), ('RBGU', 15.626537739256971), ('VASW', 15.65960731659203), ('DCCO', 15.705802157823962), ('CBCH', 15.724589490413988), ('PYNU', 15.742447906804415), ('GWGU', 16.06143149450292), ('PAGP', 16.112702834354685), ('SURF', 16.159712354031832), ('MOBL', 16.38585694486965), ('MODO', 16.57150675239372), ('PEFA', 16.61430653763232), ('SABS', 16.619973334139644), ('yecpar', 16.800916569278108), ('ALHU', 16.815754768941343), ('LEOW', 16.848703315122915), ('VESP', 16.867215051072662), ('PECO', 16.904050667056193), ('BUOW', 16.982252032483235), ('WEGU', 17.306432365345316), ('SBDO', 17.61577088023981), ('BLRA', 17.704378641694433), ('SNGO', 17.896372239524084), ('OSPR', 17.91787001874239), ('COPO', 18.000715626979783), ('DUWA', 18.28735100533962), ('PIWO', 18.399724416044027), ('GBTE', 18.476700211882985), ('BUFF', 18.570837432539953), ('KILL', 18.62114372213435), ('HEEG', 18.641071653818972), ('BLSK', 18.673321079568804), ('RODO', 18.96827923990534), ('NSWO', 19.09820594164006), ('WATA', 19.209856327280047), ('BLOY', 19.4653992890448), ('GRYE', 19.48918754998192), ('LESA', 19.491976161971518), ('LAGU', 19.787924146071667), ('AMDI', 19.90295247329546), ('MAMU', 20.236128093649693), ('WESO', 20.6932070454643), ('COMU', 21.32292012451962), ('SPOW', 22.169901163965974), ('ELOW', 22.460205414042218), ('TUSW', 22.492817124183066), ('BBPL', 22.779129209772936), ('BRAC', 23.61234120289399))
  ) slp on S.species = slp.species
where
  sp_d_pc_i <= 3
order by
  slp asc, d_pc asc
limit 31

- CO-ROUTINE 3:
  - CO-ROUTINE 337:
    - SCAN TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - USE TEMP B-TREE FOR RIGHT PART OF ORDER BY
  - SCAN SUBQUERY 337
- MATERIALIZE 334:
  - SCAN 331 CONSTANT ROWS
- SCAN SUBQUERY 3 AS S
- SEARCH SUBQUERY 334 USING AUTOMATIC COVERING INDEX (column1=?)
- USE TEMP B-TREE FOR ORDER BY


Running query...

[19.226s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,sp_d_pc_i,slp,d_pc
0,BRCR,xc:406990,1,2.53,0.688
1,BRCR,xc:352157,2,2.53,0.722
2,BRCR,xc:159463,3,2.53,0.736
3,BEWR,xc:408560,1,2.53,0.608
4,BEWR,xc:230968,2,2.53,0.726
5,BEWR,xc:160213,3,2.53,0.738
6,LISP,xc:322873,1,2.61,0.813
7,LISP,xc:322874,2,2.61,0.817
8,LISP,xc:159653,3,2.61,0.829
9,OATI,xc:350985,1,2.62,0.689


In [ ]:
%%sqla -ceng -p3 -Iplan
--# TODO (6/n): Speed up slow perf (~20s US, ~1s CA3500)
select
  S.species, S.source_id,
  sp_d_pc_i,
  coalesce(S.slp, 1e38) as slp,
  S.d_pc
  --# S.variable, S.bird_seen, S.speed, S.remarks, S.hour, S.species_com_name, S.taxon_order, S.time, S.species_species_group, S.quality, S.id, S.license, S.elevation, S.lng, S.species, S.background_species, S.background, S.license_detail, S.duration_s, S.samples_n, S.place, S.n_background_species, S.date, S.species_taxon_order, S.year, S.xc_channels, S.downloaded, S.samples_mb, S.volume, S.number_of_notes, S.license_type, S.xc_bitrate_of_mp3, S.species_longhand, S.species_sci_name, S.pitch, S.month_day, S.path, S.url, S.norm_f_preds, S.species_query, S.state, S.channels, S.basename, S.species_family, S.species_taxon_id, S.type, S.recs_for_sp, S.country, S.lat, S.species_order, S.sample_width_bit, S.subspecies, S.download, S.source_id, S.taxon_id, S.recordist, S.place_only, S.shorthand, S.length, S.xc_sampling_rate, S.month, S.xc_id, S.state_only, S.dataset, S.playback_used, S.sample_rate, S.locality
from (
  select
    S.*,
    row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
  from (
    select
      S.*,
      slp.slp,
      1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330) / S.norm_f_preds / Q.norm_f_preds as d_pc
    from search_recs S
      join (select * from search_recs where source_id = 'xc:416410') Q on true --# 1 row, for dot(S,Q)
      join (
        select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005))
        --# select column1 as species, column2 as slp from (values ('BRCR', 2.531013988420692), ('BEWR', 2.532770076870172), ('LISP', 2.612302265470383), ('OATI', 2.6209269915633775), ('YBCH', 2.693449148368873), ('RWBL', 2.85915840716264), ('YHBL', 2.9407066815495098), ('JUTI', 3.1964930770861226), ('SOSP', 3.3536266585633028), ('EUST', 3.4345511861796005), ('WIFL', 3.4793422717445033), ('ABTO', 3.496172099469417), ('GRVI', 3.5193633517510596), ('RCKI', 3.6404688761047668), ('LASP', 3.6512290161829797), ('GTGR', 3.9715303202185503), ('WEVI', 4.211134790180211), ('LOSH', 4.242274068412453), ('MAWA', 4.350559483605008), ('WIWR', 4.367698139109152), ('MOWA', 4.452379012375921), ('SWTH', 4.460531357394325), ('HUVI', 4.645053683966041), ('CBTH', 4.686734006282624), ('GCSP', 4.706664237639098), ('COYE', 4.738316347112597), ('SUTA', 4.768478476743198), ('CACW', 4.897283943714701), ('BTGN', 5.055958148609322), ('CITE', 5.13783401208553), ('PARE', 5.1870673093210415), ('SPTO', 5.289424823342004), ('HOWR', 5.313536619683756), ('VIRA', 5.359643554296269), ('NOMO', 5.435658780172663), ('STJA', 5.443618350266081), ('NOWA', 5.571608467440403), ('TOWA', 5.624281844181796), ('GTTO', 5.672155589635655), ('HOOR', 5.754228876946757), ('BRBL', 5.803098377199502), ('BUOR', 5.860230662760299), ('WETA', 5.870270129701547), ('SAVS', 5.9350801202552175), ('GRRO', 6.088533962280162), ('CASJ', 6.16486681026883), ('EUCD', 6.263018618968476), ('BHCO', 6.2676917297004735), ('GAQU', 6.277149956237753), ('PRAW', 6.322347171125688), ('MAWR', 6.394385748801781), ('BLTE', 6.495218547138002), ('LEBI', 6.50714035895642), ('HOGR', 6.519299953142001), ('GRHE', 6.594901668884092), ('RIRA', 6.778172676625919), ('CEDW', 6.792401744450868), ('BEVI', 6.8168670075358735), ('AMRE', 6.8434601571283), ('PUFI', 6.8800714610394955), ('TRES', 6.932283234725144), ('CAKI', 6.98838225278954), ('LBWO', 7.0568784274031495), ('MALL', 7.058447717494623), ('ISSJ', 7.094417567384517), ('GRCA', 7.104691391888663), ('DOWO', 7.233165856131655), ('RTHA', 7.320819893142206), ('REVI', 7.323189558427389), ('CANW', 7.340249438571233), ('FOSP', 7.341033665941331), ('BTSP', 7.379653544270154), ('CAGU', 7.4009806956460285), ('BCSP', 7.450540318791626), ('CLGR', 7.529309001424839), ('CALT', 7.665288650261056), ('AMBI', 7.670159206932208), ('WCSP', 7.677722963732932), ('CONI', 7.7165008502646755), ('WISA', 7.737650989254115), ('SCOR', 7.7756358010811), ('ROWR', 7.800027821612021), ('VEFL', 7.836716430514252), ('LUWA', 7.861275711075517), ('YGVI', 8.032603594833692), ('EAPH', 8.045889673331839), ('RECR', 8.054306272386695), ('PLVI', 8.0573258089993), ('VGSW', 8.150851135605016), ('DEJU', 8.177422672877068), ('WEKI', 8.192093961397726), ('AMRO', 8.207358641896416), ('COKI', 8.22691108048428), ('WOSJ', 8.23519637345769), ('TBKI', 8.248404895133326), ('BRSP', 8.260137874042956), ('GRAJ', 8.319454820372373), ('CLNU', 8.41210360706917), ('HEWA', 8.443646487477599), ('YBCU', 8.455966185801852), ('GBHE', 8.470547027359219), ('COBH', 8.473457680601122), ('WOTH', 8.47425251776894), ('CORA', 8.504145101647248), ('WFIB', 8.558195246726559), ('GCKI', 8.675845206307086), ('SNEG', 8.71489775217107), ('CAVI', 8.824804994737796), ('BCFL', 8.865598141201284), ('GADW', 8.881805163459138), ('PBGR', 8.883883139405508), ('OCWA', 8.91571997772636), ('BGGN', 8.979557632540418), ('PISI', 9.013475155286066), ('BHGR', 9.048014026311973), ('RNPH', 9.106074085564245), ('CAGN', 9.194456615444194), ('WESA', 9.253091637619606), ('HOSP', 9.329810940921154), ('YBMA', 9.331748009232133), ('HOFI', 9.352768888613367), ('WEGR', 9.36361691338387), ('ATFL', 9.395993001431028), ('OSFL', 9.43129582619472), ('BANO', 9.440415528979932), ('BARS', 9.468711682387298), ('NRWS', 9.483742792319145), ('DUFL', 9.494884209667863), ('BLGR', 9.506186466177965), ('WREN', 9.588832680404476), ('WHIM', 9.59023830590102), ('RCSP', 9.612597665765831), ('BCHU', 9.636988913398117), ('RNGR', 9.649324364699009), ('NAWA', 9.65689968046352), ('WEWP', 9.677391716081651), ('GIWO', 9.683855960858526), ('CAQU', 9.68931968940774), ('TRBL', 9.69457759244987), ('AMPI', 9.705954469663716), ('CATH', 9.756499339000824), ('FOTE', 9.786710739159583), ('INDO', 9.798281775522625), ('MGWA', 9.850680760487974), ('WAVI', 9.87826785453523), ('INBU', 9.973553500978726), ('BNST', 10.003575107070436), ('GWTE', 10.004672621664334), ('CATE', 10.024939059789673), ('COCK', 10.058411906242194), ('BRAN', 10.070341811872169), ('LBCU', 10.07268529924788), ('BEKI', 10.08892382643895), ('GREG', 10.12263126882629), ('GRFL', 10.225121670747903), ('MOCH', 10.288602435960087), ('AMAV', 10.289414125141299), ('NOHA', 10.296462703885892), ('VERD', 10.359538873224341), ('RWBU', 10.38093207690275), ('BTAH', 10.444779583769158), ('BOGU', 10.459158320788868), ('BTPI', 10.46969181032973), ('SATH', 10.494303412559143), ('EWPW', 10.499739379193759), ('BBWA', 10.528161986296945), ('AMCR', 10.534259580501928), ('ACWO', 10.556080342422899), ('LETE', 10.676353186733806), ('HETH', 10.703737887764852), ('YRWA', 10.740663773204677), ('BETH', 10.75306310082828), ('BESP', 10.775242600307786), ('JAWE', 10.79885071045656), ('LBDO', 10.881159712750236), ('CASP', 10.881791795833225), ('PIWA', 10.917633353380774), ('LENI', 11.001262318008), ('WIWA', 11.076173104920157), ('ELTE', 11.134156481436278), ('BTYW', 11.159490114984676), ('MEGU', 11.19561006888942), ('COGA', 11.234191400659466), ('SBMU', 11.272577125505444), ('GWFG', 11.283658145820533), ('HOLA', 11.289166873118502), ('YHPA', 11.30167898908479), ('BLPH', 11.328764066954324), ('COHA', 11.399349884857482), ('GRSP', 11.426447575635958), ('ROSA', 11.514579864617541), ('WHWO', 11.554579673748899), ('BCNH', 11.616288770600107), ('WEWA', 11.63805110813809), ('COGD', 11.660989309708581), ('WTSP', 11.705153274178675), ('YFGU', 11.710106064272063), ('RMPA', 11.745186647054975), ('WTKI', 11.745444015371856), ('DCFL', 11.775270191659672), ('SPSA', 11.8079276487243), ('WILL', 11.820715335632878), ('ANHU', 11.874078168204854), ('RBNU', 11.951802966228469), ('HAWO', 12.002295226387616), ('LEFL', 12.040031240389034), ('LAZB', 12.07708553255923), ('NAPA', 12.081497967472878), ('CSWA', 12.145381176599757), ('SOGR', 12.202756337377206), ('LAGO', 12.2319360538941), ('HAFL', 12.253908421142965), ('SOSA', 12.280515337039642), ('HOWA', 12.309078288331046), ('CAFI', 12.342163223335955), ('REDH', 12.342852696646961), ('CAHU', 12.34437175640791), ('PIGR', 12.370777588356646), ('AMCO', 12.371449846832995), ('CRTH', 12.40621727101908), ('CCSP', 12.418824267906743), ('WEME', 12.461447585447527), ('NUWO', 12.494546256185368), ('WBNU', 12.550259198156121), ('AMKE', 12.5879705692463), ('BFAL', 12.599010072947978), ('SUSC', 12.607163481842957), ('LEGO', 12.610194266715151), ('EGGO', 12.717279991381758), ('BTMJ', 12.734900484409916), ('LCPA', 12.785958480665037), ('SAPH', 12.85096814906219), ('BBWO', 12.853764320741432), ('TOSO', 12.887230564749972), ('PSFL', 12.887776107638214), ('BUSH', 12.89277208362413), ('WITU', 12.947646911113521), ('SNPL', 12.950877795143759), ('PAWR', 13.015895644703479), ('EVGR', 13.030762053934088), ('AMWI', 13.092302802697999), ('RBSA', 13.092911293379633), ('PTWH', 13.151428283688343), ('RBGR', 13.23211415043763), ('TRKI', 13.35721436984883), ('SEPL', 13.39987526822362), ('RCPA', 13.534886516815417), ('NOCA', 13.557554884405473), ('BLTU', 13.641568590587058), ('MAGO', 13.663751672726363), ('PHAI', 13.679443332396383), ('CANG', 13.712275984543163), ('PIJA', 13.779124742830192), ('WWDO', 13.93986752648307), ('NOFL', 13.949757055334633), ('NOPA', 13.967616353635064), ('FLOW', 14.046397653533175), ('AMGO', 14.09764953376981), ('MOUQ', 14.180958040833765), ('GRWA', 14.244986224414498), ('RTLO', 14.427145225017854), ('SACR', 14.49586095683052), ('LCTH', 14.535235838532195), ('RTPI', 14.566881452855002), ('COHU', 14.658086425044775), ('WTSW', 14.69155469601567), ('VATH', 14.760312386336937), ('CACG', 14.8905103260917), ('WEBL', 14.919877548339713), ('RRPA', 14.92159708600756), ('RSHA', 15.02440736450781), ('ROYT', 15.034523996218242), ('MWPW', 15.101850408480322), ('DUNL', 15.112934844811912), ('CLSW', 15.228457857956569), ('CHSP', 15.325009416549817), ('MERL', 15.36083794907528), ('SWHA', 15.483503064294434), ('SORA', 15.504577349038902), ('AFCD', 15.557755066107376), ('COLO', 15.571589072540275), ('PUMA', 15.580846337217782), ('GHOW', 15.592303703808632), ('LALO', 15.606447644743367), ('RBGU', 15.626537739256971), ('VASW', 15.65960731659203), ('DCCO', 15.705802157823962), ('CBCH', 15.724589490413988), ('PYNU', 15.742447906804415), ('GWGU', 16.06143149450292), ('PAGP', 16.112702834354685), ('SURF', 16.159712354031832), ('MOBL', 16.38585694486965), ('MODO', 16.57150675239372), ('PEFA', 16.61430653763232), ('SABS', 16.619973334139644), ('yecpar', 16.800916569278108), ('ALHU', 16.815754768941343), ('LEOW', 16.848703315122915), ('VESP', 16.867215051072662), ('PECO', 16.904050667056193), ('BUOW', 16.982252032483235), ('WEGU', 17.306432365345316), ('SBDO', 17.61577088023981), ('BLRA', 17.704378641694433), ('SNGO', 17.896372239524084), ('OSPR', 17.91787001874239), ('COPO', 18.000715626979783), ('DUWA', 18.28735100533962), ('PIWO', 18.399724416044027), ('GBTE', 18.476700211882985), ('BUFF', 18.570837432539953), ('KILL', 18.62114372213435), ('HEEG', 18.641071653818972), ('BLSK', 18.673321079568804), ('RODO', 18.96827923990534), ('NSWO', 19.09820594164006), ('WATA', 19.209856327280047), ('BLOY', 19.4653992890448), ('GRYE', 19.48918754998192), ('LESA', 19.491976161971518), ('LAGU', 19.787924146071667), ('AMDI', 19.90295247329546), ('MAMU', 20.236128093649693), ('WESO', 20.6932070454643), ('COMU', 21.32292012451962), ('SPOW', 22.169901163965974), ('ELOW', 22.460205414042218), ('TUSW', 22.492817124183066), ('BBPL', 22.779129209772936), ('BRAC', 23.61234120289399))
      ) slp on S.species = slp.species
    where true
      --# Empty subquery for species outside of placeFilter
      and S.quality in ('A', 'B')
      and S.source_id != 'xc:416410' --# Exclude query_rec from results
  ) S
) S
where
  sp_d_pc_i <= 3
order by
  slp asc, d_pc asc
limit 31

- CO-ROUTINE 3:
  - CO-ROUTINE 337:
    - SCAN TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - USE TEMP B-TREE FOR RIGHT PART OF ORDER BY
  - SCAN SUBQUERY 337
- MATERIALIZE 334:
  - SCAN 331 CONSTANT ROWS
- SCAN SUBQUERY 3 AS S
- SEARCH SUBQUERY 334 USING AUTOMATIC COVERING INDEX (column1=?)
- USE TEMP B-TREE FOR ORDER BY


Running query...

[19.226s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,sp_d_pc_i,slp,d_pc
0,BRCR,xc:406990,1,2.53,0.688
1,BRCR,xc:352157,2,2.53,0.722
2,BRCR,xc:159463,3,2.53,0.736
3,BEWR,xc:408560,1,2.53,0.608
4,BEWR,xc:230968,2,2.53,0.726
5,BEWR,xc:160213,3,2.53,0.738
6,LISP,xc:322873,1,2.61,0.813
7,LISP,xc:322874,2,2.61,0.817
8,LISP,xc:159653,3,2.61,0.829
9,OATI,xc:350985,1,2.62,0.689


# FIXME US crash [fixed via joinOpWithBalancedParens]

In [ ]:
%%sqla -ceng -p3 -Iplan
--# FIXME US crash [fixed via joinOpWithBalancedParens]
              select
                -- sp_d_pc_i, -- XXX Debug
                coalesce(S.slp, 1e38) as slp,
                S.d_pc,
                S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.license_type, S.species_order, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks
              from (
                select
                  S.*,
                  row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
                from (
                  select
                    S.*,
                    slp.slp,

              1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330 + S.f_preds_331*Q.f_preds_331 + S.f_preds_332*Q.f_preds_332 + S.f_preds_333*Q.f_preds_333 + S.f_preds_334*Q.f_preds_334 + S.f_preds_335*Q.f_preds_335 + S.f_preds_336*Q.f_preds_336 + S.f_preds_337*Q.f_preds_337 + S.f_preds_338*Q.f_preds_338 + S.f_preds_339*Q.f_preds_339 + S.f_preds_340*Q.f_preds_340 + S.f_preds_341*Q.f_preds_341 + S.f_preds_342*Q.f_preds_342 + S.f_preds_343*Q.f_preds_343 + S.f_preds_344*Q.f_preds_344 + S.f_preds_345*Q.f_preds_345 + S.f_preds_346*Q.f_preds_346 + S.f_preds_347*Q.f_preds_347 + S.f_preds_348*Q.f_preds_348 + S.f_preds_349*Q.f_preds_349 + S.f_preds_350*Q.f_preds_350 + S.f_preds_351*Q.f_preds_351 + S.f_preds_352*Q.f_preds_352 + S.f_preds_353*Q.f_preds_353 + S.f_preds_354*Q.f_preds_354 + S.f_preds_355*Q.f_preds_355 + S.f_preds_356*Q.f_preds_356 + S.f_preds_357*Q.f_preds_357 + S.f_preds_358*Q.f_preds_358 + S.f_preds_359*Q.f_preds_359 + S.f_preds_360*Q.f_preds_360 + S.f_preds_361*Q.f_preds_361 + S.f_preds_362*Q.f_preds_362 + S.f_preds_363*Q.f_preds_363 + S.f_preds_364*Q.f_preds_364 + S.f_preds_365*Q.f_preds_365 + S.f_preds_366*Q.f_preds_366 + S.f_preds_367*Q.f_preds_367 + S.f_preds_368*Q.f_preds_368 + S.f_preds_369*Q.f_preds_369 + S.f_preds_370*Q.f_preds_370 + S.f_preds_371*Q.f_preds_371 + S.f_preds_372*Q.f_preds_372 + S.f_preds_373*Q.f_preds_373 + S.f_preds_374*Q.f_preds_374 + S.f_preds_375*Q.f_preds_375 + S.f_preds_376*Q.f_preds_376 + S.f_preds_377*Q.f_preds_377 + S.f_preds_378*Q.f_preds_378 + S.f_preds_379*Q.f_preds_379 + S.f_preds_380*Q.f_preds_380 + S.f_preds_381*Q.f_preds_381 + S.f_preds_382*Q.f_preds_382 + S.f_preds_383*Q.f_preds_383 + S.f_preds_384*Q.f_preds_384 + S.f_preds_385*Q.f_preds_385 + S.f_preds_386*Q.f_preds_386 + S.f_preds_387*Q.f_preds_387 + S.f_preds_388*Q.f_preds_388 + S.f_preds_389*Q.f_preds_389 + S.f_preds_390*Q.f_preds_390 + S.f_preds_391*Q.f_preds_391 + S.f_preds_392*Q.f_preds_392 + S.f_preds_393*Q.f_preds_393 + S.f_preds_394*Q.f_preds_394 + S.f_preds_395*Q.f_preds_395 + S.f_preds_396*Q.f_preds_396 + S.f_preds_397*Q.f_preds_397 + S.f_preds_398*Q.f_preds_398 + S.f_preds_399*Q.f_preds_399 + S.f_preds_400*Q.f_preds_400 + S.f_preds_401*Q.f_preds_401 + S.f_preds_402*Q.f_preds_402 + S.f_preds_403*Q.f_preds_403 + S.f_preds_404*Q.f_preds_404 + S.f_preds_405*Q.f_preds_405 + S.f_preds_406*Q.f_preds_406 + S.f_preds_407*Q.f_preds_407 + S.f_preds_408*Q.f_preds_408 + S.f_preds_409*Q.f_preds_409 + S.f_preds_410*Q.f_preds_410 + S.f_preds_411*Q.f_preds_411 + S.f_preds_412*Q.f_preds_412 + S.f_preds_413*Q.f_preds_413 + S.f_preds_414*Q.f_preds_414 + S.f_preds_415*Q.f_preds_415 + S.f_preds_416*Q.f_preds_416 + S.f_preds_417*Q.f_preds_417 + S.f_preds_418*Q.f_preds_418 + S.f_preds_419*Q.f_preds_419 + S.f_preds_420*Q.f_preds_420 + S.f_preds_421*Q.f_preds_421 + S.f_preds_422*Q.f_preds_422 + S.f_preds_423*Q.f_preds_423 + S.f_preds_424*Q.f_preds_424 + S.f_preds_425*Q.f_preds_425 + S.f_preds_426*Q.f_preds_426 + S.f_preds_427*Q.f_preds_427 + S.f_preds_428*Q.f_preds_428 + S.f_preds_429*Q.f_preds_429 + S.f_preds_430*Q.f_preds_430 + S.f_preds_431*Q.f_preds_431 + S.f_preds_432*Q.f_preds_432 + S.f_preds_433*Q.f_preds_433 + S.f_preds_434*Q.f_preds_434 + S.f_preds_435*Q.f_preds_435 + S.f_preds_436*Q.f_preds_436 + S.f_preds_437*Q.f_preds_437 + S.f_preds_438*Q.f_preds_438 + S.f_preds_439*Q.f_preds_439 + S.f_preds_440*Q.f_preds_440 + S.f_preds_441*Q.f_preds_441 + S.f_preds_442*Q.f_preds_442 + S.f_preds_443*Q.f_preds_443 + S.f_preds_444*Q.f_preds_444 + S.f_preds_445*Q.f_preds_445 + S.f_preds_446*Q.f_preds_446 + S.f_preds_447*Q.f_preds_447 + S.f_preds_448*Q.f_preds_448 + S.f_preds_449*Q.f_preds_449 + S.f_preds_450*Q.f_preds_450 + S.f_preds_451*Q.f_preds_451 + S.f_preds_452*Q.f_preds_452 + S.f_preds_453*Q.f_preds_453 + S.f_preds_454*Q.f_preds_454 + S.f_preds_455*Q.f_preds_455 + S.f_preds_456*Q.f_preds_456 + S.f_preds_457*Q.f_preds_457 + S.f_preds_458*Q.f_preds_458 + S.f_preds_459*Q.f_preds_459 + S.f_preds_460*Q.f_preds_460 + S.f_preds_461*Q.f_preds_461 + S.f_preds_462*Q.f_preds_462 + S.f_preds_463*Q.f_preds_463 + S.f_preds_464*Q.f_preds_464 + S.f_preds_465*Q.f_preds_465 + S.f_preds_466*Q.f_preds_466 + S.f_preds_467*Q.f_preds_467 + S.f_preds_468*Q.f_preds_468 + S.f_preds_469*Q.f_preds_469 + S.f_preds_470*Q.f_preds_470 + S.f_preds_471*Q.f_preds_471 + S.f_preds_472*Q.f_preds_472 + S.f_preds_473*Q.f_preds_473 + S.f_preds_474*Q.f_preds_474 + S.f_preds_475*Q.f_preds_475 + S.f_preds_476*Q.f_preds_476 + S.f_preds_477*Q.f_preds_477 + S.f_preds_478*Q.f_preds_478 + S.f_preds_479*Q.f_preds_479 + S.f_preds_480*Q.f_preds_480 + S.f_preds_481*Q.f_preds_481 + S.f_preds_482*Q.f_preds_482 + S.f_preds_483*Q.f_preds_483 + S.f_preds_484*Q.f_preds_484 + S.f_preds_485*Q.f_preds_485 + S.f_preds_486*Q.f_preds_486 + S.f_preds_487*Q.f_preds_487 + S.f_preds_488*Q.f_preds_488 + S.f_preds_489*Q.f_preds_489 + S.f_preds_490*Q.f_preds_490 + S.f_preds_491*Q.f_preds_491 + S.f_preds_492*Q.f_preds_492 + S.f_preds_493*Q.f_preds_493 + S.f_preds_494*Q.f_preds_494 + S.f_preds_495*Q.f_preds_495 + S.f_preds_496*Q.f_preds_496 + S.f_preds_497*Q.f_preds_497 + S.f_preds_498*Q.f_preds_498 + S.f_preds_499*Q.f_preds_499 + S.f_preds_500*Q.f_preds_500 + S.f_preds_501*Q.f_preds_501 + S.f_preds_502*Q.f_preds_502 + S.f_preds_503*Q.f_preds_503 + S.f_preds_504*Q.f_preds_504 + S.f_preds_505*Q.f_preds_505 + S.f_preds_506*Q.f_preds_506 + S.f_preds_507*Q.f_preds_507 + S.f_preds_508*Q.f_preds_508 + S.f_preds_509*Q.f_preds_509 + S.f_preds_510*Q.f_preds_510 + S.f_preds_511*Q.f_preds_511 + S.f_preds_512*Q.f_preds_512 + S.f_preds_513*Q.f_preds_513 + S.f_preds_514*Q.f_preds_514 + S.f_preds_515*Q.f_preds_515 + S.f_preds_516*Q.f_preds_516 + S.f_preds_517*Q.f_preds_517 + S.f_preds_518*Q.f_preds_518 + S.f_preds_519*Q.f_preds_519 + S.f_preds_520*Q.f_preds_520 + S.f_preds_521*Q.f_preds_521 + S.f_preds_522*Q.f_preds_522 + S.f_preds_523*Q.f_preds_523 + S.f_preds_524*Q.f_preds_524 + S.f_preds_525*Q.f_preds_525 + S.f_preds_526*Q.f_preds_526 + S.f_preds_527*Q.f_preds_527 + S.f_preds_528*Q.f_preds_528 + S.f_preds_529*Q.f_preds_529 + S.f_preds_530*Q.f_preds_530 + S.f_preds_531*Q.f_preds_531 + S.f_preds_532*Q.f_preds_532 + S.f_preds_533*Q.f_preds_533 + S.f_preds_534*Q.f_preds_534 + S.f_preds_535*Q.f_preds_535 + S.f_preds_536*Q.f_preds_536 + S.f_preds_537*Q.f_preds_537 + S.f_preds_538*Q.f_preds_538 + S.f_preds_539*Q.f_preds_539 + S.f_preds_540*Q.f_preds_540 + S.f_preds_541*Q.f_preds_541 + S.f_preds_542*Q.f_preds_542 + S.f_preds_543*Q.f_preds_543 + S.f_preds_544*Q.f_preds_544 + S.f_preds_545*Q.f_preds_545 + S.f_preds_546*Q.f_preds_546 + S.f_preds_547*Q.f_preds_547 + S.f_preds_548*Q.f_preds_548 + S.f_preds_549*Q.f_preds_549 + S.f_preds_550*Q.f_preds_550 + S.f_preds_551*Q.f_preds_551 + S.f_preds_552*Q.f_preds_552 + S.f_preds_553*Q.f_preds_553 + S.f_preds_554*Q.f_preds_554 + S.f_preds_555*Q.f_preds_555 + S.f_preds_556*Q.f_preds_556 + S.f_preds_557*Q.f_preds_557 + S.f_preds_558*Q.f_preds_558 + S.f_preds_559*Q.f_preds_559 + S.f_preds_560*Q.f_preds_560 + S.f_preds_561*Q.f_preds_561 + S.f_preds_562*Q.f_preds_562 + S.f_preds_563*Q.f_preds_563 + S.f_preds_564*Q.f_preds_564 + S.f_preds_565*Q.f_preds_565 + S.f_preds_566*Q.f_preds_566 + S.f_preds_567*Q.f_preds_567 + S.f_preds_568*Q.f_preds_568 + S.f_preds_569*Q.f_preds_569 + S.f_preds_570*Q.f_preds_570 + S.f_preds_571*Q.f_preds_571 + S.f_preds_572*Q.f_preds_572 + S.f_preds_573*Q.f_preds_573 + S.f_preds_574*Q.f_preds_574 + S.f_preds_575*Q.f_preds_575 + S.f_preds_576*Q.f_preds_576 + S.f_preds_577*Q.f_preds_577 + S.f_preds_578*Q.f_preds_578 + S.f_preds_579*Q.f_preds_579 + S.f_preds_580*Q.f_preds_580 + S.f_preds_581*Q.f_preds_581 + S.f_preds_582*Q.f_preds_582 + S.f_preds_583*Q.f_preds_583 + S.f_preds_584*Q.f_preds_584 + S.f_preds_585*Q.f_preds_585 + S.f_preds_586*Q.f_preds_586 + S.f_preds_587*Q.f_preds_587 + S.f_preds_588*Q.f_preds_588 + S.f_preds_589*Q.f_preds_589 + S.f_preds_590*Q.f_preds_590 + S.f_preds_591*Q.f_preds_591 + S.f_preds_592*Q.f_preds_592 + S.f_preds_593*Q.f_preds_593 + S.f_preds_594*Q.f_preds_594 + S.f_preds_595*Q.f_preds_595 + S.f_preds_596*Q.f_preds_596 + S.f_preds_597*Q.f_preds_597 + S.f_preds_598*Q.f_preds_598 + S.f_preds_599*Q.f_preds_599 + S.f_preds_600*Q.f_preds_600 + S.f_preds_601*Q.f_preds_601 + S.f_preds_602*Q.f_preds_602 + S.f_preds_603*Q.f_preds_603 + S.f_preds_604*Q.f_preds_604 + S.f_preds_605*Q.f_preds_605 + S.f_preds_606*Q.f_preds_606 + S.f_preds_607*Q.f_preds_607 + S.f_preds_608*Q.f_preds_608 + S.f_preds_609*Q.f_preds_609 + S.f_preds_610*Q.f_preds_610 + S.f_preds_611*Q.f_preds_611 + S.f_preds_612*Q.f_preds_612 + S.f_preds_613*Q.f_preds_613 + S.f_preds_614*Q.f_preds_614 + S.f_preds_615*Q.f_preds_615 + S.f_preds_616*Q.f_preds_616 + S.f_preds_617*Q.f_preds_617 + S.f_preds_618*Q.f_preds_618 + S.f_preds_619*Q.f_preds_619 + S.f_preds_620*Q.f_preds_620 + S.f_preds_621*Q.f_preds_621 + S.f_preds_622*Q.f_preds_622 + S.f_preds_623*Q.f_preds_623 + S.f_preds_624*Q.f_preds_624 + S.f_preds_625*Q.f_preds_625 + S.f_preds_626*Q.f_preds_626 + S.f_preds_627*Q.f_preds_627 + S.f_preds_628*Q.f_preds_628 + S.f_preds_629*Q.f_preds_629 + S.f_preds_630*Q.f_preds_630 + S.f_preds_631*Q.f_preds_631 + S.f_preds_632*Q.f_preds_632 + S.f_preds_633*Q.f_preds_633 + S.f_preds_634*Q.f_preds_634 + S.f_preds_635*Q.f_preds_635 + S.f_preds_636*Q.f_preds_636 + S.f_preds_637*Q.f_preds_637 + S.f_preds_638*Q.f_preds_638 + S.f_preds_639*Q.f_preds_639 + S.f_preds_640*Q.f_preds_640 + S.f_preds_641*Q.f_preds_641 + S.f_preds_642*Q.f_preds_642 + S.f_preds_643*Q.f_preds_643 + S.f_preds_644*Q.f_preds_644 + S.f_preds_645*Q.f_preds_645 + S.f_preds_646*Q.f_preds_646 + S.f_preds_647*Q.f_preds_647 + S.f_preds_648*Q.f_preds_648 + S.f_preds_649*Q.f_preds_649 + S.f_preds_650*Q.f_preds_650 + S.f_preds_651*Q.f_preds_651 + S.f_preds_652*Q.f_preds_652 + S.f_preds_653*Q.f_preds_653 + S.f_preds_654*Q.f_preds_654 + S.f_preds_655*Q.f_preds_655 + S.f_preds_656*Q.f_preds_656 + S.f_preds_657*Q.f_preds_657 + S.f_preds_658*Q.f_preds_658 + S.f_preds_659*Q.f_preds_659 + S.f_preds_660*Q.f_preds_660 + S.f_preds_661*Q.f_preds_661 + S.f_preds_662*Q.f_preds_662 + S.f_preds_663*Q.f_preds_663 + S.f_preds_664*Q.f_preds_664 + S.f_preds_665*Q.f_preds_665 + S.f_preds_666*Q.f_preds_666 + S.f_preds_667*Q.f_preds_667 + S.f_preds_668*Q.f_preds_668 + S.f_preds_669*Q.f_preds_669 + S.f_preds_670*Q.f_preds_670 + S.f_preds_671*Q.f_preds_671 + S.f_preds_672*Q.f_preds_672 + S.f_preds_673*Q.f_preds_673 + S.f_preds_674*Q.f_preds_674 + S.f_preds_675*Q.f_preds_675 + S.f_preds_676*Q.f_preds_676 + S.f_preds_677*Q.f_preds_677 + S.f_preds_678*Q.f_preds_678 + S.f_preds_679*Q.f_preds_679 + S.f_preds_680*Q.f_preds_680 + S.f_preds_681*Q.f_preds_681 + S.f_preds_682*Q.f_preds_682 + S.f_preds_683*Q.f_preds_683 + S.f_preds_684*Q.f_preds_684 + S.f_preds_685*Q.f_preds_685 + S.f_preds_686*Q.f_preds_686 + S.f_preds_687*Q.f_preds_687 + S.f_preds_688*Q.f_preds_688 + S.f_preds_689*Q.f_preds_689 + S.f_preds_690*Q.f_preds_690 + S.f_preds_691*Q.f_preds_691 + S.f_preds_692*Q.f_preds_692 + S.f_preds_693*Q.f_preds_693 + S.f_preds_694*Q.f_preds_694 + S.f_preds_695*Q.f_preds_695 + S.f_preds_696*Q.f_preds_696 + S.f_preds_697*Q.f_preds_697 + S.f_preds_698*Q.f_preds_698 + S.f_preds_699*Q.f_preds_699 + S.f_preds_700*Q.f_preds_700 + S.f_preds_701*Q.f_preds_701 + S.f_preds_702*Q.f_preds_702 + S.f_preds_703*Q.f_preds_703 + S.f_preds_704*Q.f_preds_704 + S.f_preds_705*Q.f_preds_705 + S.f_preds_706*Q.f_preds_706 + S.f_preds_707*Q.f_preds_707 + S.f_preds_708*Q.f_preds_708 + S.f_preds_709*Q.f_preds_709 + S.f_preds_710*Q.f_preds_710 + S.f_preds_711*Q.f_preds_711 + S.f_preds_712*Q.f_preds_712 + S.f_preds_713*Q.f_preds_713 + S.f_preds_714*Q.f_preds_714 + S.f_preds_715*Q.f_preds_715 + S.f_preds_716*Q.f_preds_716 + S.f_preds_717*Q.f_preds_717 + S.f_preds_718*Q.f_preds_718 + S.f_preds_719*Q.f_preds_719 + S.f_preds_720*Q.f_preds_720 + S.f_preds_721*Q.f_preds_721 + S.f_preds_722*Q.f_preds_722 + S.f_preds_723*Q.f_preds_723 + S.f_preds_724*Q.f_preds_724 + S.f_preds_725*Q.f_preds_725 + S.f_preds_726*Q.f_preds_726 + S.f_preds_727*Q.f_preds_727 + S.f_preds_728*Q.f_preds_728 + S.f_preds_729*Q.f_preds_729 + S.f_preds_730*Q.f_preds_730 + S.f_preds_731*Q.f_preds_731 + S.f_preds_732*Q.f_preds_732 + S.f_preds_733*Q.f_preds_733 + S.f_preds_734*Q.f_preds_734 + S.f_preds_735*Q.f_preds_735 + S.f_preds_736*Q.f_preds_736 + S.f_preds_737*Q.f_preds_737 + S.f_preds_738*Q.f_preds_738) / S.norm_f_preds / Q.norm_f_preds
             as d_pc
                  from search_recs S
                    -- Perf: join i/o left join so we filter down to top n_sp species, else query is very slow
                    join (
              select column1 as species, column2 as slp from (values ('SAPH', 4.318655671272131), ('PUFI', 4.447669086630143), ('HUVI', 4.500064404595279), ('WETA', 4.512406269203205), ('OSFL', 4.544073210765893), ('SPSA', 4.5529275135170675), ('MAMU', 4.659497654302724), ('DOWO', 4.673344795456002), ('WAVI', 4.8063899553448906), ('VIRA', 4.850684938768555), ('SURF', 4.873113054834411))
            ) slp on S.species = slp.species
                    join (select * from search_recs where source_id = 'xc:320384') Q on true -- 1 row, for dot(S,Q)
                  where true
                    and S.species in ('EMGO', 'SNGO', 'ROGO', 'GWFG', 'BRAN', 'CACG', 'CANG', 'MUSW', 'WODU', 'BWTE', 'CITE', 'NSHO', 'GADW', 'EUWI', 'AMWI', 'NOPI', 'GWTE', 'CANV', 'REDH', 'RNDU', 'GRSC', 'LESC', 'SUSC', 'BLSC', 'LTDU', 'BUFF', 'COGO', 'BAGO', 'HOME', 'COME', 'RBME', 'RUDU', 'CAQU', 'RNEP', 'WITU', 'PBGR', 'HOGR', 'RNGR', 'EAGR', 'WEGR', 'CLGR', 'RODO', 'BTPI', 'EUCD', 'MODO', 'BLSW', 'VASW', 'WTSW', 'ANHU', 'RUHU', 'ALHU', 'VIRA', 'SORA', 'COGA', 'AMCO', 'SACR', 'BNST', 'AMAV', 'BLOY', 'BBPL', 'PAGP', 'SNPL', 'SEPL', 'KILL', 'WHIM', 'LBCU', 'MAGO', 'BLTU', 'SURF', 'SAND', 'DUNL', 'BASA', 'LESA', 'PESA', 'SESA', 'WESA', 'SBDO', 'LBDO', 'WISN', 'WIPH', 'RNPH', 'REPH', 'SPSA', 'SOSA', 'WATA', 'GRYE', 'WILL', 'LEYE', 'POJA', 'PAJA', 'COMU', 'PIGU', 'MAMU', 'ANMU', 'CAAU', 'RHAU', 'TUPU', 'BLKI', 'SAGU', 'BOGU', 'HEEG', 'MEGU', 'RBGU', 'WEGU', 'CAGU', 'HERG', 'ICGU', 'GWGU', 'GLGU', 'CATE', 'BLTE', 'COTE', 'FOTE', 'ELTE', 'BLSK', 'RTLO', 'PALO', 'COLO', 'NOFU', 'SOSH', 'BVSH', 'BRBO', 'BRAC', 'PECO', 'DCCO', 'BRPE', 'AMBI', 'GBHE', 'GREG', 'SNEG', 'GRHE', 'BCNH', 'WFIB', 'TUVU', 'OSPR', 'WTKI', 'GOEA', 'NOHA', 'SSHA', 'COHA', 'BAEA', 'RSHA', 'BWHA', 'RTHA', 'FEHA', 'BANO', 'GHOW', 'SEOW', 'BEKI', 'RBSA', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'PIWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'PRFA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'TRKI', 'WEKI', 'LOSH', 'HUVI', 'CAVI', 'WAVI', 'YGVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'NRWS', 'PUMA', 'TRES', 'VGSW', 'BANS', 'BARS', 'CLSW', 'CBCH', 'BUSH', 'RBNU', 'WBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'EYWA', 'WHWA', 'AMPI', 'CEDW', 'HOFI', 'PUFI', 'RECR', 'PISI', 'LEGO', 'AMGO', 'GRSP', 'CHSP', 'CCSP', 'BRSP', 'LASP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'SWSP', 'CALT', 'RCSP', 'SPTO', 'YBCH', 'YHBL', 'WEME', 'HOOR', 'BUOR', 'RWBL', 'TRBL', 'BHCO', 'RUBL', 'BRBL', 'GTGR', 'NOWA', 'BAWW', 'PROW', 'TEWA', 'OCWA', 'NAWA', 'MGWA', 'COYE', 'AMRE', 'YEWA', 'CSWA', 'BLPW', 'PAWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'WETA', 'BHGR', 'LAZB', 'INBU', 'HOSP') -- Empty subquery for species outside of placeFilter
                    and S.quality in ('A', 'B')
                    and S.source_id != 'xc:320384' -- Exclude query_rec from results
                ) S
              ) S
              where
                sp_d_pc_i <= 3
              order by
                slp asc, d_pc asc
              limit 31

- CO-ROUTINE 3:
  - CO-ROUTINE 337:
    - SCAN TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - USE TEMP B-TREE FOR RIGHT PART OF ORDER BY
  - SCAN SUBQUERY 337
- MATERIALIZE 334:
  - SCAN 331 CONSTANT ROWS
- SCAN SUBQUERY 3 AS S
- SEARCH SUBQUERY 334 USING AUTOMATIC COVERING INDEX (column1=?)
- USE TEMP B-TREE FOR ORDER BY


Running query...

[19.226s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,sp_d_pc_i,slp,d_pc
0,BRCR,xc:406990,1,2.53,0.688
1,BRCR,xc:352157,2,2.53,0.722
2,BRCR,xc:159463,3,2.53,0.736
3,BEWR,xc:408560,1,2.53,0.608
4,BEWR,xc:230968,2,2.53,0.726
5,BEWR,xc:160213,3,2.53,0.738
6,LISP,xc:322873,1,2.61,0.813
7,LISP,xc:322874,2,2.61,0.817
8,LISP,xc:159653,3,2.61,0.829
9,OATI,xc:350985,1,2.62,0.689


- CO-ROUTINE 14:
  - CO-ROUTINE 16:
    - MATERIALIZE 11:
      - SCAN 10 CONSTANT ROWS
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - SCAN SUBQUERY 11
    - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
      (species=?)
    - USE TEMP B-TREE FOR ORDER BY
  - SCAN SUBQUERY 16
- SCAN SUBQUERY 14 AS S
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.597s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,sp_d_pc_i,slp,d_pc
0,BRCR,xc:406990,1,2.53,0.688
1,BRCR,xc:352157,2,2.53,0.722
2,BRCR,xc:159463,3,2.53,0.736
3,BEWR,xc:408560,1,2.53,0.608
4,BEWR,xc:230968,2,2.53,0.726
5,BEWR,xc:160213,3,2.53,0.738
6,LISP,xc:322873,1,2.61,0.813
7,LISP,xc:322874,2,2.61,0.817
8,LISP,xc:159653,3,2.61,0.829
9,OATI,xc:350985,1,2.62,0.689


In [ ]:
%%sqla -ceng -p3 -Iplan
--# FIXME US crash [fixed via joinOpWithBalancedParens]
                with
                  -- For nested sqlPerSpecies queries
                  S_filter_dist as (
                    select
                      S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.species_order, S.license_type, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks,

                1 - (S.f_preds_0*0.00001311239248025231 + S.f_preds_1*0.0022739178966730833 + S.f_preds_2*0.000004044589331897441 + S.f_preds_3*0.002446443075314164 + S.f_preds_4*0.000007628525054315105 + S.f_preds_5*0.00001568856168887578 + S.f_preds_6*0.000006424799266824266 + S.f_preds_7*2.9643587140526506e-7 + S.f_preds_8*0.0000022504916614707327 + S.f_preds_9*0.00000209387690119911 + S.f_preds_10*0.00002307513750565704 + S.f_preds_11*0.001311492407694459 + S.f_preds_12*0.000015103913938219193 + S.f_preds_13*8.688106731824519e-7 + S.f_preds_14*0.00005533318835659884 + S.f_preds_15*0.000016482756109326147 + S.f_preds_16*0.0005267483065836132 + S.f_preds_17*0.0018814759096130729 + S.f_preds_18*0.0000017237175597983878 + S.f_preds_19*0.00039833024493418634 + S.f_preds_20*0.00000977200579654891 + S.f_preds_21*0.000006569051038241014 + S.f_preds_22*0.000012760150639223866 + S.f_preds_23*0.000009890945875667967 + S.f_preds_24*0.01804490014910698 + S.f_preds_25*0.004844821989536285 + S.f_preds_26*0.00023922703985590488 + S.f_preds_27*0.000015565292414976284 + S.f_preds_28*0.000010527898666623514 + S.f_preds_29*0.005573070142418146 + S.f_preds_30*0.000028921585908392444 + S.f_preds_31*0.0000033035873912012903 + S.f_preds_32*4.100716068933252e-7 + S.f_preds_33*6.601695758945425e-7 + S.f_preds_34*0.0004126813728362322 + S.f_preds_35*0.00003285566344857216 + S.f_preds_36*0.0018572922563180327 + S.f_preds_37*0.0006297582876868546 + S.f_preds_38*0.0001985664275707677 + S.f_preds_39*0.000013924099221185315 + S.f_preds_40*0.00005194657205720432 + S.f_preds_41*0.000042426461732247844 + S.f_preds_42*0.0005176834529265761 + S.f_preds_43*0.01681959442794323 + S.f_preds_44*0.00021062373707536608 + S.f_preds_45*8.841304293127905e-7 + S.f_preds_46*0.000049778231186792254 + S.f_preds_47*0.018970871344208717 + S.f_preds_48*0.0007048664847388864 + S.f_preds_49*0.000028281960112508386 + S.f_preds_50*0.0006428731721825898 + S.f_preds_51*0.0028357082046568394 + S.f_preds_52*0.0000034002564461843576 + S.f_preds_53*0.023662345483899117 + S.f_preds_54*0.00044131523463875055 + S.f_preds_55*0.00241877231746912 + S.f_preds_56*0.00003444610410952009 + S.f_preds_57*0.0014652458485215902 + S.f_preds_58*0.0020906205754727125 + S.f_preds_59*0.000013210412362241186 + S.f_preds_60*0.0001858155446825549 + S.f_preds_61*0.007895379327237606 + S.f_preds_62*0.0000024245182430604473 + S.f_preds_63*0.0000026513437205721857 + S.f_preds_64*0.0009304345585405827 + S.f_preds_65*0.00023366720415651798 + S.f_preds_66*0.005655152257531881 + S.f_preds_67*0.0009726776042953134 + S.f_preds_68*0.000012197562682558782 + S.f_preds_69*1.0011000739496012e-7 + S.f_preds_70*0.00020311045227572322 + S.f_preds_71*0.00025495662703178823 + S.f_preds_72*0.0008810111903585494 + S.f_preds_73*0.00011881050158990547 + S.f_preds_74*0.01844712905585766 + S.f_preds_75*0.0014010248705744743 + S.f_preds_76*6.89977426304722e-8 + S.f_preds_77*0.000001391107161907712 + S.f_preds_78*0.00006117242446634918 + S.f_preds_79*0.010232416912913322 + S.f_preds_80*0.000001088506337509898 + S.f_preds_81*0.023149210959672928 + S.f_preds_82*0.0218129251152277 + S.f_preds_83*0.00027983225299976766 + S.f_preds_84*0.000002296665115864016 + S.f_preds_85*0.00029460497898980975 + S.f_preds_86*0.0011194923426955938 + S.f_preds_87*0.01881752721965313 + S.f_preds_88*0.00012937233259435743 + S.f_preds_89*0.006397274788469076 + S.f_preds_90*0.000016658137610647827 + S.f_preds_91*0.00017631007358431816 + S.f_preds_92*0.00012058904394507408 + S.f_preds_93*0.00005393704850575887 + S.f_preds_94*0.004301635082811117 + S.f_preds_95*0.0000014354379800352035 + S.f_preds_96*0.000005481480002345052 + S.f_preds_97*0.00009395895904162899 + S.f_preds_98*0.00007922713120933622 + S.f_preds_99*1.1477159489459154e-7 + S.f_preds_100*0.0003226808912586421 + S.f_preds_101*0.00006615019083255902 + S.f_preds_102*6.292503371696512e-7 + S.f_preds_103*0.00010970210860250518 + S.f_preds_104*0.0005167542258277535 + S.f_preds_105*0.0000012490669405451627 + S.f_preds_106*0.00039587795617990196 + S.f_preds_107*0.005248239729553461 + S.f_preds_108*1.9826785546683823e-7 + S.f_preds_109*0.00005959632107988 + S.f_preds_110*0.00014260836178436875 + S.f_preds_111*0.00010338414722355083 + S.f_preds_112*0.0000205065243790159 + S.f_preds_113*0.0000020813254195672926 + S.f_preds_114*0.00020459122606553137 + S.f_preds_115*0.002042416948825121 + S.f_preds_116*0.00006533438136102632 + S.f_preds_117*0.000005608823357761139 + S.f_preds_118*0.001865064725279808 + S.f_preds_119*0.00007586278661619872 + S.f_preds_120*0.0004040422209072858 + S.f_preds_121*0.0007578746299259365 + S.f_preds_122*0.0037188904825598 + S.f_preds_123*0.014093589968979359 + S.f_preds_124*0.0002631600946187973 + S.f_preds_125*0.00681808777153492 + S.f_preds_126*3.617186052906618e-7 + S.f_preds_127*0.002112553222104907 + S.f_preds_128*0.0025430587120354176 + S.f_preds_129*0.00016653123020660132 + S.f_preds_130*0.02345724031329155 + S.f_preds_131*0.002475341083481908 + S.f_preds_132*0.00014247142826206982 + S.f_preds_133*0.0005273931892588735 + S.f_preds_134*0.000013719729395234026 + S.f_preds_135*0.0035834198351949453 + S.f_preds_136*0.0000024723869955778355 + S.f_preds_137*0.017516305670142174 + S.f_preds_138*0.00004870363409281708 + S.f_preds_139*0.00013708391634281725 + S.f_preds_140*0.000009344155841972679 + S.f_preds_141*0.00001242540565726813 + S.f_preds_142*0.023233819752931595 + S.f_preds_143*0.000028271842893445864 + S.f_preds_144*0.013134469278156757 + S.f_preds_145*0.0007207064772956073 + S.f_preds_146*0.00003083339106524363 + S.f_preds_147*0.000008576449545216747 + S.f_preds_148*0.016772819682955742 + S.f_preds_149*0.0009123460622504354 + S.f_preds_150*0.012531428597867489 + S.f_preds_151*0.0002033248165389523 + S.f_preds_152*0.00003387439573998563 + S.f_preds_153*0.00012878944107796997 + S.f_preds_154*0.000011766812349378597 + S.f_preds_155*0.0035626592580229044 + S.f_preds_156*0.0000020471034076763317 + S.f_preds_157*0.0000035694906728167553 + S.f_preds_158*0.00015981854812707752 + S.f_preds_159*0.0011637889547273517 + S.f_preds_160*0.00387061876244843 + S.f_preds_161*0.00008862809045240283 + S.f_preds_162*0.00006104543717810884 + S.f_preds_163*0.00011629491928033531 + S.f_preds_164*0.00004587496732710861 + S.f_preds_165*0.0000650517686153762 + S.f_preds_166*0.0017447016434744 + S.f_preds_167*0.021712763234972954 + S.f_preds_168*0.0005890600150451064 + S.f_preds_169*6.537584908983263e-7 + S.f_preds_170*3.564814790024684e-7 + S.f_preds_171*0.0009296662174165249 + S.f_preds_172*0.00006215124449227005 + S.f_preds_173*0.012565955519676208 + S.f_preds_174*0.007263803388923407 + S.f_preds_175*0.0027377239894121885 + S.f_preds_176*0.000008244623131758999 + S.f_preds_177*0.0000023269387838809052 + S.f_preds_178*0.00022972709848545492 + S.f_preds_179*0.00002270613913424313 + S.f_preds_180*2.0221226293415384e-7 + S.f_preds_181*0.000009496605343883857 + S.f_preds_182*0.000005108888672111789 + S.f_preds_183*0.0005137779517099261 + S.f_preds_184*0.000010669865332602058 + S.f_preds_185*0.0009854049421846867 + S.f_preds_186*0.0012607353273779154 + S.f_preds_187*0.0006667971611022949 + S.f_preds_188*0.000014654996448371094 + S.f_preds_189*2.450668716846849e-7 + S.f_preds_190*0.00021140761964488775 + S.f_preds_191*0.00004480480129132047 + S.f_preds_192*0.00010504756937734783 + S.f_preds_193*0.00002219480484200176 + S.f_preds_194*0.0014585570897907019 + S.f_preds_195*0.000023570381017634645 + S.f_preds_196*0.0035868771374225616 + S.f_preds_197*0.000026636558686732315 + S.f_preds_198*5.375615046432358e-7 + S.f_preds_199*0.004330820869654417 + S.f_preds_200*0.000013430114449874964 + S.f_preds_201*0.00009262594539904967 + S.f_preds_202*0.00008550980419386178 + S.f_preds_203*0.00005542610233533196 + S.f_preds_204*0.000022133155653136782 + S.f_preds_205*0.0016019666800275445 + S.f_preds_206*8.774132993494277e-7 + S.f_preds_207*8.674058449287259e-7 + S.f_preds_208*0.00004084217653144151 + S.f_preds_209*0.000011412805179134011 + S.f_preds_210*0.0005518625839613378 + S.f_preds_211*0.00008674130367580801 + S.f_preds_212*0.0000026616432933224132 + S.f_preds_213*0.000023289498130907305 + S.f_preds_214*0.00001417942166881403 + S.f_preds_215*0.00011560232087504119 + S.f_preds_216*0.000008560010428482201 + S.f_preds_217*0.0010935317259281874 + S.f_preds_218*0.00014956091763451695 + S.f_preds_219*0.0005274588475003839 + S.f_preds_220*0.0003844208549708128 + S.f_preds_221*0.008620514534413815 + S.f_preds_222*4.901575039184536e-7 + S.f_preds_223*0.000002932444431280601 + S.f_preds_224*0.00008497372618876398 + S.f_preds_225*0.000006627227776334621 + S.f_preds_226*0.00003267589272581972 + S.f_preds_227*3.307827682874631e-7 + S.f_preds_228*0.003242308972403407 + S.f_preds_229*0.00006843778101028875 + S.f_preds_230*0.000021847368770977482 + S.f_preds_231*0.0000022423150767281186 + S.f_preds_232*0.0026875378098338842 + S.f_preds_233*0.00005942605639575049 + S.f_preds_234*0.000003830415607808391 + S.f_preds_235*0.00003467744681984186 + S.f_preds_236*0.0003422269073780626 + S.f_preds_237*0.00003503493280732073 + S.f_preds_238*0.0000010439758852953673 + S.f_preds_239*0.0000018763837488222634 + S.f_preds_240*0.00012398957915138453 + S.f_preds_241*0.0011635670671239495 + S.f_preds_242*0.0000011317058579152217 + S.f_preds_243*0.015879152342677116 + S.f_preds_244*0.00003913496766472235 + S.f_preds_245*0.000037293462810339406 + S.f_preds_246*0.0000361335369234439 + S.f_preds_247*0.00005024169513490051 + S.f_preds_248*0.0013123423559591174 + S.f_preds_249*0.0000015925746765788062 + S.f_preds_250*0.0003526984073687345 + S.f_preds_251*0.0015677057672291994 + S.f_preds_252*0.000027051797587773763 + S.f_preds_253*0.00004910511779598892 + S.f_preds_254*0.0000018012084410656826 + S.f_preds_255*0.0000886311536305584 + S.f_preds_256*0.0005031937034800649 + S.f_preds_257*0.0002624462067615241 + S.f_preds_258*0.00016602191317360848 + S.f_preds_259*0.00024625565856695175 + S.f_preds_260*0.000034680469980230555 + S.f_preds_261*0.000011574317795748357 + S.f_preds_262*0.0006084603955969214 + S.f_preds_263*0.0000021471892068802845 + S.f_preds_264*0.00001021452590066474 + S.f_preds_265*0.00019133792375214398 + S.f_preds_266*0.000016060286725405604 + S.f_preds_267*0.00014434140757657588 + S.f_preds_268*0.00015234870079439133 + S.f_preds_269*0.000001094846425075957 + S.f_preds_270*0.000011648219697235618 + S.f_preds_271*0.0004886073293164372 + S.f_preds_272*0.0016627265140414238 + S.f_preds_273*0.01329568587243557 + S.f_preds_274*3.138311797101778e-7 + S.f_preds_275*0.0003020265430677682 + S.f_preds_276*6.091020736676e-7 + S.f_preds_277*0.000008524235454387963 + S.f_preds_278*0.00020711110846605152 + S.f_preds_279*0.000003148366204186459 + S.f_preds_280*0.00014961144188418984 + S.f_preds_281*0.000024720802684896626 + S.f_preds_282*0.00008777557377470657 + S.f_preds_283*0.000008208664439735003 + S.f_preds_284*0.000012109705494367518 + S.f_preds_285*6.579759883607039e-7 + S.f_preds_286*0.000012348501513770316 + S.f_preds_287*0.0007436243467964232 + S.f_preds_288*0.000004955889380653389 + S.f_preds_289*8.595026201874134e-7 + S.f_preds_290*0.007464772555977106 + S.f_preds_291*0.0001946113770827651 + S.f_preds_292*0.00007105382974259555 + S.f_preds_293*0.0033582847099751234 + S.f_preds_294*0.0006477180868387222 + S.f_preds_295*3.6187074670124275e-7 + S.f_preds_296*0.0000693765832693316 + S.f_preds_297*0.0000033425619676563656 + S.f_preds_298*0.0000029267844183777925 + S.f_preds_299*0.0000039293249756156 + S.f_preds_300*0.000038189460610738024 + S.f_preds_301*5.310744768394215e-7 + S.f_preds_302*0.0000017443657043259009 + S.f_preds_303*0.000011264039130765013 + S.f_preds_304*0.000007809535418346059 + S.f_preds_305*0.00004670179259846918 + S.f_preds_306*0.00004879724292550236 + S.f_preds_307*0.0015165400691330433 + S.f_preds_308*0.000002572912762843771 + S.f_preds_309*0.0009610670967958868 + S.f_preds_310*0.000015019456441223156 + S.f_preds_311*0.000018154683857574128 + S.f_preds_312*0.00042174625559709966 + S.f_preds_313*0.00000472815827379236 + S.f_preds_314*0.022676438093185425 + S.f_preds_315*5.847223860655504e-7 + S.f_preds_316*0.000004039436589664547 + S.f_preds_317*0.000008301932211907115 + S.f_preds_318*0.000013070617569610476 + S.f_preds_319*0.00009437942935619503 + S.f_preds_320*9.450283044998287e-8 + S.f_preds_321*0.00013990449951961637 + S.f_preds_322*0.00000192821448763425 + S.f_preds_323*0.0000013222486359154573 + S.f_preds_324*0.000052494549890980124 + S.f_preds_325*0.000007503287179133622 + S.f_preds_326*0.0000017911829672812019 + S.f_preds_327*0.00011045519931940362 + S.f_preds_328*0.0022690247278660536 + S.f_preds_329*0.0001165569992735982 + S.f_preds_330*0.0000018839571112039266 + S.f_preds_331*0.000032350140827475116 + S.f_preds_332*0.00002531428799557034 + S.f_preds_333*0.000050617811211850494 + S.f_preds_334*0.0007663340657018125 + S.f_preds_335*0.00013900290650781244 + S.f_preds_336*0.000049367572501068935 + S.f_preds_337*0.000042445550207048655 + S.f_preds_338*0.0001376005238853395 + S.f_preds_339*0.00004797595465788618 + S.f_preds_340*0.000022352229279931635 + S.f_preds_341*0.0002962714934255928 + S.f_preds_342*0.00004398026067065075 + S.f_preds_343*0.01037600077688694 + S.f_preds_344*0.00004516577610047534 + S.f_preds_345*0.0006199416238814592 + S.f_preds_346*0.0000012476348274503835 + S.f_preds_347*0.000004629550403478788 + S.f_preds_348*0.023418789729475975 + S.f_preds_349*0.00000777882360125659 + S.f_preds_350*0.000029099664970999584 + S.f_preds_351*0.000014523630852636416 + S.f_preds_352*0.0000023763323042658158 + S.f_preds_353*0.0005294025759212673 + S.f_preds_354*0.000005048519142292207 + S.f_preds_355*0.0000032741434097260935 + S.f_preds_356*0.000007565846317447722 + S.f_preds_357*0.00004547514981823042 + S.f_preds_358*0.0002636595454532653 + S.f_preds_359*1.800865874201918e-7 + S.f_preds_360*0.00001865157719294075 + S.f_preds_361*0.000376529002096504 + S.f_preds_362*0.0000027501305339683313 + S.f_preds_363*0.00030277753830887377 + S.f_preds_364*0.00005115754538564943 + S.f_preds_365*0.0012324352283030748 + S.f_preds_366*0.00018544473277870566 + S.f_preds_367*0.00005313345172908157 + S.f_preds_368*0.000027433607101556845 + S.f_preds_369*0.00020436079648789018 + S.f_preds_370*0.0006155724404379725 + S.f_preds_371*0.0000035860705338564003 + S.f_preds_372*0.000004185253146715695 + S.f_preds_373*0.00005271961708785966 + S.f_preds_374*0.000002029272991421749 + S.f_preds_375*0.00018188006652053446 + S.f_preds_376*0.000012025258911307901 + S.f_preds_377*0.0014208576176315546 + S.f_preds_378*9.447777671311997e-8 + S.f_preds_379*0.00006892470992170274 + S.f_preds_380*0.00003242859020247124 + S.f_preds_381*0.0007282765000127256 + S.f_preds_382*0.00375916319899261 + S.f_preds_383*0.0000028596371066669235 + S.f_preds_384*0.000009051633242052048 + S.f_preds_385*0.0011416266206651926 + S.f_preds_386*0.0000043961154005955905 + S.f_preds_387*0.000008625760528957471 + S.f_preds_388*0.00009348194726044312 + S.f_preds_389*0.000010609912351355888 + S.f_preds_390*0.0005257150623947382 + S.f_preds_391*0.00009466418850934133 + S.f_preds_392*0.000017193688108818606 + S.f_preds_393*0.007605379913002253 + S.f_preds_394*0.0004980259109288454 + S.f_preds_395*0.0007890399429015815 + S.f_preds_396*2.0968185410197293e-8 + S.f_preds_397*8.087992569016933e-7 + S.f_preds_398*6.560824772350315e-7 + S.f_preds_399*0.00014008089783601463 + S.f_preds_400*0.0006039145519025624 + S.f_preds_401*0.0000071473286880063824 + S.f_preds_402*0.00004870027623837814 + S.f_preds_403*0.000056747336202533916 + S.f_preds_404*0.0000023145769318944076 + S.f_preds_405*0.0008232579566538334 + S.f_preds_406*9.881814548862167e-7 + S.f_preds_407*0.0000017835916423791787 + S.f_preds_408*0.004612771328538656 + S.f_preds_409*0.000830142991617322 + S.f_preds_410*7.730937312544484e-8 + S.f_preds_411*0.012341471388936043 + S.f_preds_412*0.002177329733967781 + S.f_preds_413*0.00037074810825288296 + S.f_preds_414*0.0017332970164716244 + S.f_preds_415*0.0013398604933172464 + S.f_preds_416*0.0000012682459100687993 + S.f_preds_417*3.9440632804144116e-7 + S.f_preds_418*0.0000511453217768576 + S.f_preds_419*0.0000014611218830395956 + S.f_preds_420*1.3488853767285036e-7 + S.f_preds_421*0.0011070219334214926 + S.f_preds_422*0.000172565778484568 + S.f_preds_423*0.00000847523278935114 + S.f_preds_424*0.0000021629989532812033 + S.f_preds_425*0.00015388638712465763 + S.f_preds_426*1.244669363131834e-7 + S.f_preds_427*5.292966989145498e-7 + S.f_preds_428*0.0026793526485562325 + S.f_preds_429*0.00000242397072724998 + S.f_preds_430*0.0000017537361145514296 + S.f_preds_431*0.00007314266986213624 + S.f_preds_432*0.015580660663545132 + S.f_preds_433*0.00003077931614825502 + S.f_preds_434*0.001671489211730659 + S.f_preds_435*0.0051424503326416016 + S.f_preds_436*0.0000012573142385008396 + S.f_preds_437*0.0000017696989971227595 + S.f_preds_438*0.00003416148683754727 + S.f_preds_439*0.0008575073443353176 + S.f_preds_440*0.0004498166963458061 + S.f_preds_441*0.0014703682390972972 + S.f_preds_442*0.000014719891623826697 + S.f_preds_443*0.00017386121908202767 + S.f_preds_444*0.000073970302764792 + S.f_preds_445*0.00011574722157092765 + S.f_preds_446*0.005045986268669367 + S.f_preds_447*0.004470773506909609 + S.f_preds_448*0.000002035713578152354 + S.f_preds_449*0.000043419364374130964 + S.f_preds_450*0.012585037387907505 + S.f_preds_451*0.000001728835741232615 + S.f_preds_452*0.00000100265071978356 + S.f_preds_453*0.0015500244917348027 + S.f_preds_454*0.0013294772943481803 + S.f_preds_455*0.0010233717039227486 + S.f_preds_456*0.0010060714557766914 + S.f_preds_457*0.000006806942565162899 + S.f_preds_458*6.035304522811202e-7 + S.f_preds_459*0.00020313254208303988 + S.f_preds_460*0.003389031160622835 + S.f_preds_461*0.0008531991043128073 + S.f_preds_462*0.00033116189297288656 + S.f_preds_463*0.000012784821592504159 + S.f_preds_464*0.0000283551707980223 + S.f_preds_465*0.0006966462824493647 + S.f_preds_466*0.00002789941027003806 + S.f_preds_467*0.0005911355256102979 + S.f_preds_468*0.002994079142808914 + S.f_preds_469*0.0000017838807480075047 + S.f_preds_470*2.4400554821113474e-7 + S.f_preds_471*5.014719022256031e-7 + S.f_preds_472*0.000011036318028345704 + S.f_preds_473*3.725356805261981e-7 + S.f_preds_474*1.266375591058022e-7 + S.f_preds_475*0.000018449984054313973 + S.f_preds_476*0.00003187067704857327 + S.f_preds_477*0.005098472349345684 + S.f_preds_478*0.0035877684131264687 + S.f_preds_479*0.00013991973537486047 + S.f_preds_480*0.00002065905755443964 + S.f_preds_481*9.214570582116721e-7 + S.f_preds_482*0.00005737582250731066 + S.f_preds_483*1.1891671647390467e-7 + S.f_preds_484*5.138089136380586e-7 + S.f_preds_485*0.00003446516711846925 + S.f_preds_486*0.00003241106605855748 + S.f_preds_487*0.00007031692803138867 + S.f_preds_488*0.00004199549584882334 + S.f_preds_489*0.00007063855446176603 + S.f_preds_490*1.3241979957001604e-7 + S.f_preds_491*0.0014177433913573623 + S.f_preds_492*0.000021480864234035835 + S.f_preds_493*0.00009588408283889294 + S.f_preds_494*0.0008136858814395964 + S.f_preds_495*0.00000723950142855756 + S.f_preds_496*0.000041920222429325804 + S.f_preds_497*0.000004934684966428904 + S.f_preds_498*0.01999528706073761 + S.f_preds_499*0.000009792875971470494 + S.f_preds_500*0.006254177540540695 + S.f_preds_501*0.0019237757660448551 + S.f_preds_502*0.0000336199882440269 + S.f_preds_503*0.00011587099288590252 + S.f_preds_504*5.696571747648704e-7 + S.f_preds_505*0.0000030567177873308538 + S.f_preds_506*0.000029180990168242715 + S.f_preds_507*0.000054005424317438155 + S.f_preds_508*0.0003109595272690058 + S.f_preds_509*0.000013896006748836953 + S.f_preds_510*0.00003571807610569522 + S.f_preds_511*0.002394929062575102 + S.f_preds_512*0.0001247953186975792 + S.f_preds_513*0.00014427122368942946 + S.f_preds_514*0.00009447086631553248 + S.f_preds_515*0.000035395132726989686 + S.f_preds_516*0.0030862544663250446 + S.f_preds_517*0.0020114730577915907 + S.f_preds_518*0.0015699266223236918 + S.f_preds_519*0.000022298700059764087 + S.f_preds_520*0.017703445628285408 + S.f_preds_521*0.0014080592663958669 + S.f_preds_522*0.005447646602988243 + S.f_preds_523*0.01645824871957302 + S.f_preds_524*0.000027788570150732994 + S.f_preds_525*0.00001340962899121223 + S.f_preds_526*0.000019065997548750602 + S.f_preds_527*0.000009252670679416042 + S.f_preds_528*0.0000025471656499576056 + S.f_preds_529*7.461294160293619e-8 + S.f_preds_530*0.00017533496429678053 + S.f_preds_531*0.000027902546207769774 + S.f_preds_532*0.000016202477127080783 + S.f_preds_533*0.00020475717610679567 + S.f_preds_534*0.0004499423084780574 + S.f_preds_535*0.0006966591463424265 + S.f_preds_536*0.000550146447494626 + S.f_preds_537*0.00016486116510350257 + S.f_preds_538*0.000007280455974978395 + S.f_preds_539*0.000010827132427948527 + S.f_preds_540*0.0000876995109138079 + S.f_preds_541*0.0003642363881226629 + S.f_preds_542*0.0000012616019375855103 + S.f_preds_543*0.0000019034998786082724 + S.f_preds_544*0.00005276263254927471 + S.f_preds_545*0.00005496077210409567 + S.f_preds_546*0.00027376782963983715 + S.f_preds_547*0.000005710881850973237 + S.f_preds_548*8.988230604245473e-9 + S.f_preds_549*0.0000016730429024391924 + S.f_preds_550*0.00007679287227801979 + S.f_preds_551*0.0000016976399592749658 + S.f_preds_552*0.000004411753707245225 + S.f_preds_553*0.000005111335212859558 + S.f_preds_554*0.00016011434490792453 + S.f_preds_555*0.0000029296325010363944 + S.f_preds_556*0.00025140767684206367 + S.f_preds_557*0.00005884619167773053 + S.f_preds_558*0.0015645562671124935 + S.f_preds_559*0.00020754926663357764 + S.f_preds_560*0.00025478313909843564 + S.f_preds_561*0.002211499260738492 + S.f_preds_562*6.609280376324023e-7 + S.f_preds_563*0.0010862791677936912 + S.f_preds_564*0.000012614359548024368 + S.f_preds_565*0.00017907589790411294 + S.f_preds_566*0.0000037052925563330064 + S.f_preds_567*0.00010445003135828301 + S.f_preds_568*0.017669612541794777 + S.f_preds_569*0.00000553270638192771 + S.f_preds_570*0.00006034673788235523 + S.f_preds_571*0.00039125533658079803 + S.f_preds_572*0.000007338504929066403 + S.f_preds_573*0.000122875499073416 + S.f_preds_574*2.4581188995398406e-7 + S.f_preds_575*0.000011374868336133659 + S.f_preds_576*0.00008426068961853161 + S.f_preds_577*6.751050705133821e-7 + S.f_preds_578*0.0000064212285906251054 + S.f_preds_579*0.00005932396015850827 + S.f_preds_580*0.0004973491886630654 + S.f_preds_581*5.367184030546923e-7 + S.f_preds_582*0.000743536336813122 + S.f_preds_583*0.0000017930664171217359 + S.f_preds_584*0.0006554555729962885 + S.f_preds_585*0.0000037011327549407724 + S.f_preds_586*0.0002179535513278097 + S.f_preds_587*0.00003099400782957673 + S.f_preds_588*0.000004918352260574466 + S.f_preds_589*0.000009350197615276556 + S.f_preds_590*0.000016639018213027157 + S.f_preds_591*0.00010717164695961401 + S.f_preds_592*0.000023528449673904106 + S.f_preds_593*0.000001007815853881766 + S.f_preds_594*0.000005481845619215164 + S.f_preds_595*0.00015956266724970192 + S.f_preds_596*0.0002082450082525611 + S.f_preds_597*0.00020021837553940713 + S.f_preds_598*0.0010172325419262052 + S.f_preds_599*0.0000010955390052913572 + S.f_preds_600*0.0000026854265797737753 + S.f_preds_601*0.0015208498807623982 + S.f_preds_602*0.000006445457074732985 + S.f_preds_603*0.000020037914509885013 + S.f_preds_604*1.9325770495015604e-7 + S.f_preds_605*0.0000010705190334192594 + S.f_preds_606*0.000028866608772659674 + S.f_preds_607*0.00010479458433110267 + S.f_preds_608*0.010287586599588394 + S.f_preds_609*0.0012989990646019578 + S.f_preds_610*0.000003944566287827911 + S.f_preds_611*2.8879350466581855e-8 + S.f_preds_612*0.0000069853654167673085 + S.f_preds_613*0.000021991474568494596 + S.f_preds_614*0.00007703194569330662 + S.f_preds_615*0.00023750816762913018 + S.f_preds_616*0.00944328773766756 + S.f_preds_617*0.0000010323501555831172 + S.f_preds_618*1.501768736034137e-7 + S.f_preds_619*0.00007150989404181018 + S.f_preds_620*0.0000061345799622358754 + S.f_preds_621*0.0004368036170490086 + S.f_preds_622*8.976332992460812e-7 + S.f_preds_623*0.001628484926186502 + S.f_preds_624*0.000003928264959540684 + S.f_preds_625*0.0009367550374008715 + S.f_preds_626*0.0000026312400223105215 + S.f_preds_627*0.00006375551311066374 + S.f_preds_628*5.137369498697808e-7 + S.f_preds_629*0.00001137478193413699 + S.f_preds_630*0.000017766524251783267 + S.f_preds_631*0.003897836897522211 + S.f_preds_632*0.008055249229073524 + S.f_preds_633*0.00241384981200099 + S.f_preds_634*0.00003595540329115465 + S.f_preds_635*0.00005645097917295061 + S.f_preds_636*0.00006621936336159706 + S.f_preds_637*0.014510611072182655 + S.f_preds_638*1.050159710302978e-7 + S.f_preds_639*0.019783876836299896 + S.f_preds_640*0.008949201554059982 + S.f_preds_641*0.00024250347632914782 + S.f_preds_642*0.00010105391265824437 + S.f_preds_643*0.00004060524588567205 + S.f_preds_644*0.0001331030362052843 + S.f_preds_645*8.045903427955636e-7 + S.f_preds_646*0.00001027736743708374 + S.f_preds_647*1.7172181898672534e-8 + S.f_preds_648*0.0033125479239970446 + S.f_preds_649*0.0000016601031802565558 + S.f_preds_650*0.00005077445530332625 + S.f_preds_651*0.0002987897605635226 + S.f_preds_652*0.002370136557146907 + S.f_preds_653*0.0005649944650940597 + S.f_preds_654*0.000028247362934052944 + S.f_preds_655*0.000047565263230353594 + S.f_preds_656*0.00001847093335527461 + S.f_preds_657*0.000002787693574646255 + S.f_preds_658*0.000005108099685458001 + S.f_preds_659*0.0010631985496729612 + S.f_preds_660*0.00004261540743755177 + S.f_preds_661*0.0002864064881578088 + S.f_preds_662*0.00009470334771322086 + S.f_preds_663*0.00006371989729814231 + S.f_preds_664*4.832561444345629e-7 + S.f_preds_665*0.0000020821610178245464 + S.f_preds_666*0.0002292780700372532 + S.f_preds_667*3.029888162586758e-8 + S.f_preds_668*0.00005065908044343814 + S.f_preds_669*0.00020125712035223842 + S.f_preds_670*0.00047489258577115834 + S.f_preds_671*3.227335909627982e-8 + S.f_preds_672*0.0001816168805817142 + S.f_preds_673*0.0000014918402939656517 + S.f_preds_674*0.000007308628937607864 + S.f_preds_675*0.00004637502206605859 + S.f_preds_676*0.00005088275429443456 + S.f_preds_677*0.0004129812878090888 + S.f_preds_678*0.02267596498131752 + S.f_preds_679*0.00004278955384506844 + S.f_preds_680*0.0005632776883430779 + S.f_preds_681*0.0000029753814487776253 + S.f_preds_682*0.0010443839710205793 + S.f_preds_683*0.000003015262109329342 + S.f_preds_684*0.0000015277154261639225 + S.f_preds_685*0.00008885315764928237 + S.f_preds_686*0.0001388078526360914 + S.f_preds_687*0.00009291117748944089 + S.f_preds_688*0.00005038944073021412 + S.f_preds_689*4.75471949812345e-7 + S.f_preds_690*1.248018861588207e-7 + S.f_preds_691*0.001185665954835713 + S.f_preds_692*0.00003074324922636151 + S.f_preds_693*9.277101753468742e-7 + S.f_preds_694*0.00022056890884414315 + S.f_preds_695*0.007895344868302345 + S.f_preds_696*0.013903992250561714 + S.f_preds_697*0.00014689314411953092 + S.f_preds_698*0.000002063769670712645 + S.f_preds_699*0.0009233330492861569 + S.f_preds_700*0.000016178952137124725 + S.f_preds_701*0.0010267875622957945 + S.f_preds_702*0.00005498975224327296 + S.f_preds_703*0.00013355587725527585 + S.f_preds_704*0.00002518714427424129 + S.f_preds_705*0.0001367626537103206 + S.f_preds_706*0.0002352102310396731 + S.f_preds_707*0.00003683730392367579 + S.f_preds_708*0.00019189013983123004 + S.f_preds_709*0.00040489446837455034 + S.f_preds_710*0.000008120999154925812 + S.f_preds_711*0.000005265692379907705 + S.f_preds_712*0.000013629485692945309 + S.f_preds_713*0.00010411084804218262 + S.f_preds_714*0.00010439017933094874 + S.f_preds_715*0.00011334530427120626 + S.f_preds_716*0.0004283014277461916 + S.f_preds_717*0.0003968038654420525 + S.f_preds_718*0.00012781038822140545 + S.f_preds_719*0.010870814323425293 + S.f_preds_720*0.00004498447378864512 + S.f_preds_721*0.00006518877489725128 + S.f_preds_722*0.000029586220989585854 + S.f_preds_723*0.0000569157928111963 + S.f_preds_724*0.000020265539205865934 + S.f_preds_725*0.00003896027192240581 + S.f_preds_726*0.000001130614123212581 + S.f_preds_727*0.0017423968529328704 + S.f_preds_728*0.0009984647622331977 + S.f_preds_729*0.00020527135347947478 + S.f_preds_730*0.00010141624807147309 + S.f_preds_731*0.0000013939235259385896 + S.f_preds_732*0.0017145039746537805 + S.f_preds_733*0.00048482915735803545 + S.f_preds_734*0.0003125153889413923 + S.f_preds_735*0.000006966789896978298 + S.f_preds_736*0.00024413960636593401 + S.f_preds_737*0.000002054856395261595 + S.f_preds_738*0.00002746743120951578) / S.norm_f_preds / 0.11221887324529495
               as d_pc
                    from search_recs S
                    where true
                      -- Filters duplicated above (in filteredSpecies query)
                      and S.species in ('RODO', 'BTPI', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'ALHU', 'WEGU', 'CAGU', 'CATE', 'DCCO', 'BRPE', 'GBHE', 'TUVU', 'OSPR', 'NOHA', 'SSHA', 'COHA', 'RSHA', 'RTHA', 'BANO', 'GHOW', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'RMPA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'HUVI', 'CAVI', 'WAVI', 'REVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'TRES', 'VGSW', 'BARS', 'CBCH', 'BUSH', 'RBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'CEDW', 'HOFI', 'PUFI', 'PISI', 'LEGO', 'AMGO', 'CHSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'CALT', 'SPTO', 'HOOR', 'BUOR', 'BHCO', 'BRBL', 'OCWA', 'NAWA', 'COYE', 'AMRE', 'NOPA', 'YEWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'SUTA', 'WETA', 'RBGR', 'BHGR', 'LAZB', 'INBU', 'HOSP')
                      and S.quality in ('A', 'B')
                      and S.species not in ()
                      and (false
          or S.species_species_group not in ()
          or S.species in ()
        )
                      and S.source_id not in ()
                      and S.source_id != 'xc:370896' -- Exclude query_rec from results
                  )
                select *
                from (
                  -- Must wrap subqueries in 'select * from (...)' else union complains about nested order by
                  select * from (
                    select *, 3.754216598942222 as slp
                    from S_filter_dist
                    where species = 'INBU'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 3.7657945828461847 as slp
                    from S_filter_dist
                    where species = 'BGGN'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 3.8252525896860767 as slp
                    from S_filter_dist
                    where species = 'BHCO'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 3.829855023828165 as slp
                    from S_filter_dist
                    where species = 'CBCH'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 3.9228879742019362 as slp
                    from S_filter_dist
                    where species = 'TOWA'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 4.014892174340933 as slp
                    from S_filter_dist
                    where species = 'AMRE'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 4.033995990083049 as slp
                    from S_filter_dist
                    where species = 'RCKI'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 4.044623079285102 as slp
                    from S_filter_dist
                    where species = 'BUSH'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 4.142748203512467 as slp
                    from S_filter_dist
                    where species = 'EUST'
                    order by d_pc asc
                    limit 3
                  ) union all select * from (
                    select *, 4.161724834846471 as slp
                    from S_filter_dist
                    where species = 'NAWA'
                    order by d_pc asc
                    limit 3
                  )
                )
                order by slp asc, d_pc asc

- CO-ROUTINE 15:
  - CO-ROUTINE 17:
    - MATERIALIZE 11:
      - SCAN 11 CONSTANT ROWS
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - SCAN SUBQUERY 11
    - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
      (species=?)
    - USE TEMP B-TREE FOR ORDER BY
  - SCAN SUBQUERY 17
- SCAN SUBQUERY 15 AS S
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.683s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,slp,d_pc,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,license_type,species_order,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks
0,4.32,0.268,05-07,SAPH,TC006253,xc:139922,Not specified,1,0,[],TC006253,3.0,by-nc-sa,Passeriformes,[],0.422,"Wyoming, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"University of Wyoming (near Laramie), Albany, Wyoming, United States",Not specified,Not specified,221184,-106.0,Not specified,10.00,GABRIEL LEITE,yes,1,Wyoming,1 (mono),//www.xeno-canto.org/139922/download,0.091,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,B,2013-05-07 00:00:00.000000,10:30,48000 (Hz),Not specified,https://www.xeno-canto.org/139922,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,10.0,2200 m,"Tyrant Flycatchers: Pewees, Kingbirds, and Allies","University of Wyoming (near Laramie), Albany",Say's Phoebe,14633.0,22050,"University of Wyoming (near Laramie), Albany, Wyoming",41.3,139922,"cache/audio/xc/data/SAPH/139922/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,5.0,"cache/audio/xc/data/SAPH/139922/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",SAPH,,Say's Phoebe - SAPH,call,
1,4.32,0.326,03-30,SAPH,TC006253,xc:144104,Not specified,1,1,"[""VEFL""]",TC006253,3.0,by-nc-sa,Passeriformes,"[""Vermilion Flycatcher (Pyrocephalus rubinus)""]",0.422,"California, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Blythe, Riverside, California, United States",Not specified,Not specified,221184,-115.0,Not specified,10.00,Lauren Harter,yes,1,California,2 (stereo),//www.xeno-canto.org/144104/download,0.115,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,A,2013-03-30 00:00:00.000000,06:15,48000 (Hz),Not specified,https://www.xeno-canto.org/144104,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,6.0,80 m,"Tyrant Flycatchers: Pewees, Kingbirds, and Allies","Blythe, Riverside",Say's Phoebe,14633.0,22050,"Blythe, Riverside, California",33.7,144104,"cache/audio/xc/data/SAPH/144104/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,3.0,"cache/audio/xc/data/SAPH/144104/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",SAPH,saya,Say's Phoebe - SAPH,song,"Male singing just after sunrise, perched on top of a house."
2,4.32,0.350,03-29,SAPH,TC006253,xc:174095,Not specified,1,1,"[""NOFL""]",TC006253,4.0,by-nc-sa,Passeriformes,"[""Northern Flicker (Colaptes auratus)""]",0.422,"New Mexico, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Sandia Crest ski area, Cibola National Forest, Bernalillo County, New Mexico, United States",Not specified,Not specified,221184,-106.0,Not specified,10.00,Paul Marvin,no,1,New Mexico,1 (mono),//www.xeno-canto.org/174095/download,0.095,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,B,2012-03-29 00:00:00.000000,10:10,48000 (Hz),Not specified,https://www.xeno-canto.org/174095,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,10.0,2600 m,"Tyrant Flycatchers: Pewees, King

- CO-ROUTINE 14:
  - CO-ROUTINE 16:
    - MATERIALIZE 11:
      - SCAN 10 CONSTANT ROWS
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - SCAN SUBQUERY 11
    - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
      (species=?)
    - USE TEMP B-TREE FOR ORDER BY
  - SCAN SUBQUERY 16
- SCAN SUBQUERY 14 AS S
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.597s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,species,source_id,sp_d_pc_i,slp,d_pc
0,BRCR,xc:406990,1,2.53,0.688
1,BRCR,xc:352157,2,2.53,0.722
2,BRCR,xc:159463,3,2.53,0.736
3,BEWR,xc:408560,1,2.53,0.608
4,BEWR,xc:230968,2,2.53,0.726
5,BEWR,xc:160213,3,2.53,0.738
6,LISP,xc:322873,1,2.61,0.813
7,LISP,xc:322874,2,2.61,0.817
8,LISP,xc:159653,3,2.61,0.829
9,OATI,xc:350985,1,2.62,0.689


# Revert window query -> union query

In [ ]:
%%sqla -ceng -Iplan -p3
--# Union query, from before window query + other improvements
select
  S.*,
  coalesce(slp.slp, 1e38) as slp
--# Must select * from (...) else union complains about nested order by
from (select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'NOWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
) union all select * from (
  select
    S.*,
    1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1) / S.norm_f_preds / Q.norm_f_preds as d_pc
  from search_recs S
    left join (select * from search_recs where source_id = 'xc:417354') Q on true --# Only 1 row in Q
  where true
    and S.species = 'WIWA'
    --# Empty subquery for species outside of placeFilter
    and S.quality in ('A', 'B')
    and S.source_id != 'xc:417354' --# Exclude query_rec from results
  order by
    d_pc asc
  limit 3
)) S
  left join (
    select column1 as species, column2 as slp from (values ('NOWA', 2.7278727231365987), ('WIWA', 2.826477946769426))
  ) slp on S.species = slp.species
order by
  slp asc,
  d_pc asc
limit 31

- CO-ROUTINE 15:
  - CO-ROUTINE 17:
    - MATERIALIZE 11:
      - SCAN 11 CONSTANT ROWS
    - SEARCH TABLE search_recs USING INDEX ix_search_recs__source_id (source_id=?)
    - SCAN SUBQUERY 11
    - SEARCH TABLE search_recs AS S USING INDEX ix_search_recs__species__species_species_group__quality__source_id
      (species=?)
    - USE TEMP B-TREE FOR ORDER BY
  - SCAN SUBQUERY 17
- SCAN SUBQUERY 15 AS S
- USE TEMP B-TREE FOR ORDER BY


Running query...

[0.683s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,slp,d_pc,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,license_type,species_order,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks
0,4.32,0.268,05-07,SAPH,TC006253,xc:139922,Not specified,1,0,[],TC006253,3.0,by-nc-sa,Passeriformes,[],0.422,"Wyoming, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"University of Wyoming (near Laramie), Albany, Wyoming, United States",Not specified,Not specified,221184,-106.0,Not specified,10.00,GABRIEL LEITE,yes,1,Wyoming,1 (mono),//www.xeno-canto.org/139922/download,0.091,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,B,2013-05-07 00:00:00.000000,10:30,48000 (Hz),Not specified,https://www.xeno-canto.org/139922,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,10.0,2200 m,"Tyrant Flycatchers: Pewees, Kingbirds, and Allies","University of Wyoming (near Laramie), Albany",Say's Phoebe,14633.0,22050,"University of Wyoming (near Laramie), Albany, Wyoming",41.3,139922,"cache/audio/xc/data/SAPH/139922/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,5.0,"cache/audio/xc/data/SAPH/139922/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",SAPH,,Say's Phoebe - SAPH,call,
1,4.32,0.326,03-30,SAPH,TC006253,xc:144104,Not specified,1,1,"[""VEFL""]",TC006253,3.0,by-nc-sa,Passeriformes,"[""Vermilion Flycatcher (Pyrocephalus rubinus)""]",0.422,"California, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Blythe, Riverside, California, United States",Not specified,Not specified,221184,-115.0,Not specified,10.00,Lauren Harter,yes,1,California,2 (stereo),//www.xeno-canto.org/144104/download,0.115,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,A,2013-03-30 00:00:00.000000,06:15,48000 (Hz),Not specified,https://www.xeno-canto.org/144104,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,6.0,80 m,"Tyrant Flycatchers: Pewees, Kingbirds, and Allies","Blythe, Riverside",Say's Phoebe,14633.0,22050,"Blythe, Riverside, California",33.7,144104,"cache/audio/xc/data/SAPH/144104/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,3.0,"cache/audio/xc/data/SAPH/144104/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",SAPH,saya,Say's Phoebe - SAPH,song,"Male singing just after sunrise, perched on top of a house."
2,4.32,0.350,03-29,SAPH,TC006253,xc:174095,Not specified,1,1,"[""NOFL""]",TC006253,4.0,by-nc-sa,Passeriformes,"[""Northern Flicker (Colaptes auratus)""]",0.422,"New Mexico, United States",14633.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Sandia Crest ski area, Cibola National Forest, Bernalillo County, New Mexico, United States",Not specified,Not specified,221184,-106.0,Not specified,10.00,Paul Marvin,no,1,New Mexico,1 (mono),//www.xeno-canto.org/174095/download,0.095,SAPH,Tyrannidae (Tyrant Flycatchers),United States,128000 (bps),xc,53,Sayornis saya,16,B,2012-03-29 00:00:00.000000,10:10,48000 (Hz),Not specified,https://www.xeno-canto.org/174095,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,10.0,2600 m,"Tyrant Flycatchers: Pewees, King

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3
with
                --# For sqlCosineDist ('Q' = query_rec)
                Q as (
                  select *
                  from search_recs
                  where source_id = 'xc:72445'
                  limit 1 --# Should always be ≤1, but safeguard perf in case of data bugs
                ),
                --# For nested sqlPerSpecies queries
                search_recs_filtered_and_dist as (
                  select
                    S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.license_type, S.species_order, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks,

              1 - (S.f_preds_0*Q.f_preds_0 + S.f_preds_1*Q.f_preds_1 + S.f_preds_2*Q.f_preds_2 + S.f_preds_3*Q.f_preds_3 + S.f_preds_4*Q.f_preds_4 + S.f_preds_5*Q.f_preds_5 + S.f_preds_6*Q.f_preds_6 + S.f_preds_7*Q.f_preds_7 + S.f_preds_8*Q.f_preds_8 + S.f_preds_9*Q.f_preds_9 + S.f_preds_10*Q.f_preds_10 + S.f_preds_11*Q.f_preds_11 + S.f_preds_12*Q.f_preds_12 + S.f_preds_13*Q.f_preds_13 + S.f_preds_14*Q.f_preds_14 + S.f_preds_15*Q.f_preds_15 + S.f_preds_16*Q.f_preds_16 + S.f_preds_17*Q.f_preds_17 + S.f_preds_18*Q.f_preds_18 + S.f_preds_19*Q.f_preds_19 + S.f_preds_20*Q.f_preds_20 + S.f_preds_21*Q.f_preds_21 + S.f_preds_22*Q.f_preds_22 + S.f_preds_23*Q.f_preds_23 + S.f_preds_24*Q.f_preds_24 + S.f_preds_25*Q.f_preds_25 + S.f_preds_26*Q.f_preds_26 + S.f_preds_27*Q.f_preds_27 + S.f_preds_28*Q.f_preds_28 + S.f_preds_29*Q.f_preds_29 + S.f_preds_30*Q.f_preds_30 + S.f_preds_31*Q.f_preds_31 + S.f_preds_32*Q.f_preds_32 + S.f_preds_33*Q.f_preds_33 + S.f_preds_34*Q.f_preds_34 + S.f_preds_35*Q.f_preds_35 + S.f_preds_36*Q.f_preds_36 + S.f_preds_37*Q.f_preds_37 + S.f_preds_38*Q.f_preds_38 + S.f_preds_39*Q.f_preds_39 + S.f_preds_40*Q.f_preds_40 + S.f_preds_41*Q.f_preds_41 + S.f_preds_42*Q.f_preds_42 + S.f_preds_43*Q.f_preds_43 + S.f_preds_44*Q.f_preds_44 + S.f_preds_45*Q.f_preds_45 + S.f_preds_46*Q.f_preds_46 + S.f_preds_47*Q.f_preds_47 + S.f_preds_48*Q.f_preds_48 + S.f_preds_49*Q.f_preds_49 + S.f_preds_50*Q.f_preds_50 + S.f_preds_51*Q.f_preds_51 + S.f_preds_52*Q.f_preds_52 + S.f_preds_53*Q.f_preds_53 + S.f_preds_54*Q.f_preds_54 + S.f_preds_55*Q.f_preds_55 + S.f_preds_56*Q.f_preds_56 + S.f_preds_57*Q.f_preds_57 + S.f_preds_58*Q.f_preds_58 + S.f_preds_59*Q.f_preds_59 + S.f_preds_60*Q.f_preds_60 + S.f_preds_61*Q.f_preds_61 + S.f_preds_62*Q.f_preds_62 + S.f_preds_63*Q.f_preds_63 + S.f_preds_64*Q.f_preds_64 + S.f_preds_65*Q.f_preds_65 + S.f_preds_66*Q.f_preds_66 + S.f_preds_67*Q.f_preds_67 + S.f_preds_68*Q.f_preds_68 + S.f_preds_69*Q.f_preds_69 + S.f_preds_70*Q.f_preds_70 + S.f_preds_71*Q.f_preds_71 + S.f_preds_72*Q.f_preds_72 + S.f_preds_73*Q.f_preds_73 + S.f_preds_74*Q.f_preds_74 + S.f_preds_75*Q.f_preds_75 + S.f_preds_76*Q.f_preds_76 + S.f_preds_77*Q.f_preds_77 + S.f_preds_78*Q.f_preds_78 + S.f_preds_79*Q.f_preds_79 + S.f_preds_80*Q.f_preds_80 + S.f_preds_81*Q.f_preds_81 + S.f_preds_82*Q.f_preds_82 + S.f_preds_83*Q.f_preds_83 + S.f_preds_84*Q.f_preds_84 + S.f_preds_85*Q.f_preds_85 + S.f_preds_86*Q.f_preds_86 + S.f_preds_87*Q.f_preds_87 + S.f_preds_88*Q.f_preds_88 + S.f_preds_89*Q.f_preds_89 + S.f_preds_90*Q.f_preds_90 + S.f_preds_91*Q.f_preds_91 + S.f_preds_92*Q.f_preds_92 + S.f_preds_93*Q.f_preds_93 + S.f_preds_94*Q.f_preds_94 + S.f_preds_95*Q.f_preds_95 + S.f_preds_96*Q.f_preds_96 + S.f_preds_97*Q.f_preds_97 + S.f_preds_98*Q.f_preds_98 + S.f_preds_99*Q.f_preds_99 + S.f_preds_100*Q.f_preds_100 + S.f_preds_101*Q.f_preds_101 + S.f_preds_102*Q.f_preds_102 + S.f_preds_103*Q.f_preds_103 + S.f_preds_104*Q.f_preds_104 + S.f_preds_105*Q.f_preds_105 + S.f_preds_106*Q.f_preds_106 + S.f_preds_107*Q.f_preds_107 + S.f_preds_108*Q.f_preds_108 + S.f_preds_109*Q.f_preds_109 + S.f_preds_110*Q.f_preds_110 + S.f_preds_111*Q.f_preds_111 + S.f_preds_112*Q.f_preds_112 + S.f_preds_113*Q.f_preds_113 + S.f_preds_114*Q.f_preds_114 + S.f_preds_115*Q.f_preds_115 + S.f_preds_116*Q.f_preds_116 + S.f_preds_117*Q.f_preds_117 + S.f_preds_118*Q.f_preds_118 + S.f_preds_119*Q.f_preds_119 + S.f_preds_120*Q.f_preds_120 + S.f_preds_121*Q.f_preds_121 + S.f_preds_122*Q.f_preds_122 + S.f_preds_123*Q.f_preds_123 + S.f_preds_124*Q.f_preds_124 + S.f_preds_125*Q.f_preds_125 + S.f_preds_126*Q.f_preds_126 + S.f_preds_127*Q.f_preds_127 + S.f_preds_128*Q.f_preds_128 + S.f_preds_129*Q.f_preds_129 + S.f_preds_130*Q.f_preds_130 + S.f_preds_131*Q.f_preds_131 + S.f_preds_132*Q.f_preds_132 + S.f_preds_133*Q.f_preds_133 + S.f_preds_134*Q.f_preds_134 + S.f_preds_135*Q.f_preds_135 + S.f_preds_136*Q.f_preds_136 + S.f_preds_137*Q.f_preds_137 + S.f_preds_138*Q.f_preds_138 + S.f_preds_139*Q.f_preds_139 + S.f_preds_140*Q.f_preds_140 + S.f_preds_141*Q.f_preds_141 + S.f_preds_142*Q.f_preds_142 + S.f_preds_143*Q.f_preds_143 + S.f_preds_144*Q.f_preds_144 + S.f_preds_145*Q.f_preds_145 + S.f_preds_146*Q.f_preds_146 + S.f_preds_147*Q.f_preds_147 + S.f_preds_148*Q.f_preds_148 + S.f_preds_149*Q.f_preds_149 + S.f_preds_150*Q.f_preds_150 + S.f_preds_151*Q.f_preds_151 + S.f_preds_152*Q.f_preds_152 + S.f_preds_153*Q.f_preds_153 + S.f_preds_154*Q.f_preds_154 + S.f_preds_155*Q.f_preds_155 + S.f_preds_156*Q.f_preds_156 + S.f_preds_157*Q.f_preds_157 + S.f_preds_158*Q.f_preds_158 + S.f_preds_159*Q.f_preds_159 + S.f_preds_160*Q.f_preds_160 + S.f_preds_161*Q.f_preds_161 + S.f_preds_162*Q.f_preds_162 + S.f_preds_163*Q.f_preds_163 + S.f_preds_164*Q.f_preds_164 + S.f_preds_165*Q.f_preds_165 + S.f_preds_166*Q.f_preds_166 + S.f_preds_167*Q.f_preds_167 + S.f_preds_168*Q.f_preds_168 + S.f_preds_169*Q.f_preds_169 + S.f_preds_170*Q.f_preds_170 + S.f_preds_171*Q.f_preds_171 + S.f_preds_172*Q.f_preds_172 + S.f_preds_173*Q.f_preds_173 + S.f_preds_174*Q.f_preds_174 + S.f_preds_175*Q.f_preds_175 + S.f_preds_176*Q.f_preds_176 + S.f_preds_177*Q.f_preds_177 + S.f_preds_178*Q.f_preds_178 + S.f_preds_179*Q.f_preds_179 + S.f_preds_180*Q.f_preds_180 + S.f_preds_181*Q.f_preds_181 + S.f_preds_182*Q.f_preds_182 + S.f_preds_183*Q.f_preds_183 + S.f_preds_184*Q.f_preds_184 + S.f_preds_185*Q.f_preds_185 + S.f_preds_186*Q.f_preds_186 + S.f_preds_187*Q.f_preds_187 + S.f_preds_188*Q.f_preds_188 + S.f_preds_189*Q.f_preds_189 + S.f_preds_190*Q.f_preds_190 + S.f_preds_191*Q.f_preds_191 + S.f_preds_192*Q.f_preds_192 + S.f_preds_193*Q.f_preds_193 + S.f_preds_194*Q.f_preds_194 + S.f_preds_195*Q.f_preds_195 + S.f_preds_196*Q.f_preds_196 + S.f_preds_197*Q.f_preds_197 + S.f_preds_198*Q.f_preds_198 + S.f_preds_199*Q.f_preds_199 + S.f_preds_200*Q.f_preds_200 + S.f_preds_201*Q.f_preds_201 + S.f_preds_202*Q.f_preds_202 + S.f_preds_203*Q.f_preds_203 + S.f_preds_204*Q.f_preds_204 + S.f_preds_205*Q.f_preds_205 + S.f_preds_206*Q.f_preds_206 + S.f_preds_207*Q.f_preds_207 + S.f_preds_208*Q.f_preds_208 + S.f_preds_209*Q.f_preds_209 + S.f_preds_210*Q.f_preds_210 + S.f_preds_211*Q.f_preds_211 + S.f_preds_212*Q.f_preds_212 + S.f_preds_213*Q.f_preds_213 + S.f_preds_214*Q.f_preds_214 + S.f_preds_215*Q.f_preds_215 + S.f_preds_216*Q.f_preds_216 + S.f_preds_217*Q.f_preds_217 + S.f_preds_218*Q.f_preds_218 + S.f_preds_219*Q.f_preds_219 + S.f_preds_220*Q.f_preds_220 + S.f_preds_221*Q.f_preds_221 + S.f_preds_222*Q.f_preds_222 + S.f_preds_223*Q.f_preds_223 + S.f_preds_224*Q.f_preds_224 + S.f_preds_225*Q.f_preds_225 + S.f_preds_226*Q.f_preds_226 + S.f_preds_227*Q.f_preds_227 + S.f_preds_228*Q.f_preds_228 + S.f_preds_229*Q.f_preds_229 + S.f_preds_230*Q.f_preds_230 + S.f_preds_231*Q.f_preds_231 + S.f_preds_232*Q.f_preds_232 + S.f_preds_233*Q.f_preds_233 + S.f_preds_234*Q.f_preds_234 + S.f_preds_235*Q.f_preds_235 + S.f_preds_236*Q.f_preds_236 + S.f_preds_237*Q.f_preds_237 + S.f_preds_238*Q.f_preds_238 + S.f_preds_239*Q.f_preds_239 + S.f_preds_240*Q.f_preds_240 + S.f_preds_241*Q.f_preds_241 + S.f_preds_242*Q.f_preds_242 + S.f_preds_243*Q.f_preds_243 + S.f_preds_244*Q.f_preds_244 + S.f_preds_245*Q.f_preds_245 + S.f_preds_246*Q.f_preds_246 + S.f_preds_247*Q.f_preds_247 + S.f_preds_248*Q.f_preds_248 + S.f_preds_249*Q.f_preds_249 + S.f_preds_250*Q.f_preds_250 + S.f_preds_251*Q.f_preds_251 + S.f_preds_252*Q.f_preds_252 + S.f_preds_253*Q.f_preds_253 + S.f_preds_254*Q.f_preds_254 + S.f_preds_255*Q.f_preds_255 + S.f_preds_256*Q.f_preds_256 + S.f_preds_257*Q.f_preds_257 + S.f_preds_258*Q.f_preds_258 + S.f_preds_259*Q.f_preds_259 + S.f_preds_260*Q.f_preds_260 + S.f_preds_261*Q.f_preds_261 + S.f_preds_262*Q.f_preds_262 + S.f_preds_263*Q.f_preds_263 + S.f_preds_264*Q.f_preds_264 + S.f_preds_265*Q.f_preds_265 + S.f_preds_266*Q.f_preds_266 + S.f_preds_267*Q.f_preds_267 + S.f_preds_268*Q.f_preds_268 + S.f_preds_269*Q.f_preds_269 + S.f_preds_270*Q.f_preds_270 + S.f_preds_271*Q.f_preds_271 + S.f_preds_272*Q.f_preds_272 + S.f_preds_273*Q.f_preds_273 + S.f_preds_274*Q.f_preds_274 + S.f_preds_275*Q.f_preds_275 + S.f_preds_276*Q.f_preds_276 + S.f_preds_277*Q.f_preds_277 + S.f_preds_278*Q.f_preds_278 + S.f_preds_279*Q.f_preds_279 + S.f_preds_280*Q.f_preds_280 + S.f_preds_281*Q.f_preds_281 + S.f_preds_282*Q.f_preds_282 + S.f_preds_283*Q.f_preds_283 + S.f_preds_284*Q.f_preds_284 + S.f_preds_285*Q.f_preds_285 + S.f_preds_286*Q.f_preds_286 + S.f_preds_287*Q.f_preds_287 + S.f_preds_288*Q.f_preds_288 + S.f_preds_289*Q.f_preds_289 + S.f_preds_290*Q.f_preds_290 + S.f_preds_291*Q.f_preds_291 + S.f_preds_292*Q.f_preds_292 + S.f_preds_293*Q.f_preds_293 + S.f_preds_294*Q.f_preds_294 + S.f_preds_295*Q.f_preds_295 + S.f_preds_296*Q.f_preds_296 + S.f_preds_297*Q.f_preds_297 + S.f_preds_298*Q.f_preds_298 + S.f_preds_299*Q.f_preds_299 + S.f_preds_300*Q.f_preds_300 + S.f_preds_301*Q.f_preds_301 + S.f_preds_302*Q.f_preds_302 + S.f_preds_303*Q.f_preds_303 + S.f_preds_304*Q.f_preds_304 + S.f_preds_305*Q.f_preds_305 + S.f_preds_306*Q.f_preds_306 + S.f_preds_307*Q.f_preds_307 + S.f_preds_308*Q.f_preds_308 + S.f_preds_309*Q.f_preds_309 + S.f_preds_310*Q.f_preds_310 + S.f_preds_311*Q.f_preds_311 + S.f_preds_312*Q.f_preds_312 + S.f_preds_313*Q.f_preds_313 + S.f_preds_314*Q.f_preds_314 + S.f_preds_315*Q.f_preds_315 + S.f_preds_316*Q.f_preds_316 + S.f_preds_317*Q.f_preds_317 + S.f_preds_318*Q.f_preds_318 + S.f_preds_319*Q.f_preds_319 + S.f_preds_320*Q.f_preds_320 + S.f_preds_321*Q.f_preds_321 + S.f_preds_322*Q.f_preds_322 + S.f_preds_323*Q.f_preds_323 + S.f_preds_324*Q.f_preds_324 + S.f_preds_325*Q.f_preds_325 + S.f_preds_326*Q.f_preds_326 + S.f_preds_327*Q.f_preds_327 + S.f_preds_328*Q.f_preds_328 + S.f_preds_329*Q.f_preds_329 + S.f_preds_330*Q.f_preds_330 + S.f_preds_331*Q.f_preds_331 + S.f_preds_332*Q.f_preds_332 + S.f_preds_333*Q.f_preds_333 + S.f_preds_334*Q.f_preds_334 + S.f_preds_335*Q.f_preds_335 + S.f_preds_336*Q.f_preds_336 + S.f_preds_337*Q.f_preds_337 + S.f_preds_338*Q.f_preds_338 + S.f_preds_339*Q.f_preds_339 + S.f_preds_340*Q.f_preds_340 + S.f_preds_341*Q.f_preds_341 + S.f_preds_342*Q.f_preds_342 + S.f_preds_343*Q.f_preds_343 + S.f_preds_344*Q.f_preds_344 + S.f_preds_345*Q.f_preds_345 + S.f_preds_346*Q.f_preds_346 + S.f_preds_347*Q.f_preds_347 + S.f_preds_348*Q.f_preds_348 + S.f_preds_349*Q.f_preds_349 + S.f_preds_350*Q.f_preds_350 + S.f_preds_351*Q.f_preds_351 + S.f_preds_352*Q.f_preds_352 + S.f_preds_353*Q.f_preds_353 + S.f_preds_354*Q.f_preds_354 + S.f_preds_355*Q.f_preds_355 + S.f_preds_356*Q.f_preds_356 + S.f_preds_357*Q.f_preds_357 + S.f_preds_358*Q.f_preds_358 + S.f_preds_359*Q.f_preds_359 + S.f_preds_360*Q.f_preds_360 + S.f_preds_361*Q.f_preds_361 + S.f_preds_362*Q.f_preds_362 + S.f_preds_363*Q.f_preds_363 + S.f_preds_364*Q.f_preds_364 + S.f_preds_365*Q.f_preds_365 + S.f_preds_366*Q.f_preds_366 + S.f_preds_367*Q.f_preds_367 + S.f_preds_368*Q.f_preds_368 + S.f_preds_369*Q.f_preds_369 + S.f_preds_370*Q.f_preds_370 + S.f_preds_371*Q.f_preds_371 + S.f_preds_372*Q.f_preds_372 + S.f_preds_373*Q.f_preds_373 + S.f_preds_374*Q.f_preds_374 + S.f_preds_375*Q.f_preds_375 + S.f_preds_376*Q.f_preds_376 + S.f_preds_377*Q.f_preds_377 + S.f_preds_378*Q.f_preds_378 + S.f_preds_379*Q.f_preds_379 + S.f_preds_380*Q.f_preds_380 + S.f_preds_381*Q.f_preds_381 + S.f_preds_382*Q.f_preds_382 + S.f_preds_383*Q.f_preds_383 + S.f_preds_384*Q.f_preds_384 + S.f_preds_385*Q.f_preds_385 + S.f_preds_386*Q.f_preds_386 + S.f_preds_387*Q.f_preds_387 + S.f_preds_388*Q.f_preds_388 + S.f_preds_389*Q.f_preds_389 + S.f_preds_390*Q.f_preds_390 + S.f_preds_391*Q.f_preds_391 + S.f_preds_392*Q.f_preds_392 + S.f_preds_393*Q.f_preds_393 + S.f_preds_394*Q.f_preds_394 + S.f_preds_395*Q.f_preds_395 + S.f_preds_396*Q.f_preds_396 + S.f_preds_397*Q.f_preds_397 + S.f_preds_398*Q.f_preds_398 + S.f_preds_399*Q.f_preds_399 + S.f_preds_400*Q.f_preds_400 + S.f_preds_401*Q.f_preds_401 + S.f_preds_402*Q.f_preds_402 + S.f_preds_403*Q.f_preds_403 + S.f_preds_404*Q.f_preds_404 + S.f_preds_405*Q.f_preds_405 + S.f_preds_406*Q.f_preds_406 + S.f_preds_407*Q.f_preds_407 + S.f_preds_408*Q.f_preds_408 + S.f_preds_409*Q.f_preds_409 + S.f_preds_410*Q.f_preds_410 + S.f_preds_411*Q.f_preds_411 + S.f_preds_412*Q.f_preds_412 + S.f_preds_413*Q.f_preds_413 + S.f_preds_414*Q.f_preds_414 + S.f_preds_415*Q.f_preds_415 + S.f_preds_416*Q.f_preds_416 + S.f_preds_417*Q.f_preds_417 + S.f_preds_418*Q.f_preds_418 + S.f_preds_419*Q.f_preds_419 + S.f_preds_420*Q.f_preds_420 + S.f_preds_421*Q.f_preds_421 + S.f_preds_422*Q.f_preds_422 + S.f_preds_423*Q.f_preds_423 + S.f_preds_424*Q.f_preds_424 + S.f_preds_425*Q.f_preds_425 + S.f_preds_426*Q.f_preds_426 + S.f_preds_427*Q.f_preds_427 + S.f_preds_428*Q.f_preds_428 + S.f_preds_429*Q.f_preds_429 + S.f_preds_430*Q.f_preds_430 + S.f_preds_431*Q.f_preds_431 + S.f_preds_432*Q.f_preds_432 + S.f_preds_433*Q.f_preds_433 + S.f_preds_434*Q.f_preds_434 + S.f_preds_435*Q.f_preds_435 + S.f_preds_436*Q.f_preds_436 + S.f_preds_437*Q.f_preds_437 + S.f_preds_438*Q.f_preds_438 + S.f_preds_439*Q.f_preds_439 + S.f_preds_440*Q.f_preds_440 + S.f_preds_441*Q.f_preds_441 + S.f_preds_442*Q.f_preds_442 + S.f_preds_443*Q.f_preds_443 + S.f_preds_444*Q.f_preds_444 + S.f_preds_445*Q.f_preds_445 + S.f_preds_446*Q.f_preds_446 + S.f_preds_447*Q.f_preds_447 + S.f_preds_448*Q.f_preds_448 + S.f_preds_449*Q.f_preds_449 + S.f_preds_450*Q.f_preds_450 + S.f_preds_451*Q.f_preds_451 + S.f_preds_452*Q.f_preds_452 + S.f_preds_453*Q.f_preds_453 + S.f_preds_454*Q.f_preds_454 + S.f_preds_455*Q.f_preds_455 + S.f_preds_456*Q.f_preds_456 + S.f_preds_457*Q.f_preds_457 + S.f_preds_458*Q.f_preds_458 + S.f_preds_459*Q.f_preds_459 + S.f_preds_460*Q.f_preds_460 + S.f_preds_461*Q.f_preds_461 + S.f_preds_462*Q.f_preds_462 + S.f_preds_463*Q.f_preds_463 + S.f_preds_464*Q.f_preds_464 + S.f_preds_465*Q.f_preds_465 + S.f_preds_466*Q.f_preds_466 + S.f_preds_467*Q.f_preds_467 + S.f_preds_468*Q.f_preds_468 + S.f_preds_469*Q.f_preds_469 + S.f_preds_470*Q.f_preds_470 + S.f_preds_471*Q.f_preds_471 + S.f_preds_472*Q.f_preds_472 + S.f_preds_473*Q.f_preds_473 + S.f_preds_474*Q.f_preds_474 + S.f_preds_475*Q.f_preds_475 + S.f_preds_476*Q.f_preds_476 + S.f_preds_477*Q.f_preds_477 + S.f_preds_478*Q.f_preds_478 + S.f_preds_479*Q.f_preds_479 + S.f_preds_480*Q.f_preds_480 + S.f_preds_481*Q.f_preds_481 + S.f_preds_482*Q.f_preds_482 + S.f_preds_483*Q.f_preds_483 + S.f_preds_484*Q.f_preds_484 + S.f_preds_485*Q.f_preds_485 + S.f_preds_486*Q.f_preds_486 + S.f_preds_487*Q.f_preds_487 + S.f_preds_488*Q.f_preds_488 + S.f_preds_489*Q.f_preds_489 + S.f_preds_490*Q.f_preds_490 + S.f_preds_491*Q.f_preds_491 + S.f_preds_492*Q.f_preds_492 + S.f_preds_493*Q.f_preds_493 + S.f_preds_494*Q.f_preds_494 + S.f_preds_495*Q.f_preds_495 + S.f_preds_496*Q.f_preds_496 + S.f_preds_497*Q.f_preds_497 + S.f_preds_498*Q.f_preds_498 + S.f_preds_499*Q.f_preds_499 + S.f_preds_500*Q.f_preds_500 + S.f_preds_501*Q.f_preds_501 + S.f_preds_502*Q.f_preds_502 + S.f_preds_503*Q.f_preds_503 + S.f_preds_504*Q.f_preds_504 + S.f_preds_505*Q.f_preds_505 + S.f_preds_506*Q.f_preds_506 + S.f_preds_507*Q.f_preds_507 + S.f_preds_508*Q.f_preds_508 + S.f_preds_509*Q.f_preds_509 + S.f_preds_510*Q.f_preds_510 + S.f_preds_511*Q.f_preds_511 + S.f_preds_512*Q.f_preds_512 + S.f_preds_513*Q.f_preds_513 + S.f_preds_514*Q.f_preds_514 + S.f_preds_515*Q.f_preds_515 + S.f_preds_516*Q.f_preds_516 + S.f_preds_517*Q.f_preds_517 + S.f_preds_518*Q.f_preds_518 + S.f_preds_519*Q.f_preds_519 + S.f_preds_520*Q.f_preds_520 + S.f_preds_521*Q.f_preds_521 + S.f_preds_522*Q.f_preds_522 + S.f_preds_523*Q.f_preds_523 + S.f_preds_524*Q.f_preds_524 + S.f_preds_525*Q.f_preds_525 + S.f_preds_526*Q.f_preds_526 + S.f_preds_527*Q.f_preds_527 + S.f_preds_528*Q.f_preds_528 + S.f_preds_529*Q.f_preds_529 + S.f_preds_530*Q.f_preds_530 + S.f_preds_531*Q.f_preds_531 + S.f_preds_532*Q.f_preds_532 + S.f_preds_533*Q.f_preds_533 + S.f_preds_534*Q.f_preds_534 + S.f_preds_535*Q.f_preds_535 + S.f_preds_536*Q.f_preds_536 + S.f_preds_537*Q.f_preds_537 + S.f_preds_538*Q.f_preds_538 + S.f_preds_539*Q.f_preds_539 + S.f_preds_540*Q.f_preds_540 + S.f_preds_541*Q.f_preds_541 + S.f_preds_542*Q.f_preds_542 + S.f_preds_543*Q.f_preds_543 + S.f_preds_544*Q.f_preds_544 + S.f_preds_545*Q.f_preds_545 + S.f_preds_546*Q.f_preds_546 + S.f_preds_547*Q.f_preds_547 + S.f_preds_548*Q.f_preds_548 + S.f_preds_549*Q.f_preds_549 + S.f_preds_550*Q.f_preds_550 + S.f_preds_551*Q.f_preds_551 + S.f_preds_552*Q.f_preds_552 + S.f_preds_553*Q.f_preds_553 + S.f_preds_554*Q.f_preds_554 + S.f_preds_555*Q.f_preds_555 + S.f_preds_556*Q.f_preds_556 + S.f_preds_557*Q.f_preds_557 + S.f_preds_558*Q.f_preds_558 + S.f_preds_559*Q.f_preds_559 + S.f_preds_560*Q.f_preds_560 + S.f_preds_561*Q.f_preds_561 + S.f_preds_562*Q.f_preds_562 + S.f_preds_563*Q.f_preds_563 + S.f_preds_564*Q.f_preds_564 + S.f_preds_565*Q.f_preds_565 + S.f_preds_566*Q.f_preds_566 + S.f_preds_567*Q.f_preds_567 + S.f_preds_568*Q.f_preds_568 + S.f_preds_569*Q.f_preds_569 + S.f_preds_570*Q.f_preds_570 + S.f_preds_571*Q.f_preds_571 + S.f_preds_572*Q.f_preds_572 + S.f_preds_573*Q.f_preds_573 + S.f_preds_574*Q.f_preds_574 + S.f_preds_575*Q.f_preds_575 + S.f_preds_576*Q.f_preds_576 + S.f_preds_577*Q.f_preds_577 + S.f_preds_578*Q.f_preds_578 + S.f_preds_579*Q.f_preds_579 + S.f_preds_580*Q.f_preds_580 + S.f_preds_581*Q.f_preds_581 + S.f_preds_582*Q.f_preds_582 + S.f_preds_583*Q.f_preds_583 + S.f_preds_584*Q.f_preds_584 + S.f_preds_585*Q.f_preds_585 + S.f_preds_586*Q.f_preds_586 + S.f_preds_587*Q.f_preds_587 + S.f_preds_588*Q.f_preds_588 + S.f_preds_589*Q.f_preds_589 + S.f_preds_590*Q.f_preds_590 + S.f_preds_591*Q.f_preds_591 + S.f_preds_592*Q.f_preds_592 + S.f_preds_593*Q.f_preds_593 + S.f_preds_594*Q.f_preds_594 + S.f_preds_595*Q.f_preds_595 + S.f_preds_596*Q.f_preds_596 + S.f_preds_597*Q.f_preds_597 + S.f_preds_598*Q.f_preds_598 + S.f_preds_599*Q.f_preds_599 + S.f_preds_600*Q.f_preds_600 + S.f_preds_601*Q.f_preds_601 + S.f_preds_602*Q.f_preds_602 + S.f_preds_603*Q.f_preds_603 + S.f_preds_604*Q.f_preds_604 + S.f_preds_605*Q.f_preds_605 + S.f_preds_606*Q.f_preds_606 + S.f_preds_607*Q.f_preds_607 + S.f_preds_608*Q.f_preds_608 + S.f_preds_609*Q.f_preds_609 + S.f_preds_610*Q.f_preds_610 + S.f_preds_611*Q.f_preds_611 + S.f_preds_612*Q.f_preds_612 + S.f_preds_613*Q.f_preds_613 + S.f_preds_614*Q.f_preds_614 + S.f_preds_615*Q.f_preds_615 + S.f_preds_616*Q.f_preds_616 + S.f_preds_617*Q.f_preds_617 + S.f_preds_618*Q.f_preds_618 + S.f_preds_619*Q.f_preds_619 + S.f_preds_620*Q.f_preds_620 + S.f_preds_621*Q.f_preds_621 + S.f_preds_622*Q.f_preds_622 + S.f_preds_623*Q.f_preds_623 + S.f_preds_624*Q.f_preds_624 + S.f_preds_625*Q.f_preds_625 + S.f_preds_626*Q.f_preds_626 + S.f_preds_627*Q.f_preds_627 + S.f_preds_628*Q.f_preds_628 + S.f_preds_629*Q.f_preds_629 + S.f_preds_630*Q.f_preds_630 + S.f_preds_631*Q.f_preds_631 + S.f_preds_632*Q.f_preds_632 + S.f_preds_633*Q.f_preds_633 + S.f_preds_634*Q.f_preds_634 + S.f_preds_635*Q.f_preds_635 + S.f_preds_636*Q.f_preds_636 + S.f_preds_637*Q.f_preds_637 + S.f_preds_638*Q.f_preds_638 + S.f_preds_639*Q.f_preds_639 + S.f_preds_640*Q.f_preds_640 + S.f_preds_641*Q.f_preds_641 + S.f_preds_642*Q.f_preds_642 + S.f_preds_643*Q.f_preds_643 + S.f_preds_644*Q.f_preds_644 + S.f_preds_645*Q.f_preds_645 + S.f_preds_646*Q.f_preds_646 + S.f_preds_647*Q.f_preds_647 + S.f_preds_648*Q.f_preds_648 + S.f_preds_649*Q.f_preds_649 + S.f_preds_650*Q.f_preds_650 + S.f_preds_651*Q.f_preds_651 + S.f_preds_652*Q.f_preds_652 + S.f_preds_653*Q.f_preds_653 + S.f_preds_654*Q.f_preds_654 + S.f_preds_655*Q.f_preds_655 + S.f_preds_656*Q.f_preds_656 + S.f_preds_657*Q.f_preds_657 + S.f_preds_658*Q.f_preds_658 + S.f_preds_659*Q.f_preds_659 + S.f_preds_660*Q.f_preds_660 + S.f_preds_661*Q.f_preds_661 + S.f_preds_662*Q.f_preds_662 + S.f_preds_663*Q.f_preds_663 + S.f_preds_664*Q.f_preds_664 + S.f_preds_665*Q.f_preds_665 + S.f_preds_666*Q.f_preds_666 + S.f_preds_667*Q.f_preds_667 + S.f_preds_668*Q.f_preds_668 + S.f_preds_669*Q.f_preds_669 + S.f_preds_670*Q.f_preds_670 + S.f_preds_671*Q.f_preds_671 + S.f_preds_672*Q.f_preds_672 + S.f_preds_673*Q.f_preds_673 + S.f_preds_674*Q.f_preds_674 + S.f_preds_675*Q.f_preds_675 + S.f_preds_676*Q.f_preds_676 + S.f_preds_677*Q.f_preds_677 + S.f_preds_678*Q.f_preds_678 + S.f_preds_679*Q.f_preds_679 + S.f_preds_680*Q.f_preds_680 + S.f_preds_681*Q.f_preds_681 + S.f_preds_682*Q.f_preds_682 + S.f_preds_683*Q.f_preds_683 + S.f_preds_684*Q.f_preds_684 + S.f_preds_685*Q.f_preds_685 + S.f_preds_686*Q.f_preds_686 + S.f_preds_687*Q.f_preds_687 + S.f_preds_688*Q.f_preds_688 + S.f_preds_689*Q.f_preds_689 + S.f_preds_690*Q.f_preds_690 + S.f_preds_691*Q.f_preds_691 + S.f_preds_692*Q.f_preds_692 + S.f_preds_693*Q.f_preds_693 + S.f_preds_694*Q.f_preds_694 + S.f_preds_695*Q.f_preds_695 + S.f_preds_696*Q.f_preds_696 + S.f_preds_697*Q.f_preds_697 + S.f_preds_698*Q.f_preds_698 + S.f_preds_699*Q.f_preds_699 + S.f_preds_700*Q.f_preds_700 + S.f_preds_701*Q.f_preds_701 + S.f_preds_702*Q.f_preds_702 + S.f_preds_703*Q.f_preds_703 + S.f_preds_704*Q.f_preds_704 + S.f_preds_705*Q.f_preds_705 + S.f_preds_706*Q.f_preds_706 + S.f_preds_707*Q.f_preds_707 + S.f_preds_708*Q.f_preds_708 + S.f_preds_709*Q.f_preds_709 + S.f_preds_710*Q.f_preds_710 + S.f_preds_711*Q.f_preds_711 + S.f_preds_712*Q.f_preds_712 + S.f_preds_713*Q.f_preds_713 + S.f_preds_714*Q.f_preds_714 + S.f_preds_715*Q.f_preds_715 + S.f_preds_716*Q.f_preds_716 + S.f_preds_717*Q.f_preds_717 + S.f_preds_718*Q.f_preds_718 + S.f_preds_719*Q.f_preds_719 + S.f_preds_720*Q.f_preds_720 + S.f_preds_721*Q.f_preds_721 + S.f_preds_722*Q.f_preds_722 + S.f_preds_723*Q.f_preds_723 + S.f_preds_724*Q.f_preds_724 + S.f_preds_725*Q.f_preds_725 + S.f_preds_726*Q.f_preds_726 + S.f_preds_727*Q.f_preds_727 + S.f_preds_728*Q.f_preds_728 + S.f_preds_729*Q.f_preds_729 + S.f_preds_730*Q.f_preds_730 + S.f_preds_731*Q.f_preds_731 + S.f_preds_732*Q.f_preds_732 + S.f_preds_733*Q.f_preds_733 + S.f_preds_734*Q.f_preds_734 + S.f_preds_735*Q.f_preds_735 + S.f_preds_736*Q.f_preds_736 + S.f_preds_737*Q.f_preds_737 + S.f_preds_738*Q.f_preds_738) / S.norm_f_preds / Q.norm_f_preds
             as d_pc
                  from search_recs S
                    left join Q on true --# (1 row)
                  where true
                    and S.species in ('EMGO', 'SNGO', 'ROGO', 'GWFG', 'BRAN', 'CACG', 'CANG', 'MUSW', 'WODU', 'BWTE', 'CITE', 'NSHO', 'GADW', 'EUWI', 'AMWI', 'NOPI', 'GWTE', 'CANV', 'REDH', 'RNDU', 'GRSC', 'LESC', 'SUSC', 'BLSC', 'LTDU', 'BUFF', 'COGO', 'BAGO', 'HOME', 'COME', 'RBME', 'RUDU', 'CAQU', 'RNEP', 'WITU', 'PBGR', 'HOGR', 'RNGR', 'EAGR', 'WEGR', 'CLGR', 'RODO', 'BTPI', 'EUCD', 'MODO', 'BLSW', 'VASW', 'WTSW', 'ANHU', 'RUHU', 'ALHU', 'VIRA', 'SORA', 'COGA', 'AMCO', 'SACR', 'BNST', 'AMAV', 'BLOY', 'BBPL', 'PAGP', 'SNPL', 'SEPL', 'KILL', 'WHIM', 'LBCU', 'MAGO', 'BLTU', 'SURF', 'SAND', 'DUNL', 'BASA', 'LESA', 'PESA', 'SESA', 'WESA', 'SBDO', 'LBDO', 'WISN', 'WIPH', 'RNPH', 'REPH', 'SPSA', 'SOSA', 'WATA', 'GRYE', 'WILL', 'LEYE', 'POJA', 'PAJA', 'COMU', 'PIGU', 'MAMU', 'ANMU', 'CAAU', 'RHAU', 'TUPU', 'BLKI', 'SAGU', 'BOGU', 'HEEG', 'MEGU', 'RBGU', 'WEGU', 'CAGU', 'HERG', 'ICGU', 'GWGU', 'GLGU', 'CATE', 'BLTE', 'COTE', 'FOTE', 'ELTE', 'BLSK', 'RTLO', 'PALO', 'COLO', 'NOFU', 'SOSH', 'BVSH', 'BRBO', 'BRAC', 'PECO', 'DCCO', 'BRPE', 'AMBI', 'GBHE', 'GREG', 'SNEG', 'GRHE', 'BCNH', 'WFIB', 'TUVU', 'OSPR', 'WTKI', 'GOEA', 'NOHA', 'SSHA', 'COHA', 'BAEA', 'RSHA', 'BWHA', 'RTHA', 'FEHA', 'BANO', 'GHOW', 'SEOW', 'BEKI', 'RBSA', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'PIWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'PRFA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'TRKI', 'WEKI', 'LOSH', 'HUVI', 'CAVI', 'WAVI', 'YGVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'NRWS', 'PUMA', 'TRES', 'VGSW', 'BANS', 'BARS', 'CLSW', 'CBCH', 'BUSH', 'RBNU', 'WBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'EYWA', 'WHWA', 'AMPI', 'CEDW', 'HOFI', 'PUFI', 'RECR', 'PISI', 'LEGO', 'AMGO', 'GRSP', 'CHSP', 'CCSP', 'BRSP', 'LASP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'SWSP', 'CALT', 'RCSP', 'SPTO', 'YBCH', 'YHBL', 'WEME', 'HOOR', 'BUOR', 'RWBL', 'TRBL', 'BHCO', 'RUBL', 'BRBL', 'GTGR', 'NOWA', 'BAWW', 'PROW', 'TEWA', 'OCWA', 'NAWA', 'MGWA', 'COYE', 'AMRE', 'YEWA', 'CSWA', 'BLPW', 'PAWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'WETA', 'BHGR', 'LAZB', 'INBU', 'HOSP') --# Empty subquery for species outside of placeFilter
                    and S.quality in ('A', 'B')
                    and species not in ()
                    and species_species_group not in ()
                    and S.source_id != 'xc:72445' --# Exclude query_rec from results
                )
              select *
              from (
                -- Must wrap subqueries in 'select * from (...)' else union complains about nested order by
                select * from (
                select *, coalesce(4.044174434487836, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'WREN'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.070386964691503, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'WETA'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.138125620241494, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'PUFI'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.29554059701118, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'VIRA'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.316455553030161, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'HOFI'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.31658506772705, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'YGVI'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.386598299366651, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'RECR'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.396162508933393, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'WEBL'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.432474618873793, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'RWBL'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.437134218173955, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'HOSP'
                order by d_pc asc
                limit 3
              ) union all select * from (
                select *, coalesce(4.449093520076393, 1e38) as slp
                from search_recs_filtered_and_dist where species = 'ATFL'
                order by d_pc asc
                limit 3
              )
              )
              order by
                slp asc,
                d_pc asc
              limit 31

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan
select count(distinct species)
from search_recs S
where true
    -- and S.species in ('RODO', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'WEGU', 'CAGU', 'CATE', 'FOTE', 'DCCO', 'GBHE', 'SNEG', 'BCNH', 'TUVU', 'OSPR', 'COHA', 'RTHA', 'NOFL', 'MERL', 'PEFA', 'RMPA', 'BLPH', 'SAPH', 'STJA', 'CASJ', 'AMCR', 'CORA', 'BARS', 'CBCH', 'BUSH', 'RCKI', 'HETH', 'AMRO', 'NOMO', 'EUST', 'HOFI', 'PUFI', 'LEGO', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'SOSP', 'CALT', 'WEME', 'HOOR', 'YRWA', 'TOWA', 'HOSP')
    and S.quality in ('A', 'B')
    and species not in ('HETH')
    and species_species_group not in ('foo bar')
    and S.source_id != 'xc:72445'
limit 3

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan
select count(distinct species)
from search_recs S
where true
    and S.species in ('BBWD', 'FUWD', 'EMGO', 'SNGO', 'ROGO', 'GWFG', 'TABG', 'TUBG', 'BRAN', 'CACG', 'CANG', 'MUSW', 'TRUS', 'TUSW', 'WHOS', 'EGGO', 'MUDU', 'WODU', 'manduc', 'GARG', 'BWTE', 'CITE', 'NSHO', 'GADW', 'FADU', 'EUWI', 'AMWI', 'NOPI', 'GWTE', 'CANV', 'REDH', 'COMP', 'RNDU', 'TUDU', 'GRSC', 'LESC', 'KIEI', 'COEI', 'HADU', 'SUSC', 'COSC', 'BLSC', 'LTDU', 'BUFF', 'COGO', 'BAGO', 'HOME', 'COME', 'RBME', 'RUDU', 'MOUQ', 'CAQU', 'GAQU', 'INPE', 'CHUK', 'RNEP', 'RUGR', 'GRSG', 'WTPT', 'SOGR', 'WITU', 'PBGR', 'HOGR', 'RNGR', 'EAGR', 'WEGR', 'CLGR', 'RODO', 'BTPI', 'EUCD', 'AFCD', 'SPDO', 'INDO', 'COGD', 'RUGD', 'WWDO', 'MODO', 'GBAN', 'GRRO', 'YBCU', 'BBCU', 'COCU', 'LENI', 'CONI', 'COPO', 'EWPW', 'MWPW', 'BLSW', 'CHSW', 'VASW', 'COSW', 'WTSW', 'BTHH', 'RTHU', 'BCHU', 'ANHU', 'COHU', 'BTAH', 'RUHU', 'ALHU', 'CAHU', 'BBIH', 'VCHU', 'RIRA', 'VIRA', 'SORA', 'COGA', 'AMCO', 'PUGA', 'YERA', 'BLRA', 'SACR', 'CCRA', 'BNST', 'AMAV', 'AMOY', 'BLOY', 'BBPL', 'AMGP', 'PAGP', 'LSAP', 'SNPL', 'WIPL', 'CRPL', 'SEPL', 'KILL', 'MOPL', 'UPSA', 'WHIM', 'LBCU', 'BTGO', 'HUGO', 'MAGO', 'RUTU', 'BLTU', 'REKN', 'SURF', 'RUFF', 'SPTS', 'STSA', 'CUSA', 'RNST', 'SAND', 'DUNL', 'ROSA', 'PUSA', 'BASA', 'LIST', 'LESA', 'WRSA', 'BBSA', 'PESA', 'SESA', 'WESA', 'SBDO', 'LBDO', 'AMWO', 'COSN', 'WISN', 'WIPH', 'RNPH', 'REPH', 'SPSA', 'SOSA', 'WATA', 'GRYE', 'WILL', 'LEYE', 'MASA', 'WOSA', 'SPSK', 'POJA', 'PAJA', 'LTJA', 'COMU', 'TBMU', 'PIGU', 'LBMU', 'MAMU', 'SCMU', 'GUMU', 'CRMU', 'ANMU', 'CAAU', 'PAAU', 'RHAU', 'HOPU', 'TUPU', 'STGU', 'BLKI', 'IVGU', 'SAGU', 'BOGU', 'BHGU', 'LIGU', 'ROGU', 'LAGU', 'FRGU', 'BTGU', 'HEEG', 'MEGU', 'RBGU', 'WEGU', 'YFGU', 'CAGU', 'HERG', 'ICGU', 'LBBG', 'SBGU', 'GWGU', 'GLGU', 'GBBG', 'KEGU', 'SOTE', 'BRTE', 'LETE', 'GBTE', 'CATE', 'BLTE', 'COTE', 'ARTE', 'FOTE', 'ROYT', 'ELTE', 'BLSK', 'RBTR', 'RTTR', 'RTLO', 'ARLO', 'PALO', 'COLO', 'YBLO', 'SAAL', 'LAAL', 'BFAL', 'STAL', 'WISP', 'FTSP', 'LESP', 'TOSP', 'ASSP', 'BSTP', 'WRSP', 'BLSP', 'LSTP', 'NOFU', 'GFPE', 'KEPE', 'MUPE', 'MOPE', 'HAPE', 'COPE', 'STPE', 'JOPE', 'WCPE', 'COSH', 'PFSH', 'FFSH', 'GRSH', 'WTSH', 'BULS', 'SOSH', 'SRTS', 'MASH', 'BVSH', 'WOST', 'MAFR', 'GREF', 'MABO', 'NABO', 'BFBO', 'BRBO', 'RFBO', 'NOGA', 'BRAC', 'PECO', 'NECO', 'DCCO', 'AWPE', 'BRPE', 'AMBI', 'LEBI', 'GBHE', 'GREG', 'SNEG', 'LBHE', 'TRHE', 'REEG', 'CAEG', 'GRHE', 'BCNH', 'YCNH', 'WHIB', 'GLIB', 'WFIB', 'ROSP', 'CACO', 'BLVU', 'TUVU', 'OSPR', 'WTKI', 'STKI', 'GOEA', 'MIKI', 'NOHA', 'SSHA', 'COHA', 'NOGO', 'BAEA', 'COBH', 'HASH', 'GRHA', 'RSHA', 'BWHA', 'SWHA', 'ZTHA', 'RTHA', 'RLHA', 'FEHA', 'BANO', 'FLOW', 'WESO', 'GHOW', 'SNOW', 'NOPO', 'ELOW', 'BUOW', 'SPOW', 'BADO', 'GGOW', 'LEOW', 'SEOW', 'NSWO', 'BEKI', 'EUWR', 'WISA', 'YBSA', 'RNSA', 'RBSA', 'LEWO', 'ACWO', 'GIWO', 'BBWO', 'DOWO', 'NUWO', 'LBWO', 'HAWO', 'WHWO', 'PIWO', 'NOFL', 'GIFL', 'CRCA', 'AMKE', 'MERL', 'GYRF', 'PEFA', 'PRFA', 'RRPA', 'MOPA', 'WWPA', 'yecpar', 'RCPA', 'LCPA', 'RLPA', 'YHPA', 'YCPA', 'TFAM', 'WFPA', 'NAPA', 'bucpar', 'MIPA', 'RMPA', 'OSFL', 'GRPE', 'WEWP', 'EAWP', 'YBFL', 'ALFL', 'WIFL', 'LEFL', 'HAFL', 'GRFL', 'DUFL', 'PSFL', 'COFL', 'BBFL', 'BLPH', 'EAPH', 'SAPH', 'VEFL', 'DCFL', 'ATFL', 'GCFL', 'BCFL', 'SBFL', 'TRKI', 'COKI', 'CAKI', 'TBKI', 'WEKI', 'EAKI', 'STFL', 'BROS', 'LOSH', 'NSHR', 'WEVI', 'BEVI', 'GRVI', 'HUVI', 'YTVI', 'CAVI', 'BHVI', 'PLVI', 'PHVI', 'WAVI', 'YGVI', 'GRAJ', 'BTMJ', 'PIJA', 'STJA', 'BLJA', 'ISSJ', 'CASJ', 'WOSJ', 'BBMA', 'YBMA', 'CLNU', 'AMCR', 'CORA', 'HOLA', 'EUSK', 'NRWS', 'PUMA', 'TRES', 'VGSW', 'BANS', 'BARS', 'CLSW', 'CASW', 'BCCH', 'MOCH', 'CBCH', 'OATI', 'JUTI', 'VERD', 'BUSH', 'RBNU', 'WBNU', 'PYNU', 'BRCR', 'ROWR', 'CANW', 'HOWR', 'PAWR', 'WIWR', 'SEWR', 'MAWR', 'BEWR', 'CACW', 'BGGN', 'CAGN', 'BTGN', 'AMDI', 'RWBU', 'GCKI', 'RCKI', 'DUWA', 'ARWA', 'WREN', 'JAWE', 'BLUE', 'RFBL', 'NOWH', 'WEBL', 'MOBL', 'TOSO', 'VATH', 'VEER', 'GCTH', 'SWTH', 'HETH', 'WOTH', 'AMRO', 'RBRO', 'GRCA', 'CBTH', 'BRTH', 'BETH', 'CATH', 'LCTH', 'CRTH', 'SATH', 'NOMO', 'EUST', 'EYWA', 'CIWA', 'WHWA', 'OBPI', 'RTPI', 'AMPI', 'SPPI', 'BOWA', 'CEDW', 'PHAI', 'BRAM', 'EVGR', 'HAWF', 'PIGR', 'GCRF', 'BLRF', 'HOFI', 'PUFI', 'CAFI', 'CORE', 'RECR', 'WWCR', 'EUGO', 'PISI', 'LEGO', 'LAGO', 'AMGO', 'LALO', 'CCLO', 'SMLO', 'MCLO', 'SNBU', 'LIBU', 'RUBU', 'CASP', 'GRSP', 'CHSP', 'CCSP', 'BCSP', 'FISP', 'BRSP', 'BTSP', 'LASP', 'LARB', 'ATSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'HASP', 'WTSP', 'SABS', 'BESP', 'VESP', 'LCSP', 'NESP', 'SAVS', 'BAIS', 'SOSP', 'LISP', 'SWSP', 'ABTO', 'CALT', 'RCSP', 'GTTO', 'SPTO', 'YBCH', 'YHBL', 'BOBO', 'WEME', 'EAME', 'OROR', 'HOOR', 'SBAO', 'BUOR', 'BAOR', 'SCOR', 'RWBL', 'TRBL', 'BROC', 'BHCO', 'RUBL', 'BRBL', 'COGR', 'GTGR', 'OVEN', 'WEWA', 'LOWA', 'NOWA', 'GWWA', 'BWWA', 'BAWW', 'PROW', 'TEWA', 'OCWA', 'LUWA', 'NAWA', 'VIWA', 'CONW', 'MGWA', 'MOWA', 'KEWA', 'COYE', 'HOWA', 'AMRE', 'CMWA', 'CERW', 'NOPA', 'TRPA', 'MAWA', 'BBWA', 'BLBW', 'YEWA', 'CSWA', 'BLPW', 'BTBW', 'PAWA', 'PIWA', 'YRWA', 'YTWA', 'PRAW', 'GRWA', 'BTYW', 'TOWA', 'HEWA', 'BTNW', 'CAWA', 'WIWA', 'RFWA', 'PARE', 'HETA', 'SUTA', 'SCTA', 'WETA', 'NOCA', 'PYRR', 'RBGR', 'BHGR', 'BLGR', 'LAZB', 'INBU', 'VABU', 'PABU', 'DICK', 'HOSP', 'NRBI', 'OCHW', 'BRMA', 'SBMU', 'PTWH')
    --# and S.quality in ('A', 'B')
    and species not in ()
    and species_species_group not in ()
    --# and source_id not in ('xc:72445')
limit 3

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan -Olen
select distinct species
from search_recs S
--# indexed by ix_search_recs__source_id
--# indexed by ix_search_recs__species
--# indexed by ix_search_recs__species__source_id
--# indexed by ix_search_recs__species__species_species_group__quality__source_id --# TODO WORKS (so far)
--# indexed by ix_search_recs__species_species_group__species__quality__source_id
--# indexed by ix_search_recs__source_id__species
where true
    and S.species in ('BBWD', 'FUWD', 'EMGO', 'SNGO', 'ROGO', 'GWFG', 'TABG', 'TUBG', 'BRAN', 'CACG', 'CANG', 'MUSW', 'TRUS', 'TUSW', 'WHOS', 'EGGO', 'MUDU', 'WODU', 'manduc', 'GARG', 'BWTE', 'CITE', 'NSHO', 'GADW', 'FADU', 'EUWI', 'AMWI', 'NOPI', 'GWTE', 'CANV', 'REDH', 'COMP', 'RNDU', 'TUDU', 'GRSC', 'LESC', 'KIEI', 'COEI', 'HADU', 'SUSC', 'COSC', 'BLSC', 'LTDU', 'BUFF', 'COGO', 'BAGO', 'HOME', 'COME', 'RBME', 'RUDU', 'MOUQ', 'CAQU', 'GAQU', 'INPE', 'CHUK', 'RNEP', 'RUGR', 'GRSG', 'WTPT', 'SOGR', 'WITU', 'PBGR', 'HOGR', 'RNGR', 'EAGR', 'WEGR', 'CLGR', 'RODO', 'BTPI', 'EUCD', 'AFCD', 'SPDO', 'INDO', 'COGD', 'RUGD', 'WWDO', 'MODO', 'GBAN', 'GRRO', 'YBCU', 'BBCU', 'COCU', 'LENI', 'CONI', 'COPO', 'EWPW', 'MWPW', 'BLSW', 'CHSW', 'VASW', 'COSW', 'WTSW', 'BTHH', 'RTHU', 'BCHU', 'ANHU', 'COHU', 'BTAH', 'RUHU', 'ALHU', 'CAHU', 'BBIH', 'VCHU', 'RIRA', 'VIRA', 'SORA', 'COGA', 'AMCO', 'PUGA', 'YERA', 'BLRA', 'SACR', 'CCRA', 'BNST', 'AMAV', 'AMOY', 'BLOY', 'BBPL', 'AMGP', 'PAGP', 'LSAP', 'SNPL', 'WIPL', 'CRPL', 'SEPL', 'KILL', 'MOPL', 'UPSA', 'WHIM', 'LBCU', 'BTGO', 'HUGO', 'MAGO', 'RUTU', 'BLTU', 'REKN', 'SURF', 'RUFF', 'SPTS', 'STSA', 'CUSA', 'RNST', 'SAND', 'DUNL', 'ROSA', 'PUSA', 'BASA', 'LIST', 'LESA', 'WRSA', 'BBSA', 'PESA', 'SESA', 'WESA', 'SBDO', 'LBDO', 'AMWO', 'COSN', 'WISN', 'WIPH', 'RNPH', 'REPH', 'SPSA', 'SOSA', 'WATA', 'GRYE', 'WILL', 'LEYE', 'MASA', 'WOSA', 'SPSK', 'POJA', 'PAJA', 'LTJA', 'COMU', 'TBMU', 'PIGU', 'LBMU', 'MAMU', 'SCMU', 'GUMU', 'CRMU', 'ANMU', 'CAAU', 'PAAU', 'RHAU', 'HOPU', 'TUPU', 'STGU', 'BLKI', 'IVGU', 'SAGU', 'BOGU', 'BHGU', 'LIGU', 'ROGU', 'LAGU', 'FRGU', 'BTGU', 'HEEG', 'MEGU', 'RBGU', 'WEGU', 'YFGU', 'CAGU', 'HERG', 'ICGU', 'LBBG', 'SBGU', 'GWGU', 'GLGU', 'GBBG', 'KEGU', 'SOTE', 'BRTE', 'LETE', 'GBTE', 'CATE', 'BLTE', 'COTE', 'ARTE', 'FOTE', 'ROYT', 'ELTE', 'BLSK', 'RBTR', 'RTTR', 'RTLO', 'ARLO', 'PALO', 'COLO', 'YBLO', 'SAAL', 'LAAL', 'BFAL', 'STAL', 'WISP', 'FTSP', 'LESP', 'TOSP', 'ASSP', 'BSTP', 'WRSP', 'BLSP', 'LSTP', 'NOFU', 'GFPE', 'KEPE', 'MUPE', 'MOPE', 'HAPE', 'COPE', 'STPE', 'JOPE', 'WCPE', 'COSH', 'PFSH', 'FFSH', 'GRSH', 'WTSH', 'BULS', 'SOSH', 'SRTS', 'MASH', 'BVSH', 'WOST', 'MAFR', 'GREF', 'MABO', 'NABO', 'BFBO', 'BRBO', 'RFBO', 'NOGA', 'BRAC', 'PECO', 'NECO', 'DCCO', 'AWPE', 'BRPE', 'AMBI', 'LEBI', 'GBHE', 'GREG', 'SNEG', 'LBHE', 'TRHE', 'REEG', 'CAEG', 'GRHE', 'BCNH', 'YCNH', 'WHIB', 'GLIB', 'WFIB', 'ROSP', 'CACO', 'BLVU', 'TUVU', 'OSPR', 'WTKI', 'STKI', 'GOEA', 'MIKI', 'NOHA', 'SSHA', 'COHA', 'NOGO', 'BAEA', 'COBH', 'HASH', 'GRHA', 'RSHA', 'BWHA', 'SWHA', 'ZTHA', 'RTHA', 'RLHA', 'FEHA', 'BANO', 'FLOW', 'WESO', 'GHOW', 'SNOW', 'NOPO', 'ELOW', 'BUOW', 'SPOW', 'BADO', 'GGOW', 'LEOW', 'SEOW', 'NSWO', 'BEKI', 'EUWR', 'WISA', 'YBSA', 'RNSA', 'RBSA', 'LEWO', 'ACWO', 'GIWO', 'BBWO', 'DOWO', 'NUWO', 'LBWO', 'HAWO', 'WHWO', 'PIWO', 'NOFL', 'GIFL', 'CRCA', 'AMKE', 'MERL', 'GYRF', 'PEFA', 'PRFA', 'RRPA', 'MOPA', 'WWPA', 'yecpar', 'RCPA', 'LCPA', 'RLPA', 'YHPA', 'YCPA', 'TFAM', 'WFPA', 'NAPA', 'bucpar', 'MIPA', 'RMPA', 'OSFL', 'GRPE', 'WEWP', 'EAWP', 'YBFL', 'ALFL', 'WIFL', 'LEFL', 'HAFL', 'GRFL', 'DUFL', 'PSFL', 'COFL', 'BBFL', 'BLPH', 'EAPH', 'SAPH', 'VEFL', 'DCFL', 'ATFL', 'GCFL', 'BCFL', 'SBFL', 'TRKI', 'COKI', 'CAKI', 'TBKI', 'WEKI', 'EAKI', 'STFL', 'BROS', 'LOSH', 'NSHR', 'WEVI', 'BEVI', 'GRVI', 'HUVI', 'YTVI', 'CAVI', 'BHVI', 'PLVI', 'PHVI', 'WAVI', 'YGVI', 'GRAJ', 'BTMJ', 'PIJA', 'STJA', 'BLJA', 'ISSJ', 'CASJ', 'WOSJ', 'BBMA', 'YBMA', 'CLNU', 'AMCR', 'CORA', 'HOLA', 'EUSK', 'NRWS', 'PUMA', 'TRES', 'VGSW', 'BANS', 'BARS', 'CLSW', 'CASW', 'BCCH', 'MOCH', 'CBCH', 'OATI', 'JUTI', 'VERD', 'BUSH', 'RBNU', 'WBNU', 'PYNU', 'BRCR', 'ROWR', 'CANW', 'HOWR', 'PAWR', 'WIWR', 'SEWR', 'MAWR', 'BEWR', 'CACW', 'BGGN', 'CAGN', 'BTGN', 'AMDI', 'RWBU', 'GCKI', 'RCKI', 'DUWA', 'ARWA', 'WREN', 'JAWE', 'BLUE', 'RFBL', 'NOWH', 'WEBL', 'MOBL', 'TOSO', 'VATH', 'VEER', 'GCTH', 'SWTH', 'HETH', 'WOTH', 'AMRO', 'RBRO', 'GRCA', 'CBTH', 'BRTH', 'BETH', 'CATH', 'LCTH', 'CRTH', 'SATH', 'NOMO', 'EUST', 'EYWA', 'CIWA', 'WHWA', 'OBPI', 'RTPI', 'AMPI', 'SPPI', 'BOWA', 'CEDW', 'PHAI', 'BRAM', 'EVGR', 'HAWF', 'PIGR', 'GCRF', 'BLRF', 'HOFI', 'PUFI', 'CAFI', 'CORE', 'RECR', 'WWCR', 'EUGO', 'PISI', 'LEGO', 'LAGO', 'AMGO', 'LALO', 'CCLO', 'SMLO', 'MCLO', 'SNBU', 'LIBU', 'RUBU', 'CASP', 'GRSP', 'CHSP', 'CCSP', 'BCSP', 'FISP', 'BRSP', 'BTSP', 'LASP', 'LARB', 'ATSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'HASP', 'WTSP', 'SABS', 'BESP', 'VESP', 'LCSP', 'NESP', 'SAVS', 'BAIS', 'SOSP', 'LISP', 'SWSP', 'ABTO', 'CALT', 'RCSP', 'GTTO', 'SPTO', 'YBCH', 'YHBL', 'BOBO', 'WEME', 'EAME', 'OROR', 'HOOR', 'SBAO', 'BUOR', 'BAOR', 'SCOR', 'RWBL', 'TRBL', 'BROC', 'BHCO', 'RUBL', 'BRBL', 'COGR', 'GTGR', 'OVEN', 'WEWA', 'LOWA', 'NOWA', 'GWWA', 'BWWA', 'BAWW', 'PROW', 'TEWA', 'OCWA', 'LUWA', 'NAWA', 'VIWA', 'CONW', 'MGWA', 'MOWA', 'KEWA', 'COYE', 'HOWA', 'AMRE', 'CMWA', 'CERW', 'NOPA', 'TRPA', 'MAWA', 'BBWA', 'BLBW', 'YEWA', 'CSWA', 'BLPW', 'BTBW', 'PAWA', 'PIWA', 'YRWA', 'YTWA', 'PRAW', 'GRWA', 'BTYW', 'TOWA', 'HEWA', 'BTNW', 'CAWA', 'WIWA', 'RFWA', 'PARE', 'HETA', 'SUTA', 'SCTA', 'WETA', 'NOCA', 'PYRR', 'RBGR', 'BHGR', 'BLGR', 'LAZB', 'INBU', 'VABU', 'PABU', 'DICK', 'HOSP', 'NRBI', 'OCHW', 'BRMA', 'SBMU', 'PTWH')
    --# and species not in ('SOSP')
    and S.quality in ('A', 'B')
    --# and species_species_group not in ('Wrens')
    --# and species_species_group in ('Wrens')
    and source_id not in ('xc:72445')
--# limit 30

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan -Olen
select distinct source_id, species
from search_recs

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan -Olen
select distinct xc_id, species
from search_recs

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3 -Iplan -Olen
select xc_id, species
from search_recs

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3
select
    quality as x,
    count(*) as y
from search_recs
where species='CORA'
group by x
order by y desc
limit 50

- MERGE (UNION ALL):
  - LEFT:
    - MERGE (UNION ALL):
      - LEFT:
        - MERGE (UNION ALL):
          - LEFT:
            - MERGE (UNION ALL):
              - LEFT:
                - MERGE (UNION ALL):
                  - LEFT:
                    - MERGE (UNION ALL):
                      - LEFT:
                        - MERGE (UNION ALL):
                          - LEFT:
                            - MERGE (UNION ALL):
                              - LEFT:
                                - MERGE (UNION ALL):
                                  - LEFT:
                                    - CO-ROUTINE 2:
                                      - SEARCH TABLE search_recs AS S USING INDEX
                                        ix_search_recs__species__species_species_group__quality__source_id
                                        (species=?)
                                      - USE TEMP B-TREE FOR ORDER BY
                                    - SCAN SUBQUERY 2
                 

Running query...

[0.487s, sqlite:////Users/danb/hack/birdgram/mobile/app/assets/payloads/US/search_recs/search_recs.sqlite3]

,month_day,shorthand,species_taxon_id,source_id,variable,channels,n_background_species,background_species,taxon_id,license_detail,species_order,license_type,background,samples_mb,state,species_taxon_order,basename,playback_used,place,speed,length,samples_n,lng,volume,duration_s,recordist,bird_seen,downloaded,state_only,xc_channels,download,norm_f_preds,species,species_family,country,xc_bitrate_of_mp3,dataset,recs_for_sp,species_sci_name,sample_width_bit,quality,date,time,xc_sampling_rate,pitch,url,year,license,hour,elevation,species_species_group,place_only,species_com_name,taxon_order,sample_rate,locality,lat,xc_id,path,number_of_notes,month,id,species_query,subspecies,species_longhand,type,remarks,d_pc,slp
0,04-02,INBU,TC011444,xc:363046,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.158,"Florida, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Orlando Wetlands, Christmas, Orange County, Florida, United States",Not specified,Not specified,82944,-81.0,Not specified,3.76,Paul Marvin,yes,1,Florida,1 (mono),//www.xeno-canto.org/363046/download,0.088,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2017-04-02 00:00:00.000000,08:36,48000 (Hz),Not specified,https://www.xeno-canto.org/363046,2020.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,0 m,"Cardinals, Grosbeaks, and Allies","Orlando Wetlands, Christmas, Orange County",Indigo Bunting,30322.0,22050,"Orlando Wetlands, Christmas, Orange County, Florida",28.6,363046,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",Not specified,4.0,"cache/audio/xc/data/INBU/363046/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,call,buzz call,0.285,3.75
1,05-22,INBU,TC011444,xc:135621,no,1,0,[],TC011444,3.0,Passeriformes,by-nc-sa,[],0.422,"Tenessee, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Cove Lake State Park, Caryville, Tenessee, United States",level,0-3(s),221184,-84.2,level,10.00,Mike Nelson,yes,1,Tenessee,2 (stereo),//www.xeno-canto.org/135621/download,0.093,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,B,2013-05-22 00:00:00.000000,11:30,44100 (Hz),both,https://www.xeno-canto.org/135621,2010.0,//creativecommons.org/licenses/by-nc-sa/3.0/,11.0,320 m,"Cardinals, Grosbeaks, and Allies","Cove Lake State Park, Caryville",Indigo Bunting,30322.0,22050,"Cove Lake State Park, Caryville, Tenessee",36.3,135621,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",7-20,5.0,"cache/audio/xc/data/INBU/135621/audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",INBU,,Indigo Bunting - INBU,song,Very odd song from this species. Seen well in a area of open deciduous trees.,0.327,3.75
2,06-10,INBU,TC011444,xc:192316,Not specified,1,0,[],TC011444,4.0,Passeriformes,by-nc-sa,[],0.422,"Virginia, United States",30322.0,"audio.mp3.resample(22050,1,16).enc(wav).slice(0,10000).spectro_denoise().enc(mp4,libfdk_aac,32k)",no,"Shenandoah National Park (near Elkton), Rockingham County, Virginia, United States",Not specified,Not specified,221184,-78.6,Not specified,10.00,Paul Marvin,no,1,Virginia,1 (mono),//www.xeno-canto.org/192316/download,0.118,INBU,Cardinalidae (Cardinals and Allies),United States,128000 (bps),xc,178,Passerina cyanea,16,A,2012-06-10 00:00:00.000000,08:22,48000 (Hz),Not specified,https://www.xeno-canto.org/192316,2010.0,//creativecommons.org/licenses/by-nc-sa/4.0/,8.0,750 m,"Cardinals, Grosbeaks, and Allies","Shenandoah National Park (near Elkton), Rockingham County",Indigo Bunting,30322.0,22050,"Shenandoah National Park (near Elkton), Rockingham County, V

In [ ]:
%%sqla -ceng -p3
select xc_id, country, state, lat, lng
from (
    select *
    from search_recs
    order by random()
    limit 10
)
order by lat desc

In [ ]:
%%sqla -ceng -p3 -T
select *
from search_recs
limit 1

In [ ]:
%%sqla -ceng -p3
select species, species_longhand, species_com_name
from search_recs
where species_longhand like '%mallar%'
limit 10

# 2020-01-11: Retry doing window query
- Now that we've successfully upgraded sqlite (and fixed the US crash)

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO (0/n): Baseline, copied from app logs

                with
                  -- For nested sqlPerSpecies queries
                  S_filter_dist as (
                    select

                      S.species, S.source_id,
                      --# S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.species_order, S.license_type, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks,

                      1 - ((((S.f_preds_0*0.00001311239248025231 + S.f_preds_1*0.0022739178966730833 + S.f_preds_2*0.000004044589331897441 + S.f_preds_3*0.002446443075314164 + S.f_preds_4*0.000007628525054315105 + S.f_preds_5*0.00001568856168887578 + S.f_preds_6*0.000006424799266824266 + S.f_preds_7*2.9643587140526506e-7 + S.f_preds_8*0.0000022504916614707327 + S.f_preds_9*0.00000209387690119911 + S.f_preds_10*0.00002307513750565704 + S.f_preds_11*0.001311492407694459 + S.f_preds_12*0.000015103913938219193 + S.f_preds_13*8.688106731824519e-7 + S.f_preds_14*0.00005533318835659884 + S.f_preds_15*0.000016482756109326147 + S.f_preds_16*0.0005267483065836132 + S.f_preds_17*0.0018814759096130729 + S.f_preds_18*0.0000017237175597983878 + S.f_preds_19*0.00039833024493418634 + S.f_preds_20*0.00000977200579654891 + S.f_preds_21*0.000006569051038241014 + S.f_preds_22*0.000012760150639223866 + S.f_preds_23*0.000009890945875667967 + S.f_preds_24*0.01804490014910698 + S.f_preds_25*0.004844821989536285 + S.f_preds_26*0.00023922703985590488 + S.f_preds_27*0.000015565292414976284 + S.f_preds_28*0.000010527898666623514 + S.f_preds_29*0.005573070142418146 + S.f_preds_30*0.000028921585908392444 + S.f_preds_31*0.0000033035873912012903 + S.f_preds_32*4.100716068933252e-7 + S.f_preds_33*6.601695758945425e-7 + S.f_preds_34*0.0004126813728362322 + S.f_preds_35*0.00003285566344857216 + S.f_preds_36*0.0018572922563180327 + S.f_preds_37*0.0006297582876868546 + S.f_preds_38*0.0001985664275707677 + S.f_preds_39*0.000013924099221185315 + S.f_preds_40*0.00005194657205720432 + S.f_preds_41*0.000042426461732247844 + S.f_preds_42*0.0005176834529265761 + S.f_preds_43*0.01681959442794323 + S.f_preds_44*0.00021062373707536608 + S.f_preds_45*8.841304293127905e-7 + S.f_preds_46*0.000049778231186792254 + S.f_preds_47*0.018970871344208717 + S.f_preds_48*0.0007048664847388864 + S.f_preds_49*0.000028281960112508386 + S.f_preds_50*0.0006428731721825898 + S.f_preds_51*0.0028357082046568394 + S.f_preds_52*0.0000034002564461843576 + S.f_preds_53*0.023662345483899117 + S.f_preds_54*0.00044131523463875055 + S.f_preds_55*0.00241877231746912 + S.f_preds_56*0.00003444610410952009 + S.f_preds_57*0.0014652458485215902 + S.f_preds_58*0.0020906205754727125 + S.f_preds_59*0.000013210412362241186 + S.f_preds_60*0.0001858155446825549 + S.f_preds_61*0.007895379327237606 + S.f_preds_62*0.0000024245182430604473 + S.f_preds_63*0.0000026513437205721857 + S.f_preds_64*0.0009304345585405827 + S.f_preds_65*0.00023366720415651798 + S.f_preds_66*0.005655152257531881 + S.f_preds_67*0.0009726776042953134 + S.f_preds_68*0.000012197562682558782 + S.f_preds_69*1.0011000739496012e-7 + S.f_preds_70*0.00020311045227572322 + S.f_preds_71*0.00025495662703178823 + S.f_preds_72*0.0008810111903585494 + S.f_preds_73*0.00011881050158990547 + S.f_preds_74*0.01844712905585766 + S.f_preds_75*0.0014010248705744743 + S.f_preds_76*6.89977426304722e-8 + S.f_preds_77*0.000001391107161907712 + S.f_preds_78*0.00006117242446634918 + S.f_preds_79*0.010232416912913322 + S.f_preds_80*0.000001088506337509898 + S.f_preds_81*0.023149210959672928 + S.f_preds_82*0.0218129251152277 + S.f_preds_83*0.00027983225299976766 + S.f_preds_84*0.000002296665115864016 + S.f_preds_85*0.00029460497898980975 + S.f_preds_86*0.0011194923426955938 + S.f_preds_87*0.01881752721965313 + S.f_preds_88*0.00012937233259435743 + S.f_preds_89*0.006397274788469076 + S.f_preds_90*0.000016658137610647827 + S.f_preds_91*0.00017631007358431816) + (S.f_preds_92*0.00012058904394507408 + S.f_preds_93*0.00005393704850575887 + S.f_preds_94*0.004301635082811117 + S.f_preds_95*0.0000014354379800352035 + S.f_preds_96*0.000005481480002345052 + S.f_preds_97*0.00009395895904162899 + S.f_preds_98*0.00007922713120933622 + S.f_preds_99*1.1477159489459154e-7 + S.f_preds_100*0.0003226808912586421 + S.f_preds_101*0.00006615019083255902 + S.f_preds_102*6.292503371696512e-7 + S.f_preds_103*0.00010970210860250518 + S.f_preds_104*0.0005167542258277535 + S.f_preds_105*0.0000012490669405451627 + S.f_preds_106*0.00039587795617990196 + S.f_preds_107*0.005248239729553461 + S.f_preds_108*1.9826785546683823e-7 + S.f_preds_109*0.00005959632107988 + S.f_preds_110*0.00014260836178436875 + S.f_preds_111*0.00010338414722355083 + S.f_preds_112*0.0000205065243790159 + S.f_preds_113*0.0000020813254195672926 + S.f_preds_114*0.00020459122606553137 + S.f_preds_115*0.002042416948825121 + S.f_preds_116*0.00006533438136102632 + S.f_preds_117*0.000005608823357761139 + S.f_preds_118*0.001865064725279808 + S.f_preds_119*0.00007586278661619872 + S.f_preds_120*0.0004040422209072858 + S.f_preds_121*0.0007578746299259365 + S.f_preds_122*0.0037188904825598 + S.f_preds_123*0.014093589968979359 + S.f_preds_124*0.0002631600946187973 + S.f_preds_125*0.00681808777153492 + S.f_preds_126*3.617186052906618e-7 + S.f_preds_127*0.002112553222104907 + S.f_preds_128*0.0025430587120354176 + S.f_preds_129*0.00016653123020660132 + S.f_preds_130*0.02345724031329155 + S.f_preds_131*0.002475341083481908 + S.f_preds_132*0.00014247142826206982 + S.f_preds_133*0.0005273931892588735 + S.f_preds_134*0.000013719729395234026 + S.f_preds_135*0.0035834198351949453 + S.f_preds_136*0.0000024723869955778355 + S.f_preds_137*0.017516305670142174 + S.f_preds_138*0.00004870363409281708 + S.f_preds_139*0.00013708391634281725 + S.f_preds_140*0.000009344155841972679 + S.f_preds_141*0.00001242540565726813 + S.f_preds_142*0.023233819752931595 + S.f_preds_143*0.000028271842893445864 + S.f_preds_144*0.013134469278156757 + S.f_preds_145*0.0007207064772956073 + S.f_preds_146*0.00003083339106524363 + S.f_preds_147*0.000008576449545216747 + S.f_preds_148*0.016772819682955742 + S.f_preds_149*0.0009123460622504354 + S.f_preds_150*0.012531428597867489 + S.f_preds_151*0.0002033248165389523 + S.f_preds_152*0.00003387439573998563 + S.f_preds_153*0.00012878944107796997 + S.f_preds_154*0.000011766812349378597 + S.f_preds_155*0.0035626592580229044 + S.f_preds_156*0.0000020471034076763317 + S.f_preds_157*0.0000035694906728167553 + S.f_preds_158*0.00015981854812707752 + S.f_preds_159*0.0011637889547273517 + S.f_preds_160*0.00387061876244843 + S.f_preds_161*0.00008862809045240283 + S.f_preds_162*0.00006104543717810884 + S.f_preds_163*0.00011629491928033531 + S.f_preds_164*0.00004587496732710861 + S.f_preds_165*0.0000650517686153762 + S.f_preds_166*0.0017447016434744 + S.f_preds_167*0.021712763234972954 + S.f_preds_168*0.0005890600150451064 + S.f_preds_169*6.537584908983263e-7 + S.f_preds_170*3.564814790024684e-7 + S.f_preds_171*0.0009296662174165249 + S.f_preds_172*0.00006215124449227005 + S.f_preds_173*0.012565955519676208 + S.f_preds_174*0.007263803388923407 + S.f_preds_175*0.0027377239894121885 + S.f_preds_176*0.000008244623131758999 + S.f_preds_177*0.0000023269387838809052 + S.f_preds_178*0.00022972709848545492 + S.f_preds_179*0.00002270613913424313 + S.f_preds_180*2.0221226293415384e-7 + S.f_preds_181*0.000009496605343883857 + S.f_preds_182*0.000005108888672111789 + S.f_preds_183*0.0005137779517099261)) + ((S.f_preds_184*0.000010669865332602058 + S.f_preds_185*0.0009854049421846867 + S.f_preds_186*0.0012607353273779154 + S.f_preds_187*0.0006667971611022949 + S.f_preds_188*0.000014654996448371094 + S.f_preds_189*2.450668716846849e-7 + S.f_preds_190*0.00021140761964488775 + S.f_preds_191*0.00004480480129132047 + S.f_preds_192*0.00010504756937734783 + S.f_preds_193*0.00002219480484200176 + S.f_preds_194*0.0014585570897907019 + S.f_preds_195*0.000023570381017634645 + S.f_preds_196*0.0035868771374225616 + S.f_preds_197*0.000026636558686732315 + S.f_preds_198*5.375615046432358e-7 + S.f_preds_199*0.004330820869654417 + S.f_preds_200*0.000013430114449874964 + S.f_preds_201*0.00009262594539904967 + S.f_preds_202*0.00008550980419386178 + S.f_preds_203*0.00005542610233533196 + S.f_preds_204*0.000022133155653136782 + S.f_preds_205*0.0016019666800275445 + S.f_preds_206*8.774132993494277e-7 + S.f_preds_207*8.674058449287259e-7 + S.f_preds_208*0.00004084217653144151 + S.f_preds_209*0.000011412805179134011 + S.f_preds_210*0.0005518625839613378 + S.f_preds_211*0.00008674130367580801 + S.f_preds_212*0.0000026616432933224132 + S.f_preds_213*0.000023289498130907305 + S.f_preds_214*0.00001417942166881403 + S.f_preds_215*0.00011560232087504119 + S.f_preds_216*0.000008560010428482201 + S.f_preds_217*0.0010935317259281874 + S.f_preds_218*0.00014956091763451695 + S.f_preds_219*0.0005274588475003839 + S.f_preds_220*0.0003844208549708128 + S.f_preds_221*0.008620514534413815 + S.f_preds_222*4.901575039184536e-7 + S.f_preds_223*0.000002932444431280601 + S.f_preds_224*0.00008497372618876398 + S.f_preds_225*0.000006627227776334621 + S.f_preds_226*0.00003267589272581972 + S.f_preds_227*3.307827682874631e-7 + S.f_preds_228*0.003242308972403407 + S.f_preds_229*0.00006843778101028875 + S.f_preds_230*0.000021847368770977482 + S.f_preds_231*0.0000022423150767281186 + S.f_preds_232*0.0026875378098338842 + S.f_preds_233*0.00005942605639575049 + S.f_preds_234*0.000003830415607808391 + S.f_preds_235*0.00003467744681984186 + S.f_preds_236*0.0003422269073780626 + S.f_preds_237*0.00003503493280732073 + S.f_preds_238*0.0000010439758852953673 + S.f_preds_239*0.0000018763837488222634 + S.f_preds_240*0.00012398957915138453 + S.f_preds_241*0.0011635670671239495 + S.f_preds_242*0.0000011317058579152217 + S.f_preds_243*0.015879152342677116 + S.f_preds_244*0.00003913496766472235 + S.f_preds_245*0.000037293462810339406 + S.f_preds_246*0.0000361335369234439 + S.f_preds_247*0.00005024169513490051 + S.f_preds_248*0.0013123423559591174 + S.f_preds_249*0.0000015925746765788062 + S.f_preds_250*0.0003526984073687345 + S.f_preds_251*0.0015677057672291994 + S.f_preds_252*0.000027051797587773763 + S.f_preds_253*0.00004910511779598892 + S.f_preds_254*0.0000018012084410656826 + S.f_preds_255*0.0000886311536305584 + S.f_preds_256*0.0005031937034800649 + S.f_preds_257*0.0002624462067615241 + S.f_preds_258*0.00016602191317360848 + S.f_preds_259*0.00024625565856695175 + S.f_preds_260*0.000034680469980230555 + S.f_preds_261*0.000011574317795748357 + S.f_preds_262*0.0006084603955969214 + S.f_preds_263*0.0000021471892068802845 + S.f_preds_264*0.00001021452590066474 + S.f_preds_265*0.00019133792375214398 + S.f_preds_266*0.000016060286725405604 + S.f_preds_267*0.00014434140757657588 + S.f_preds_268*0.00015234870079439133 + S.f_preds_269*0.000001094846425075957 + S.f_preds_270*0.000011648219697235618 + S.f_preds_271*0.0004886073293164372 + S.f_preds_272*0.0016627265140414238 + S.f_preds_273*0.01329568587243557 + S.f_preds_274*3.138311797101778e-7 + S.f_preds_275*0.0003020265430677682) + (S.f_preds_276*6.091020736676e-7 + S.f_preds_277*0.000008524235454387963 + S.f_preds_278*0.00020711110846605152 + S.f_preds_279*0.000003148366204186459 + S.f_preds_280*0.00014961144188418984 + S.f_preds_281*0.000024720802684896626 + S.f_preds_282*0.00008777557377470657 + S.f_preds_283*0.000008208664439735003 + S.f_preds_284*0.000012109705494367518 + S.f_preds_285*6.579759883607039e-7 + S.f_preds_286*0.000012348501513770316 + S.f_preds_287*0.0007436243467964232 + S.f_preds_288*0.000004955889380653389 + S.f_preds_289*8.595026201874134e-7 + S.f_preds_290*0.007464772555977106 + S.f_preds_291*0.0001946113770827651 + S.f_preds_292*0.00007105382974259555 + S.f_preds_293*0.0033582847099751234 + S.f_preds_294*0.0006477180868387222 + S.f_preds_295*3.6187074670124275e-7 + S.f_preds_296*0.0000693765832693316 + S.f_preds_297*0.0000033425619676563656 + S.f_preds_298*0.0000029267844183777925 + S.f_preds_299*0.0000039293249756156 + S.f_preds_300*0.000038189460610738024 + S.f_preds_301*5.310744768394215e-7 + S.f_preds_302*0.0000017443657043259009 + S.f_preds_303*0.000011264039130765013 + S.f_preds_304*0.000007809535418346059 + S.f_preds_305*0.00004670179259846918 + S.f_preds_306*0.00004879724292550236 + S.f_preds_307*0.0015165400691330433 + S.f_preds_308*0.000002572912762843771 + S.f_preds_309*0.0009610670967958868 + S.f_preds_310*0.000015019456441223156 + S.f_preds_311*0.000018154683857574128 + S.f_preds_312*0.00042174625559709966 + S.f_preds_313*0.00000472815827379236 + S.f_preds_314*0.022676438093185425 + S.f_preds_315*5.847223860655504e-7 + S.f_preds_316*0.000004039436589664547 + S.f_preds_317*0.000008301932211907115 + S.f_preds_318*0.000013070617569610476 + S.f_preds_319*0.00009437942935619503 + S.f_preds_320*9.450283044998287e-8 + S.f_preds_321*0.00013990449951961637 + S.f_preds_322*0.00000192821448763425 + S.f_preds_323*0.0000013222486359154573 + S.f_preds_324*0.000052494549890980124 + S.f_preds_325*0.000007503287179133622 + S.f_preds_326*0.0000017911829672812019 + S.f_preds_327*0.00011045519931940362 + S.f_preds_328*0.0022690247278660536 + S.f_preds_329*0.0001165569992735982 + S.f_preds_330*0.0000018839571112039266 + S.f_preds_331*0.000032350140827475116 + S.f_preds_332*0.00002531428799557034 + S.f_preds_333*0.000050617811211850494 + S.f_preds_334*0.0007663340657018125 + S.f_preds_335*0.00013900290650781244 + S.f_preds_336*0.000049367572501068935 + S.f_preds_337*0.000042445550207048655 + S.f_preds_338*0.0001376005238853395 + S.f_preds_339*0.00004797595465788618 + S.f_preds_340*0.000022352229279931635 + S.f_preds_341*0.0002962714934255928 + S.f_preds_342*0.00004398026067065075 + S.f_preds_343*0.01037600077688694 + S.f_preds_344*0.00004516577610047534 + S.f_preds_345*0.0006199416238814592 + S.f_preds_346*0.0000012476348274503835 + S.f_preds_347*0.000004629550403478788 + S.f_preds_348*0.023418789729475975 + S.f_preds_349*0.00000777882360125659 + S.f_preds_350*0.000029099664970999584 + S.f_preds_351*0.000014523630852636416 + S.f_preds_352*0.0000023763323042658158 + S.f_preds_353*0.0005294025759212673 + S.f_preds_354*0.000005048519142292207 + S.f_preds_355*0.0000032741434097260935 + S.f_preds_356*0.000007565846317447722 + S.f_preds_357*0.00004547514981823042 + S.f_preds_358*0.0002636595454532653 + S.f_preds_359*1.800865874201918e-7 + S.f_preds_360*0.00001865157719294075 + S.f_preds_361*0.000376529002096504 + S.f_preds_362*0.0000027501305339683313 + S.f_preds_363*0.00030277753830887377 + S.f_preds_364*0.00005115754538564943 + S.f_preds_365*0.0012324352283030748 + S.f_preds_366*0.00018544473277870566 + S.f_preds_367*0.00005313345172908157 + S.f_preds_368*0.000027433607101556845))) + (((S.f_preds_369*0.00020436079648789018 + S.f_preds_370*0.0006155724404379725 + S.f_preds_371*0.0000035860705338564003 + S.f_preds_372*0.000004185253146715695 + S.f_preds_373*0.00005271961708785966 + S.f_preds_374*0.000002029272991421749 + S.f_preds_375*0.00018188006652053446 + S.f_preds_376*0.000012025258911307901 + S.f_preds_377*0.0014208576176315546 + S.f_preds_378*9.447777671311997e-8 + S.f_preds_379*0.00006892470992170274 + S.f_preds_380*0.00003242859020247124 + S.f_preds_381*0.0007282765000127256 + S.f_preds_382*0.00375916319899261 + S.f_preds_383*0.0000028596371066669235 + S.f_preds_384*0.000009051633242052048 + S.f_preds_385*0.0011416266206651926 + S.f_preds_386*0.0000043961154005955905 + S.f_preds_387*0.000008625760528957471 + S.f_preds_388*0.00009348194726044312 + S.f_preds_389*0.000010609912351355888 + S.f_preds_390*0.0005257150623947382 + S.f_preds_391*0.00009466418850934133 + S.f_preds_392*0.000017193688108818606 + S.f_preds_393*0.007605379913002253 + S.f_preds_394*0.0004980259109288454 + S.f_preds_395*0.0007890399429015815 + S.f_preds_396*2.0968185410197293e-8 + S.f_preds_397*8.087992569016933e-7 + S.f_preds_398*6.560824772350315e-7 + S.f_preds_399*0.00014008089783601463 + S.f_preds_400*0.0006039145519025624 + S.f_preds_401*0.0000071473286880063824 + S.f_preds_402*0.00004870027623837814 + S.f_preds_403*0.000056747336202533916 + S.f_preds_404*0.0000023145769318944076 + S.f_preds_405*0.0008232579566538334 + S.f_preds_406*9.881814548862167e-7 + S.f_preds_407*0.0000017835916423791787 + S.f_preds_408*0.004612771328538656 + S.f_preds_409*0.000830142991617322 + S.f_preds_410*7.730937312544484e-8 + S.f_preds_411*0.012341471388936043 + S.f_preds_412*0.002177329733967781 + S.f_preds_413*0.00037074810825288296 + S.f_preds_414*0.0017332970164716244 + S.f_preds_415*0.0013398604933172464 + S.f_preds_416*0.0000012682459100687993 + S.f_preds_417*3.9440632804144116e-7 + S.f_preds_418*0.0000511453217768576 + S.f_preds_419*0.0000014611218830395956 + S.f_preds_420*1.3488853767285036e-7 + S.f_preds_421*0.0011070219334214926 + S.f_preds_422*0.000172565778484568 + S.f_preds_423*0.00000847523278935114 + S.f_preds_424*0.0000021629989532812033 + S.f_preds_425*0.00015388638712465763 + S.f_preds_426*1.244669363131834e-7 + S.f_preds_427*5.292966989145498e-7 + S.f_preds_428*0.0026793526485562325 + S.f_preds_429*0.00000242397072724998 + S.f_preds_430*0.0000017537361145514296 + S.f_preds_431*0.00007314266986213624 + S.f_preds_432*0.015580660663545132 + S.f_preds_433*0.00003077931614825502 + S.f_preds_434*0.001671489211730659 + S.f_preds_435*0.0051424503326416016 + S.f_preds_436*0.0000012573142385008396 + S.f_preds_437*0.0000017696989971227595 + S.f_preds_438*0.00003416148683754727 + S.f_preds_439*0.0008575073443353176 + S.f_preds_440*0.0004498166963458061 + S.f_preds_441*0.0014703682390972972 + S.f_preds_442*0.000014719891623826697 + S.f_preds_443*0.00017386121908202767 + S.f_preds_444*0.000073970302764792 + S.f_preds_445*0.00011574722157092765 + S.f_preds_446*0.005045986268669367 + S.f_preds_447*0.004470773506909609 + S.f_preds_448*0.000002035713578152354 + S.f_preds_449*0.000043419364374130964 + S.f_preds_450*0.012585037387907505 + S.f_preds_451*0.000001728835741232615 + S.f_preds_452*0.00000100265071978356 + S.f_preds_453*0.0015500244917348027 + S.f_preds_454*0.0013294772943481803 + S.f_preds_455*0.0010233717039227486 + S.f_preds_456*0.0010060714557766914 + S.f_preds_457*0.000006806942565162899 + S.f_preds_458*6.035304522811202e-7 + S.f_preds_459*0.00020313254208303988 + S.f_preds_460*0.003389031160622835) + (S.f_preds_461*0.0008531991043128073 + S.f_preds_462*0.00033116189297288656 + S.f_preds_463*0.000012784821592504159 + S.f_preds_464*0.0000283551707980223 + S.f_preds_465*0.0006966462824493647 + S.f_preds_466*0.00002789941027003806 + S.f_preds_467*0.0005911355256102979 + S.f_preds_468*0.002994079142808914 + S.f_preds_469*0.0000017838807480075047 + S.f_preds_470*2.4400554821113474e-7 + S.f_preds_471*5.014719022256031e-7 + S.f_preds_472*0.000011036318028345704 + S.f_preds_473*3.725356805261981e-7 + S.f_preds_474*1.266375591058022e-7 + S.f_preds_475*0.000018449984054313973 + S.f_preds_476*0.00003187067704857327 + S.f_preds_477*0.005098472349345684 + S.f_preds_478*0.0035877684131264687 + S.f_preds_479*0.00013991973537486047 + S.f_preds_480*0.00002065905755443964 + S.f_preds_481*9.214570582116721e-7 + S.f_preds_482*0.00005737582250731066 + S.f_preds_483*1.1891671647390467e-7 + S.f_preds_484*5.138089136380586e-7 + S.f_preds_485*0.00003446516711846925 + S.f_preds_486*0.00003241106605855748 + S.f_preds_487*0.00007031692803138867 + S.f_preds_488*0.00004199549584882334 + S.f_preds_489*0.00007063855446176603 + S.f_preds_490*1.3241979957001604e-7 + S.f_preds_491*0.0014177433913573623 + S.f_preds_492*0.000021480864234035835 + S.f_preds_493*0.00009588408283889294 + S.f_preds_494*0.0008136858814395964 + S.f_preds_495*0.00000723950142855756 + S.f_preds_496*0.000041920222429325804 + S.f_preds_497*0.000004934684966428904 + S.f_preds_498*0.01999528706073761 + S.f_preds_499*0.000009792875971470494 + S.f_preds_500*0.006254177540540695 + S.f_preds_501*0.0019237757660448551 + S.f_preds_502*0.0000336199882440269 + S.f_preds_503*0.00011587099288590252 + S.f_preds_504*5.696571747648704e-7 + S.f_preds_505*0.0000030567177873308538 + S.f_preds_506*0.000029180990168242715 + S.f_preds_507*0.000054005424317438155 + S.f_preds_508*0.0003109595272690058 + S.f_preds_509*0.000013896006748836953 + S.f_preds_510*0.00003571807610569522 + S.f_preds_511*0.002394929062575102 + S.f_preds_512*0.0001247953186975792 + S.f_preds_513*0.00014427122368942946 + S.f_preds_514*0.00009447086631553248 + S.f_preds_515*0.000035395132726989686 + S.f_preds_516*0.0030862544663250446 + S.f_preds_517*0.0020114730577915907 + S.f_preds_518*0.0015699266223236918 + S.f_preds_519*0.000022298700059764087 + S.f_preds_520*0.017703445628285408 + S.f_preds_521*0.0014080592663958669 + S.f_preds_522*0.005447646602988243 + S.f_preds_523*0.01645824871957302 + S.f_preds_524*0.000027788570150732994 + S.f_preds_525*0.00001340962899121223 + S.f_preds_526*0.000019065997548750602 + S.f_preds_527*0.000009252670679416042 + S.f_preds_528*0.0000025471656499576056 + S.f_preds_529*7.461294160293619e-8 + S.f_preds_530*0.00017533496429678053 + S.f_preds_531*0.000027902546207769774 + S.f_preds_532*0.000016202477127080783 + S.f_preds_533*0.00020475717610679567 + S.f_preds_534*0.0004499423084780574 + S.f_preds_535*0.0006966591463424265 + S.f_preds_536*0.000550146447494626 + S.f_preds_537*0.00016486116510350257 + S.f_preds_538*0.000007280455974978395 + S.f_preds_539*0.000010827132427948527 + S.f_preds_540*0.0000876995109138079 + S.f_preds_541*0.0003642363881226629 + S.f_preds_542*0.0000012616019375855103 + S.f_preds_543*0.0000019034998786082724 + S.f_preds_544*0.00005276263254927471 + S.f_preds_545*0.00005496077210409567 + S.f_preds_546*0.00027376782963983715 + S.f_preds_547*0.000005710881850973237 + S.f_preds_548*8.988230604245473e-9 + S.f_preds_549*0.0000016730429024391924 + S.f_preds_550*0.00007679287227801979 + S.f_preds_551*0.0000016976399592749658 + S.f_preds_552*0.000004411753707245225 + S.f_preds_553*0.000005111335212859558)) + ((S.f_preds_554*0.00016011434490792453 + S.f_preds_555*0.0000029296325010363944 + S.f_preds_556*0.00025140767684206367 + S.f_preds_557*0.00005884619167773053 + S.f_preds_558*0.0015645562671124935 + S.f_preds_559*0.00020754926663357764 + S.f_preds_560*0.00025478313909843564 + S.f_preds_561*0.002211499260738492 + S.f_preds_562*6.609280376324023e-7 + S.f_preds_563*0.0010862791677936912 + S.f_preds_564*0.000012614359548024368 + S.f_preds_565*0.00017907589790411294 + S.f_preds_566*0.0000037052925563330064 + S.f_preds_567*0.00010445003135828301 + S.f_preds_568*0.017669612541794777 + S.f_preds_569*0.00000553270638192771 + S.f_preds_570*0.00006034673788235523 + S.f_preds_571*0.00039125533658079803 + S.f_preds_572*0.000007338504929066403 + S.f_preds_573*0.000122875499073416 + S.f_preds_574*2.4581188995398406e-7 + S.f_preds_575*0.000011374868336133659 + S.f_preds_576*0.00008426068961853161 + S.f_preds_577*6.751050705133821e-7 + S.f_preds_578*0.0000064212285906251054 + S.f_preds_579*0.00005932396015850827 + S.f_preds_580*0.0004973491886630654 + S.f_preds_581*5.367184030546923e-7 + S.f_preds_582*0.000743536336813122 + S.f_preds_583*0.0000017930664171217359 + S.f_preds_584*0.0006554555729962885 + S.f_preds_585*0.0000037011327549407724 + S.f_preds_586*0.0002179535513278097 + S.f_preds_587*0.00003099400782957673 + S.f_preds_588*0.000004918352260574466 + S.f_preds_589*0.000009350197615276556 + S.f_preds_590*0.000016639018213027157 + S.f_preds_591*0.00010717164695961401 + S.f_preds_592*0.000023528449673904106 + S.f_preds_593*0.000001007815853881766 + S.f_preds_594*0.000005481845619215164 + S.f_preds_595*0.00015956266724970192 + S.f_preds_596*0.0002082450082525611 + S.f_preds_597*0.00020021837553940713 + S.f_preds_598*0.0010172325419262052 + S.f_preds_599*0.0000010955390052913572 + S.f_preds_600*0.0000026854265797737753 + S.f_preds_601*0.0015208498807623982 + S.f_preds_602*0.000006445457074732985 + S.f_preds_603*0.000020037914509885013 + S.f_preds_604*1.9325770495015604e-7 + S.f_preds_605*0.0000010705190334192594 + S.f_preds_606*0.000028866608772659674 + S.f_preds_607*0.00010479458433110267 + S.f_preds_608*0.010287586599588394 + S.f_preds_609*0.0012989990646019578 + S.f_preds_610*0.000003944566287827911 + S.f_preds_611*2.8879350466581855e-8 + S.f_preds_612*0.0000069853654167673085 + S.f_preds_613*0.000021991474568494596 + S.f_preds_614*0.00007703194569330662 + S.f_preds_615*0.00023750816762913018 + S.f_preds_616*0.00944328773766756 + S.f_preds_617*0.0000010323501555831172 + S.f_preds_618*1.501768736034137e-7 + S.f_preds_619*0.00007150989404181018 + S.f_preds_620*0.0000061345799622358754 + S.f_preds_621*0.0004368036170490086 + S.f_preds_622*8.976332992460812e-7 + S.f_preds_623*0.001628484926186502 + S.f_preds_624*0.000003928264959540684 + S.f_preds_625*0.0009367550374008715 + S.f_preds_626*0.0000026312400223105215 + S.f_preds_627*0.00006375551311066374 + S.f_preds_628*5.137369498697808e-7 + S.f_preds_629*0.00001137478193413699 + S.f_preds_630*0.000017766524251783267 + S.f_preds_631*0.003897836897522211 + S.f_preds_632*0.008055249229073524 + S.f_preds_633*0.00241384981200099 + S.f_preds_634*0.00003595540329115465 + S.f_preds_635*0.00005645097917295061 + S.f_preds_636*0.00006621936336159706 + S.f_preds_637*0.014510611072182655 + S.f_preds_638*1.050159710302978e-7 + S.f_preds_639*0.019783876836299896 + S.f_preds_640*0.008949201554059982 + S.f_preds_641*0.00024250347632914782 + S.f_preds_642*0.00010105391265824437 + S.f_preds_643*0.00004060524588567205 + S.f_preds_644*0.0001331030362052843 + S.f_preds_645*8.045903427955636e-7) + (S.f_preds_646*0.00001027736743708374 + S.f_preds_647*1.7172181898672534e-8 + S.f_preds_648*0.0033125479239970446 + S.f_preds_649*0.0000016601031802565558 + S.f_preds_650*0.00005077445530332625 + S.f_preds_651*0.0002987897605635226 + S.f_preds_652*0.002370136557146907 + S.f_preds_653*0.0005649944650940597 + S.f_preds_654*0.000028247362934052944 + S.f_preds_655*0.000047565263230353594 + S.f_preds_656*0.00001847093335527461 + S.f_preds_657*0.000002787693574646255 + S.f_preds_658*0.000005108099685458001 + S.f_preds_659*0.0010631985496729612 + S.f_preds_660*0.00004261540743755177 + S.f_preds_661*0.0002864064881578088 + S.f_preds_662*0.00009470334771322086 + S.f_preds_663*0.00006371989729814231 + S.f_preds_664*4.832561444345629e-7 + S.f_preds_665*0.0000020821610178245464 + S.f_preds_666*0.0002292780700372532 + S.f_preds_667*3.029888162586758e-8 + S.f_preds_668*0.00005065908044343814 + S.f_preds_669*0.00020125712035223842 + S.f_preds_670*0.00047489258577115834 + S.f_preds_671*3.227335909627982e-8 + S.f_preds_672*0.0001816168805817142 + S.f_preds_673*0.0000014918402939656517 + S.f_preds_674*0.000007308628937607864 + S.f_preds_675*0.00004637502206605859 + S.f_preds_676*0.00005088275429443456 + S.f_preds_677*0.0004129812878090888 + S.f_preds_678*0.02267596498131752 + S.f_preds_679*0.00004278955384506844 + S.f_preds_680*0.0005632776883430779 + S.f_preds_681*0.0000029753814487776253 + S.f_preds_682*0.0010443839710205793 + S.f_preds_683*0.000003015262109329342 + S.f_preds_684*0.0000015277154261639225 + S.f_preds_685*0.00008885315764928237 + S.f_preds_686*0.0001388078526360914 + S.f_preds_687*0.00009291117748944089 + S.f_preds_688*0.00005038944073021412 + S.f_preds_689*4.75471949812345e-7 + S.f_preds_690*1.248018861588207e-7 + S.f_preds_691*0.001185665954835713 + S.f_preds_692*0.00003074324922636151 + S.f_preds_693*9.277101753468742e-7 + S.f_preds_694*0.00022056890884414315 + S.f_preds_695*0.007895344868302345 + S.f_preds_696*0.013903992250561714 + S.f_preds_697*0.00014689314411953092 + S.f_preds_698*0.000002063769670712645 + S.f_preds_699*0.0009233330492861569 + S.f_preds_700*0.000016178952137124725 + S.f_preds_701*0.0010267875622957945 + S.f_preds_702*0.00005498975224327296 + S.f_preds_703*0.00013355587725527585 + S.f_preds_704*0.00002518714427424129 + S.f_preds_705*0.0001367626537103206 + S.f_preds_706*0.0002352102310396731 + S.f_preds_707*0.00003683730392367579 + S.f_preds_708*0.00019189013983123004 + S.f_preds_709*0.00040489446837455034 + S.f_preds_710*0.000008120999154925812 + S.f_preds_711*0.000005265692379907705 + S.f_preds_712*0.000013629485692945309 + S.f_preds_713*0.00010411084804218262 + S.f_preds_714*0.00010439017933094874 + S.f_preds_715*0.00011334530427120626 + S.f_preds_716*0.0004283014277461916 + S.f_preds_717*0.0003968038654420525 + S.f_preds_718*0.00012781038822140545 + S.f_preds_719*0.010870814323425293 + S.f_preds_720*0.00004498447378864512 + S.f_preds_721*0.00006518877489725128 + S.f_preds_722*0.000029586220989585854 + S.f_preds_723*0.0000569157928111963 + S.f_preds_724*0.000020265539205865934 + S.f_preds_725*0.00003896027192240581 + S.f_preds_726*0.000001130614123212581 + S.f_preds_727*0.0017423968529328704 + S.f_preds_728*0.0009984647622331977 + S.f_preds_729*0.00020527135347947478 + S.f_preds_730*0.00010141624807147309 + S.f_preds_731*0.0000013939235259385896 + S.f_preds_732*0.0017145039746537805 + S.f_preds_733*0.00048482915735803545 + S.f_preds_734*0.0003125153889413923 + S.f_preds_735*0.000006966789896978298 + S.f_preds_736*0.00024413960636593401 + S.f_preds_737*0.000002054856395261595 + S.f_preds_738*0.00002746743120951578)))) / S.norm_f_preds / 0.11221887324529495
                      as d_pc
                    from search_recs S
                    where true
                      -- Filters duplicated above (in filteredSpecies query)
                      and S.species in ('RODO', 'BTPI', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'ALHU', 'WEGU', 'CAGU', 'CATE', 'DCCO', 'BRPE', 'GBHE', 'TUVU', 'OSPR', 'NOHA', 'SSHA', 'COHA', 'RSHA', 'RTHA', 'BANO', 'GHOW', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'RMPA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'HUVI', 'CAVI', 'WAVI', 'REVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'TRES', 'VGSW', 'BARS', 'CBCH', 'BUSH', 'RBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'CEDW', 'HOFI', 'PUFI', 'PISI', 'LEGO', 'AMGO', 'CHSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'CALT', 'SPTO', 'HOOR', 'BUOR', 'BHCO', 'BRBL', 'OCWA', 'NAWA', 'COYE', 'AMRE', 'NOPA', 'YEWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'SUTA', 'WETA', 'RBGR', 'BHGR', 'LAZB', 'INBU', 'HOSP')
                      and S.quality in ('A', 'B')
                      and S.species not in ()
                      and (false
                        or S.species_species_group not in ()
                        or S.species in ()
                      )
                      and S.source_id not in ()
                      and S.source_id != 'xc:370896' -- Exclude query_rec from results
                  )
                select *
                from (
                  -- Must wrap subqueries in 'select * from (...)' else union complains about nested order by
                  select * from (
                    select *, 3.754216598942222 as slp
                    from S_filter_dist
                    where species = 'INBU'
                    order by d_pc asc
                    limit 2
                  ) union all select * from (
                    select *, 3.7657945828461847 as slp
                    from S_filter_dist
                    where species = 'BGGN'
                    order by d_pc asc
                    limit 2
                  ) union all select * from (
                    select *, 3.8252525896860767 as slp
                    from S_filter_dist
                    where species = 'BHCO'
                    order by d_pc asc
                    limit 2
                  ) union all select * from (
                    select *, 3.829855023828165 as slp
                    from S_filter_dist
                    where species = 'CBCH'
                    order by d_pc asc
                    limit 2
                  ) union all select * from (
                    select *, 3.9228879742019362 as slp
                    from S_filter_dist
                    where species = 'TOWA'
                    order by d_pc asc
                    limit 2
                  )
                )
                order by slp asc, d_pc asc

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO (1/n): Massage 0/n into a basic window query
--# - Slow...
with

    S_filter_dist as (
        select
            S.species, S.source_id, --# S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.species_order, S.license_type, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks,
            1 - ((((S.f_preds_0*0.00001311239248025231 + S.f_preds_1*0.0022739178966730833 + S.f_preds_2*0.000004044589331897441 + S.f_preds_3*0.002446443075314164 + S.f_preds_4*0.000007628525054315105 + S.f_preds_5*0.00001568856168887578 + S.f_preds_6*0.000006424799266824266 + S.f_preds_7*2.9643587140526506e-7 + S.f_preds_8*0.0000022504916614707327 + S.f_preds_9*0.00000209387690119911 + S.f_preds_10*0.00002307513750565704 + S.f_preds_11*0.001311492407694459 + S.f_preds_12*0.000015103913938219193 + S.f_preds_13*8.688106731824519e-7 + S.f_preds_14*0.00005533318835659884 + S.f_preds_15*0.000016482756109326147 + S.f_preds_16*0.0005267483065836132 + S.f_preds_17*0.0018814759096130729 + S.f_preds_18*0.0000017237175597983878 + S.f_preds_19*0.00039833024493418634 + S.f_preds_20*0.00000977200579654891 + S.f_preds_21*0.000006569051038241014 + S.f_preds_22*0.000012760150639223866 + S.f_preds_23*0.000009890945875667967 + S.f_preds_24*0.01804490014910698 + S.f_preds_25*0.004844821989536285 + S.f_preds_26*0.00023922703985590488 + S.f_preds_27*0.000015565292414976284 + S.f_preds_28*0.000010527898666623514 + S.f_preds_29*0.005573070142418146 + S.f_preds_30*0.000028921585908392444 + S.f_preds_31*0.0000033035873912012903 + S.f_preds_32*4.100716068933252e-7 + S.f_preds_33*6.601695758945425e-7 + S.f_preds_34*0.0004126813728362322 + S.f_preds_35*0.00003285566344857216 + S.f_preds_36*0.0018572922563180327 + S.f_preds_37*0.0006297582876868546 + S.f_preds_38*0.0001985664275707677 + S.f_preds_39*0.000013924099221185315 + S.f_preds_40*0.00005194657205720432 + S.f_preds_41*0.000042426461732247844 + S.f_preds_42*0.0005176834529265761 + S.f_preds_43*0.01681959442794323 + S.f_preds_44*0.00021062373707536608 + S.f_preds_45*8.841304293127905e-7 + S.f_preds_46*0.000049778231186792254 + S.f_preds_47*0.018970871344208717 + S.f_preds_48*0.0007048664847388864 + S.f_preds_49*0.000028281960112508386 + S.f_preds_50*0.0006428731721825898 + S.f_preds_51*0.0028357082046568394 + S.f_preds_52*0.0000034002564461843576 + S.f_preds_53*0.023662345483899117 + S.f_preds_54*0.00044131523463875055 + S.f_preds_55*0.00241877231746912 + S.f_preds_56*0.00003444610410952009 + S.f_preds_57*0.0014652458485215902 + S.f_preds_58*0.0020906205754727125 + S.f_preds_59*0.000013210412362241186 + S.f_preds_60*0.0001858155446825549 + S.f_preds_61*0.007895379327237606 + S.f_preds_62*0.0000024245182430604473 + S.f_preds_63*0.0000026513437205721857 + S.f_preds_64*0.0009304345585405827 + S.f_preds_65*0.00023366720415651798 + S.f_preds_66*0.005655152257531881 + S.f_preds_67*0.0009726776042953134 + S.f_preds_68*0.000012197562682558782 + S.f_preds_69*1.0011000739496012e-7 + S.f_preds_70*0.00020311045227572322 + S.f_preds_71*0.00025495662703178823 + S.f_preds_72*0.0008810111903585494 + S.f_preds_73*0.00011881050158990547 + S.f_preds_74*0.01844712905585766 + S.f_preds_75*0.0014010248705744743 + S.f_preds_76*6.89977426304722e-8 + S.f_preds_77*0.000001391107161907712 + S.f_preds_78*0.00006117242446634918 + S.f_preds_79*0.010232416912913322 + S.f_preds_80*0.000001088506337509898 + S.f_preds_81*0.023149210959672928 + S.f_preds_82*0.0218129251152277 + S.f_preds_83*0.00027983225299976766 + S.f_preds_84*0.000002296665115864016 + S.f_preds_85*0.00029460497898980975 + S.f_preds_86*0.0011194923426955938 + S.f_preds_87*0.01881752721965313 + S.f_preds_88*0.00012937233259435743 + S.f_preds_89*0.006397274788469076 + S.f_preds_90*0.000016658137610647827 + S.f_preds_91*0.00017631007358431816) + (S.f_preds_92*0.00012058904394507408 + S.f_preds_93*0.00005393704850575887 + S.f_preds_94*0.004301635082811117 + S.f_preds_95*0.0000014354379800352035 + S.f_preds_96*0.000005481480002345052 + S.f_preds_97*0.00009395895904162899 + S.f_preds_98*0.00007922713120933622 + S.f_preds_99*1.1477159489459154e-7 + S.f_preds_100*0.0003226808912586421 + S.f_preds_101*0.00006615019083255902 + S.f_preds_102*6.292503371696512e-7 + S.f_preds_103*0.00010970210860250518 + S.f_preds_104*0.0005167542258277535 + S.f_preds_105*0.0000012490669405451627 + S.f_preds_106*0.00039587795617990196 + S.f_preds_107*0.005248239729553461 + S.f_preds_108*1.9826785546683823e-7 + S.f_preds_109*0.00005959632107988 + S.f_preds_110*0.00014260836178436875 + S.f_preds_111*0.00010338414722355083 + S.f_preds_112*0.0000205065243790159 + S.f_preds_113*0.0000020813254195672926 + S.f_preds_114*0.00020459122606553137 + S.f_preds_115*0.002042416948825121 + S.f_preds_116*0.00006533438136102632 + S.f_preds_117*0.000005608823357761139 + S.f_preds_118*0.001865064725279808 + S.f_preds_119*0.00007586278661619872 + S.f_preds_120*0.0004040422209072858 + S.f_preds_121*0.0007578746299259365 + S.f_preds_122*0.0037188904825598 + S.f_preds_123*0.014093589968979359 + S.f_preds_124*0.0002631600946187973 + S.f_preds_125*0.00681808777153492 + S.f_preds_126*3.617186052906618e-7 + S.f_preds_127*0.002112553222104907 + S.f_preds_128*0.0025430587120354176 + S.f_preds_129*0.00016653123020660132 + S.f_preds_130*0.02345724031329155 + S.f_preds_131*0.002475341083481908 + S.f_preds_132*0.00014247142826206982 + S.f_preds_133*0.0005273931892588735 + S.f_preds_134*0.000013719729395234026 + S.f_preds_135*0.0035834198351949453 + S.f_preds_136*0.0000024723869955778355 + S.f_preds_137*0.017516305670142174 + S.f_preds_138*0.00004870363409281708 + S.f_preds_139*0.00013708391634281725 + S.f_preds_140*0.000009344155841972679 + S.f_preds_141*0.00001242540565726813 + S.f_preds_142*0.023233819752931595 + S.f_preds_143*0.000028271842893445864 + S.f_preds_144*0.013134469278156757 + S.f_preds_145*0.0007207064772956073 + S.f_preds_146*0.00003083339106524363 + S.f_preds_147*0.000008576449545216747 + S.f_preds_148*0.016772819682955742 + S.f_preds_149*0.0009123460622504354 + S.f_preds_150*0.012531428597867489 + S.f_preds_151*0.0002033248165389523 + S.f_preds_152*0.00003387439573998563 + S.f_preds_153*0.00012878944107796997 + S.f_preds_154*0.000011766812349378597 + S.f_preds_155*0.0035626592580229044 + S.f_preds_156*0.0000020471034076763317 + S.f_preds_157*0.0000035694906728167553 + S.f_preds_158*0.00015981854812707752 + S.f_preds_159*0.0011637889547273517 + S.f_preds_160*0.00387061876244843 + S.f_preds_161*0.00008862809045240283 + S.f_preds_162*0.00006104543717810884 + S.f_preds_163*0.00011629491928033531 + S.f_preds_164*0.00004587496732710861 + S.f_preds_165*0.0000650517686153762 + S.f_preds_166*0.0017447016434744 + S.f_preds_167*0.021712763234972954 + S.f_preds_168*0.0005890600150451064 + S.f_preds_169*6.537584908983263e-7 + S.f_preds_170*3.564814790024684e-7 + S.f_preds_171*0.0009296662174165249 + S.f_preds_172*0.00006215124449227005 + S.f_preds_173*0.012565955519676208 + S.f_preds_174*0.007263803388923407 + S.f_preds_175*0.0027377239894121885 + S.f_preds_176*0.000008244623131758999 + S.f_preds_177*0.0000023269387838809052 + S.f_preds_178*0.00022972709848545492 + S.f_preds_179*0.00002270613913424313 + S.f_preds_180*2.0221226293415384e-7 + S.f_preds_181*0.000009496605343883857 + S.f_preds_182*0.000005108888672111789 + S.f_preds_183*0.0005137779517099261)) + ((S.f_preds_184*0.000010669865332602058 + S.f_preds_185*0.0009854049421846867 + S.f_preds_186*0.0012607353273779154 + S.f_preds_187*0.0006667971611022949 + S.f_preds_188*0.000014654996448371094 + S.f_preds_189*2.450668716846849e-7 + S.f_preds_190*0.00021140761964488775 + S.f_preds_191*0.00004480480129132047 + S.f_preds_192*0.00010504756937734783 + S.f_preds_193*0.00002219480484200176 + S.f_preds_194*0.0014585570897907019 + S.f_preds_195*0.000023570381017634645 + S.f_preds_196*0.0035868771374225616 + S.f_preds_197*0.000026636558686732315 + S.f_preds_198*5.375615046432358e-7 + S.f_preds_199*0.004330820869654417 + S.f_preds_200*0.000013430114449874964 + S.f_preds_201*0.00009262594539904967 + S.f_preds_202*0.00008550980419386178 + S.f_preds_203*0.00005542610233533196 + S.f_preds_204*0.000022133155653136782 + S.f_preds_205*0.0016019666800275445 + S.f_preds_206*8.774132993494277e-7 + S.f_preds_207*8.674058449287259e-7 + S.f_preds_208*0.00004084217653144151 + S.f_preds_209*0.000011412805179134011 + S.f_preds_210*0.0005518625839613378 + S.f_preds_211*0.00008674130367580801 + S.f_preds_212*0.0000026616432933224132 + S.f_preds_213*0.000023289498130907305 + S.f_preds_214*0.00001417942166881403 + S.f_preds_215*0.00011560232087504119 + S.f_preds_216*0.000008560010428482201 + S.f_preds_217*0.0010935317259281874 + S.f_preds_218*0.00014956091763451695 + S.f_preds_219*0.0005274588475003839 + S.f_preds_220*0.0003844208549708128 + S.f_preds_221*0.008620514534413815 + S.f_preds_222*4.901575039184536e-7 + S.f_preds_223*0.000002932444431280601 + S.f_preds_224*0.00008497372618876398 + S.f_preds_225*0.000006627227776334621 + S.f_preds_226*0.00003267589272581972 + S.f_preds_227*3.307827682874631e-7 + S.f_preds_228*0.003242308972403407 + S.f_preds_229*0.00006843778101028875 + S.f_preds_230*0.000021847368770977482 + S.f_preds_231*0.0000022423150767281186 + S.f_preds_232*0.0026875378098338842 + S.f_preds_233*0.00005942605639575049 + S.f_preds_234*0.000003830415607808391 + S.f_preds_235*0.00003467744681984186 + S.f_preds_236*0.0003422269073780626 + S.f_preds_237*0.00003503493280732073 + S.f_preds_238*0.0000010439758852953673 + S.f_preds_239*0.0000018763837488222634 + S.f_preds_240*0.00012398957915138453 + S.f_preds_241*0.0011635670671239495 + S.f_preds_242*0.0000011317058579152217 + S.f_preds_243*0.015879152342677116 + S.f_preds_244*0.00003913496766472235 + S.f_preds_245*0.000037293462810339406 + S.f_preds_246*0.0000361335369234439 + S.f_preds_247*0.00005024169513490051 + S.f_preds_248*0.0013123423559591174 + S.f_preds_249*0.0000015925746765788062 + S.f_preds_250*0.0003526984073687345 + S.f_preds_251*0.0015677057672291994 + S.f_preds_252*0.000027051797587773763 + S.f_preds_253*0.00004910511779598892 + S.f_preds_254*0.0000018012084410656826 + S.f_preds_255*0.0000886311536305584 + S.f_preds_256*0.0005031937034800649 + S.f_preds_257*0.0002624462067615241 + S.f_preds_258*0.00016602191317360848 + S.f_preds_259*0.00024625565856695175 + S.f_preds_260*0.000034680469980230555 + S.f_preds_261*0.000011574317795748357 + S.f_preds_262*0.0006084603955969214 + S.f_preds_263*0.0000021471892068802845 + S.f_preds_264*0.00001021452590066474 + S.f_preds_265*0.00019133792375214398 + S.f_preds_266*0.000016060286725405604 + S.f_preds_267*0.00014434140757657588 + S.f_preds_268*0.00015234870079439133 + S.f_preds_269*0.000001094846425075957 + S.f_preds_270*0.000011648219697235618 + S.f_preds_271*0.0004886073293164372 + S.f_preds_272*0.0016627265140414238 + S.f_preds_273*0.01329568587243557 + S.f_preds_274*3.138311797101778e-7 + S.f_preds_275*0.0003020265430677682) + (S.f_preds_276*6.091020736676e-7 + S.f_preds_277*0.000008524235454387963 + S.f_preds_278*0.00020711110846605152 + S.f_preds_279*0.000003148366204186459 + S.f_preds_280*0.00014961144188418984 + S.f_preds_281*0.000024720802684896626 + S.f_preds_282*0.00008777557377470657 + S.f_preds_283*0.000008208664439735003 + S.f_preds_284*0.000012109705494367518 + S.f_preds_285*6.579759883607039e-7 + S.f_preds_286*0.000012348501513770316 + S.f_preds_287*0.0007436243467964232 + S.f_preds_288*0.000004955889380653389 + S.f_preds_289*8.595026201874134e-7 + S.f_preds_290*0.007464772555977106 + S.f_preds_291*0.0001946113770827651 + S.f_preds_292*0.00007105382974259555 + S.f_preds_293*0.0033582847099751234 + S.f_preds_294*0.0006477180868387222 + S.f_preds_295*3.6187074670124275e-7 + S.f_preds_296*0.0000693765832693316 + S.f_preds_297*0.0000033425619676563656 + S.f_preds_298*0.0000029267844183777925 + S.f_preds_299*0.0000039293249756156 + S.f_preds_300*0.000038189460610738024 + S.f_preds_301*5.310744768394215e-7 + S.f_preds_302*0.0000017443657043259009 + S.f_preds_303*0.000011264039130765013 + S.f_preds_304*0.000007809535418346059 + S.f_preds_305*0.00004670179259846918 + S.f_preds_306*0.00004879724292550236 + S.f_preds_307*0.0015165400691330433 + S.f_preds_308*0.000002572912762843771 + S.f_preds_309*0.0009610670967958868 + S.f_preds_310*0.000015019456441223156 + S.f_preds_311*0.000018154683857574128 + S.f_preds_312*0.00042174625559709966 + S.f_preds_313*0.00000472815827379236 + S.f_preds_314*0.022676438093185425 + S.f_preds_315*5.847223860655504e-7 + S.f_preds_316*0.000004039436589664547 + S.f_preds_317*0.000008301932211907115 + S.f_preds_318*0.000013070617569610476 + S.f_preds_319*0.00009437942935619503 + S.f_preds_320*9.450283044998287e-8 + S.f_preds_321*0.00013990449951961637 + S.f_preds_322*0.00000192821448763425 + S.f_preds_323*0.0000013222486359154573 + S.f_preds_324*0.000052494549890980124 + S.f_preds_325*0.000007503287179133622 + S.f_preds_326*0.0000017911829672812019 + S.f_preds_327*0.00011045519931940362 + S.f_preds_328*0.0022690247278660536 + S.f_preds_329*0.0001165569992735982 + S.f_preds_330*0.0000018839571112039266 + S.f_preds_331*0.000032350140827475116 + S.f_preds_332*0.00002531428799557034 + S.f_preds_333*0.000050617811211850494 + S.f_preds_334*0.0007663340657018125 + S.f_preds_335*0.00013900290650781244 + S.f_preds_336*0.000049367572501068935 + S.f_preds_337*0.000042445550207048655 + S.f_preds_338*0.0001376005238853395 + S.f_preds_339*0.00004797595465788618 + S.f_preds_340*0.000022352229279931635 + S.f_preds_341*0.0002962714934255928 + S.f_preds_342*0.00004398026067065075 + S.f_preds_343*0.01037600077688694 + S.f_preds_344*0.00004516577610047534 + S.f_preds_345*0.0006199416238814592 + S.f_preds_346*0.0000012476348274503835 + S.f_preds_347*0.000004629550403478788 + S.f_preds_348*0.023418789729475975 + S.f_preds_349*0.00000777882360125659 + S.f_preds_350*0.000029099664970999584 + S.f_preds_351*0.000014523630852636416 + S.f_preds_352*0.0000023763323042658158 + S.f_preds_353*0.0005294025759212673 + S.f_preds_354*0.000005048519142292207 + S.f_preds_355*0.0000032741434097260935 + S.f_preds_356*0.000007565846317447722 + S.f_preds_357*0.00004547514981823042 + S.f_preds_358*0.0002636595454532653 + S.f_preds_359*1.800865874201918e-7 + S.f_preds_360*0.00001865157719294075 + S.f_preds_361*0.000376529002096504 + S.f_preds_362*0.0000027501305339683313 + S.f_preds_363*0.00030277753830887377 + S.f_preds_364*0.00005115754538564943 + S.f_preds_365*0.0012324352283030748 + S.f_preds_366*0.00018544473277870566 + S.f_preds_367*0.00005313345172908157 + S.f_preds_368*0.000027433607101556845))) + (((S.f_preds_369*0.00020436079648789018 + S.f_preds_370*0.0006155724404379725 + S.f_preds_371*0.0000035860705338564003 + S.f_preds_372*0.000004185253146715695 + S.f_preds_373*0.00005271961708785966 + S.f_preds_374*0.000002029272991421749 + S.f_preds_375*0.00018188006652053446 + S.f_preds_376*0.000012025258911307901 + S.f_preds_377*0.0014208576176315546 + S.f_preds_378*9.447777671311997e-8 + S.f_preds_379*0.00006892470992170274 + S.f_preds_380*0.00003242859020247124 + S.f_preds_381*0.0007282765000127256 + S.f_preds_382*0.00375916319899261 + S.f_preds_383*0.0000028596371066669235 + S.f_preds_384*0.000009051633242052048 + S.f_preds_385*0.0011416266206651926 + S.f_preds_386*0.0000043961154005955905 + S.f_preds_387*0.000008625760528957471 + S.f_preds_388*0.00009348194726044312 + S.f_preds_389*0.000010609912351355888 + S.f_preds_390*0.0005257150623947382 + S.f_preds_391*0.00009466418850934133 + S.f_preds_392*0.000017193688108818606 + S.f_preds_393*0.007605379913002253 + S.f_preds_394*0.0004980259109288454 + S.f_preds_395*0.0007890399429015815 + S.f_preds_396*2.0968185410197293e-8 + S.f_preds_397*8.087992569016933e-7 + S.f_preds_398*6.560824772350315e-7 + S.f_preds_399*0.00014008089783601463 + S.f_preds_400*0.0006039145519025624 + S.f_preds_401*0.0000071473286880063824 + S.f_preds_402*0.00004870027623837814 + S.f_preds_403*0.000056747336202533916 + S.f_preds_404*0.0000023145769318944076 + S.f_preds_405*0.0008232579566538334 + S.f_preds_406*9.881814548862167e-7 + S.f_preds_407*0.0000017835916423791787 + S.f_preds_408*0.004612771328538656 + S.f_preds_409*0.000830142991617322 + S.f_preds_410*7.730937312544484e-8 + S.f_preds_411*0.012341471388936043 + S.f_preds_412*0.002177329733967781 + S.f_preds_413*0.00037074810825288296 + S.f_preds_414*0.0017332970164716244 + S.f_preds_415*0.0013398604933172464 + S.f_preds_416*0.0000012682459100687993 + S.f_preds_417*3.9440632804144116e-7 + S.f_preds_418*0.0000511453217768576 + S.f_preds_419*0.0000014611218830395956 + S.f_preds_420*1.3488853767285036e-7 + S.f_preds_421*0.0011070219334214926 + S.f_preds_422*0.000172565778484568 + S.f_preds_423*0.00000847523278935114 + S.f_preds_424*0.0000021629989532812033 + S.f_preds_425*0.00015388638712465763 + S.f_preds_426*1.244669363131834e-7 + S.f_preds_427*5.292966989145498e-7 + S.f_preds_428*0.0026793526485562325 + S.f_preds_429*0.00000242397072724998 + S.f_preds_430*0.0000017537361145514296 + S.f_preds_431*0.00007314266986213624 + S.f_preds_432*0.015580660663545132 + S.f_preds_433*0.00003077931614825502 + S.f_preds_434*0.001671489211730659 + S.f_preds_435*0.0051424503326416016 + S.f_preds_436*0.0000012573142385008396 + S.f_preds_437*0.0000017696989971227595 + S.f_preds_438*0.00003416148683754727 + S.f_preds_439*0.0008575073443353176 + S.f_preds_440*0.0004498166963458061 + S.f_preds_441*0.0014703682390972972 + S.f_preds_442*0.000014719891623826697 + S.f_preds_443*0.00017386121908202767 + S.f_preds_444*0.000073970302764792 + S.f_preds_445*0.00011574722157092765 + S.f_preds_446*0.005045986268669367 + S.f_preds_447*0.004470773506909609 + S.f_preds_448*0.000002035713578152354 + S.f_preds_449*0.000043419364374130964 + S.f_preds_450*0.012585037387907505 + S.f_preds_451*0.000001728835741232615 + S.f_preds_452*0.00000100265071978356 + S.f_preds_453*0.0015500244917348027 + S.f_preds_454*0.0013294772943481803 + S.f_preds_455*0.0010233717039227486 + S.f_preds_456*0.0010060714557766914 + S.f_preds_457*0.000006806942565162899 + S.f_preds_458*6.035304522811202e-7 + S.f_preds_459*0.00020313254208303988 + S.f_preds_460*0.003389031160622835) + (S.f_preds_461*0.0008531991043128073 + S.f_preds_462*0.00033116189297288656 + S.f_preds_463*0.000012784821592504159 + S.f_preds_464*0.0000283551707980223 + S.f_preds_465*0.0006966462824493647 + S.f_preds_466*0.00002789941027003806 + S.f_preds_467*0.0005911355256102979 + S.f_preds_468*0.002994079142808914 + S.f_preds_469*0.0000017838807480075047 + S.f_preds_470*2.4400554821113474e-7 + S.f_preds_471*5.014719022256031e-7 + S.f_preds_472*0.000011036318028345704 + S.f_preds_473*3.725356805261981e-7 + S.f_preds_474*1.266375591058022e-7 + S.f_preds_475*0.000018449984054313973 + S.f_preds_476*0.00003187067704857327 + S.f_preds_477*0.005098472349345684 + S.f_preds_478*0.0035877684131264687 + S.f_preds_479*0.00013991973537486047 + S.f_preds_480*0.00002065905755443964 + S.f_preds_481*9.214570582116721e-7 + S.f_preds_482*0.00005737582250731066 + S.f_preds_483*1.1891671647390467e-7 + S.f_preds_484*5.138089136380586e-7 + S.f_preds_485*0.00003446516711846925 + S.f_preds_486*0.00003241106605855748 + S.f_preds_487*0.00007031692803138867 + S.f_preds_488*0.00004199549584882334 + S.f_preds_489*0.00007063855446176603 + S.f_preds_490*1.3241979957001604e-7 + S.f_preds_491*0.0014177433913573623 + S.f_preds_492*0.000021480864234035835 + S.f_preds_493*0.00009588408283889294 + S.f_preds_494*0.0008136858814395964 + S.f_preds_495*0.00000723950142855756 + S.f_preds_496*0.000041920222429325804 + S.f_preds_497*0.000004934684966428904 + S.f_preds_498*0.01999528706073761 + S.f_preds_499*0.000009792875971470494 + S.f_preds_500*0.006254177540540695 + S.f_preds_501*0.0019237757660448551 + S.f_preds_502*0.0000336199882440269 + S.f_preds_503*0.00011587099288590252 + S.f_preds_504*5.696571747648704e-7 + S.f_preds_505*0.0000030567177873308538 + S.f_preds_506*0.000029180990168242715 + S.f_preds_507*0.000054005424317438155 + S.f_preds_508*0.0003109595272690058 + S.f_preds_509*0.000013896006748836953 + S.f_preds_510*0.00003571807610569522 + S.f_preds_511*0.002394929062575102 + S.f_preds_512*0.0001247953186975792 + S.f_preds_513*0.00014427122368942946 + S.f_preds_514*0.00009447086631553248 + S.f_preds_515*0.000035395132726989686 + S.f_preds_516*0.0030862544663250446 + S.f_preds_517*0.0020114730577915907 + S.f_preds_518*0.0015699266223236918 + S.f_preds_519*0.000022298700059764087 + S.f_preds_520*0.017703445628285408 + S.f_preds_521*0.0014080592663958669 + S.f_preds_522*0.005447646602988243 + S.f_preds_523*0.01645824871957302 + S.f_preds_524*0.000027788570150732994 + S.f_preds_525*0.00001340962899121223 + S.f_preds_526*0.000019065997548750602 + S.f_preds_527*0.000009252670679416042 + S.f_preds_528*0.0000025471656499576056 + S.f_preds_529*7.461294160293619e-8 + S.f_preds_530*0.00017533496429678053 + S.f_preds_531*0.000027902546207769774 + S.f_preds_532*0.000016202477127080783 + S.f_preds_533*0.00020475717610679567 + S.f_preds_534*0.0004499423084780574 + S.f_preds_535*0.0006966591463424265 + S.f_preds_536*0.000550146447494626 + S.f_preds_537*0.00016486116510350257 + S.f_preds_538*0.000007280455974978395 + S.f_preds_539*0.000010827132427948527 + S.f_preds_540*0.0000876995109138079 + S.f_preds_541*0.0003642363881226629 + S.f_preds_542*0.0000012616019375855103 + S.f_preds_543*0.0000019034998786082724 + S.f_preds_544*0.00005276263254927471 + S.f_preds_545*0.00005496077210409567 + S.f_preds_546*0.00027376782963983715 + S.f_preds_547*0.000005710881850973237 + S.f_preds_548*8.988230604245473e-9 + S.f_preds_549*0.0000016730429024391924 + S.f_preds_550*0.00007679287227801979 + S.f_preds_551*0.0000016976399592749658 + S.f_preds_552*0.000004411753707245225 + S.f_preds_553*0.000005111335212859558)) + ((S.f_preds_554*0.00016011434490792453 + S.f_preds_555*0.0000029296325010363944 + S.f_preds_556*0.00025140767684206367 + S.f_preds_557*0.00005884619167773053 + S.f_preds_558*0.0015645562671124935 + S.f_preds_559*0.00020754926663357764 + S.f_preds_560*0.00025478313909843564 + S.f_preds_561*0.002211499260738492 + S.f_preds_562*6.609280376324023e-7 + S.f_preds_563*0.0010862791677936912 + S.f_preds_564*0.000012614359548024368 + S.f_preds_565*0.00017907589790411294 + S.f_preds_566*0.0000037052925563330064 + S.f_preds_567*0.00010445003135828301 + S.f_preds_568*0.017669612541794777 + S.f_preds_569*0.00000553270638192771 + S.f_preds_570*0.00006034673788235523 + S.f_preds_571*0.00039125533658079803 + S.f_preds_572*0.000007338504929066403 + S.f_preds_573*0.000122875499073416 + S.f_preds_574*2.4581188995398406e-7 + S.f_preds_575*0.000011374868336133659 + S.f_preds_576*0.00008426068961853161 + S.f_preds_577*6.751050705133821e-7 + S.f_preds_578*0.0000064212285906251054 + S.f_preds_579*0.00005932396015850827 + S.f_preds_580*0.0004973491886630654 + S.f_preds_581*5.367184030546923e-7 + S.f_preds_582*0.000743536336813122 + S.f_preds_583*0.0000017930664171217359 + S.f_preds_584*0.0006554555729962885 + S.f_preds_585*0.0000037011327549407724 + S.f_preds_586*0.0002179535513278097 + S.f_preds_587*0.00003099400782957673 + S.f_preds_588*0.000004918352260574466 + S.f_preds_589*0.000009350197615276556 + S.f_preds_590*0.000016639018213027157 + S.f_preds_591*0.00010717164695961401 + S.f_preds_592*0.000023528449673904106 + S.f_preds_593*0.000001007815853881766 + S.f_preds_594*0.000005481845619215164 + S.f_preds_595*0.00015956266724970192 + S.f_preds_596*0.0002082450082525611 + S.f_preds_597*0.00020021837553940713 + S.f_preds_598*0.0010172325419262052 + S.f_preds_599*0.0000010955390052913572 + S.f_preds_600*0.0000026854265797737753 + S.f_preds_601*0.0015208498807623982 + S.f_preds_602*0.000006445457074732985 + S.f_preds_603*0.000020037914509885013 + S.f_preds_604*1.9325770495015604e-7 + S.f_preds_605*0.0000010705190334192594 + S.f_preds_606*0.000028866608772659674 + S.f_preds_607*0.00010479458433110267 + S.f_preds_608*0.010287586599588394 + S.f_preds_609*0.0012989990646019578 + S.f_preds_610*0.000003944566287827911 + S.f_preds_611*2.8879350466581855e-8 + S.f_preds_612*0.0000069853654167673085 + S.f_preds_613*0.000021991474568494596 + S.f_preds_614*0.00007703194569330662 + S.f_preds_615*0.00023750816762913018 + S.f_preds_616*0.00944328773766756 + S.f_preds_617*0.0000010323501555831172 + S.f_preds_618*1.501768736034137e-7 + S.f_preds_619*0.00007150989404181018 + S.f_preds_620*0.0000061345799622358754 + S.f_preds_621*0.0004368036170490086 + S.f_preds_622*8.976332992460812e-7 + S.f_preds_623*0.001628484926186502 + S.f_preds_624*0.000003928264959540684 + S.f_preds_625*0.0009367550374008715 + S.f_preds_626*0.0000026312400223105215 + S.f_preds_627*0.00006375551311066374 + S.f_preds_628*5.137369498697808e-7 + S.f_preds_629*0.00001137478193413699 + S.f_preds_630*0.000017766524251783267 + S.f_preds_631*0.003897836897522211 + S.f_preds_632*0.008055249229073524 + S.f_preds_633*0.00241384981200099 + S.f_preds_634*0.00003595540329115465 + S.f_preds_635*0.00005645097917295061 + S.f_preds_636*0.00006621936336159706 + S.f_preds_637*0.014510611072182655 + S.f_preds_638*1.050159710302978e-7 + S.f_preds_639*0.019783876836299896 + S.f_preds_640*0.008949201554059982 + S.f_preds_641*0.00024250347632914782 + S.f_preds_642*0.00010105391265824437 + S.f_preds_643*0.00004060524588567205 + S.f_preds_644*0.0001331030362052843 + S.f_preds_645*8.045903427955636e-7) + (S.f_preds_646*0.00001027736743708374 + S.f_preds_647*1.7172181898672534e-8 + S.f_preds_648*0.0033125479239970446 + S.f_preds_649*0.0000016601031802565558 + S.f_preds_650*0.00005077445530332625 + S.f_preds_651*0.0002987897605635226 + S.f_preds_652*0.002370136557146907 + S.f_preds_653*0.0005649944650940597 + S.f_preds_654*0.000028247362934052944 + S.f_preds_655*0.000047565263230353594 + S.f_preds_656*0.00001847093335527461 + S.f_preds_657*0.000002787693574646255 + S.f_preds_658*0.000005108099685458001 + S.f_preds_659*0.0010631985496729612 + S.f_preds_660*0.00004261540743755177 + S.f_preds_661*0.0002864064881578088 + S.f_preds_662*0.00009470334771322086 + S.f_preds_663*0.00006371989729814231 + S.f_preds_664*4.832561444345629e-7 + S.f_preds_665*0.0000020821610178245464 + S.f_preds_666*0.0002292780700372532 + S.f_preds_667*3.029888162586758e-8 + S.f_preds_668*0.00005065908044343814 + S.f_preds_669*0.00020125712035223842 + S.f_preds_670*0.00047489258577115834 + S.f_preds_671*3.227335909627982e-8 + S.f_preds_672*0.0001816168805817142 + S.f_preds_673*0.0000014918402939656517 + S.f_preds_674*0.000007308628937607864 + S.f_preds_675*0.00004637502206605859 + S.f_preds_676*0.00005088275429443456 + S.f_preds_677*0.0004129812878090888 + S.f_preds_678*0.02267596498131752 + S.f_preds_679*0.00004278955384506844 + S.f_preds_680*0.0005632776883430779 + S.f_preds_681*0.0000029753814487776253 + S.f_preds_682*0.0010443839710205793 + S.f_preds_683*0.000003015262109329342 + S.f_preds_684*0.0000015277154261639225 + S.f_preds_685*0.00008885315764928237 + S.f_preds_686*0.0001388078526360914 + S.f_preds_687*0.00009291117748944089 + S.f_preds_688*0.00005038944073021412 + S.f_preds_689*4.75471949812345e-7 + S.f_preds_690*1.248018861588207e-7 + S.f_preds_691*0.001185665954835713 + S.f_preds_692*0.00003074324922636151 + S.f_preds_693*9.277101753468742e-7 + S.f_preds_694*0.00022056890884414315 + S.f_preds_695*0.007895344868302345 + S.f_preds_696*0.013903992250561714 + S.f_preds_697*0.00014689314411953092 + S.f_preds_698*0.000002063769670712645 + S.f_preds_699*0.0009233330492861569 + S.f_preds_700*0.000016178952137124725 + S.f_preds_701*0.0010267875622957945 + S.f_preds_702*0.00005498975224327296 + S.f_preds_703*0.00013355587725527585 + S.f_preds_704*0.00002518714427424129 + S.f_preds_705*0.0001367626537103206 + S.f_preds_706*0.0002352102310396731 + S.f_preds_707*0.00003683730392367579 + S.f_preds_708*0.00019189013983123004 + S.f_preds_709*0.00040489446837455034 + S.f_preds_710*0.000008120999154925812 + S.f_preds_711*0.000005265692379907705 + S.f_preds_712*0.000013629485692945309 + S.f_preds_713*0.00010411084804218262 + S.f_preds_714*0.00010439017933094874 + S.f_preds_715*0.00011334530427120626 + S.f_preds_716*0.0004283014277461916 + S.f_preds_717*0.0003968038654420525 + S.f_preds_718*0.00012781038822140545 + S.f_preds_719*0.010870814323425293 + S.f_preds_720*0.00004498447378864512 + S.f_preds_721*0.00006518877489725128 + S.f_preds_722*0.000029586220989585854 + S.f_preds_723*0.0000569157928111963 + S.f_preds_724*0.000020265539205865934 + S.f_preds_725*0.00003896027192240581 + S.f_preds_726*0.000001130614123212581 + S.f_preds_727*0.0017423968529328704 + S.f_preds_728*0.0009984647622331977 + S.f_preds_729*0.00020527135347947478 + S.f_preds_730*0.00010141624807147309 + S.f_preds_731*0.0000013939235259385896 + S.f_preds_732*0.0017145039746537805 + S.f_preds_733*0.00048482915735803545 + S.f_preds_734*0.0003125153889413923 + S.f_preds_735*0.000006966789896978298 + S.f_preds_736*0.00024413960636593401 + S.f_preds_737*0.000002054856395261595 + S.f_preds_738*0.00002746743120951578)))) / S.norm_f_preds / 0.11221887324529495
            as d_pc
        from search_recs S
        where true
            --# Filters
            and S.species in ('RODO', 'BTPI', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'ALHU', 'WEGU', 'CAGU', 'CATE', 'DCCO', 'BRPE', 'GBHE', 'TUVU', 'OSPR', 'NOHA', 'SSHA', 'COHA', 'RSHA', 'RTHA', 'BANO', 'GHOW', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'RMPA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'HUVI', 'CAVI', 'WAVI', 'REVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'TRES', 'VGSW', 'BARS', 'CBCH', 'BUSH', 'RBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'CEDW', 'HOFI', 'PUFI', 'PISI', 'LEGO', 'AMGO', 'CHSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'CALT', 'SPTO', 'HOOR', 'BUOR', 'BHCO', 'BRBL', 'OCWA', 'NAWA', 'COYE', 'AMRE', 'NOPA', 'YEWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'SUTA', 'WETA', 'RBGR', 'BHGR', 'LAZB', 'INBU', 'HOSP')
            and S.quality in ('A', 'B')
            and S.species not in ()
            and (false
                or S.species_species_group not in ()
                or S.species in ()
            )
            and S.source_id not in ()
            and S.source_id != 'xc:370896' -- Exclude query_rec from results
    ),

    S_filter_dist_spi as (
        select
            S.*,
            row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
        from
            S_filter_dist S
    ),

    slp as (
        select column1 as species, column2 as slp from (values
            ('INBU', 3.754216598942222),
            ('BGGN', 3.7657945828461847),
            ('BHCO', 3.8252525896860767),
            ('CBCH', 3.829855023828165),
            ('TOWA', 3.9228879742019362)
        )
    ),

    S_filter_dist_spi_slp_top as (
        select
            S.species, S.source_id, --# ...
            sp_d_pc_i,
            coalesce(slp.slp, 1e38) as slp,
            S.d_pc
        from slp
            join S_filter_dist_spi S using (species)
        where
            sp_d_pc_i <= 2
    )

select *
from S_filter_dist_spi_slp_top
order by
    slp asc, d_pc asc
limit 31

In [ ]:
%%sqla -ceng -Iplan -p3
--# TODO (2/n): Massage 1/n into a faster window query
--# - Fast! Let's base on this approach
--# - TODO Hmm, but even this doesn't appear that it will be any faster than 0/n... (based on running each a few times to warm caches)
with

    slp as (
        select column1 as species, column2 as slp from (values
            ('INBU', 3.754216598942222),
            ('BGGN', 3.7657945828461847),
            ('BHCO', 3.8252525896860767),
            ('CBCH', 3.829855023828165),
            ('TOWA', 3.9228879742019362)
        )
    ),

    S_filter_dist as (
        select
            --# S.species, S.source_id,
            S.month_day, S.shorthand, S.species_taxon_id, S.source_id, S.variable, S.channels, S.n_background_species, S.background_species, S.taxon_id, S.license_detail, S.species_order, S.license_type, S.background, S.samples_mb, S.state, S.species_taxon_order, S.basename, S.playback_used, S.place, S.speed, S.length, S.samples_n, S.lng, S.volume, S.duration_s, S.recordist, S.bird_seen, S.downloaded, S.state_only, S.xc_channels, S.download, S.norm_f_preds, S.species, S.species_family, S.country, S.xc_bitrate_of_mp3, S.dataset, S.recs_for_sp, S.species_sci_name, S.sample_width_bit, S.quality, S.date, S.time, S.xc_sampling_rate, S.pitch, S.url, S.year, S.license, S.hour, S.elevation, S.species_species_group, S.place_only, S.species_com_name, S.taxon_order, S.sample_rate, S.locality, S.lat, S.xc_id, S.path, S.number_of_notes, S.month, S.id, S.species_query, S.subspecies, S.species_longhand, S.type, S.remarks,
            1 - ((((S.f_preds_0*0.00001311239248025231 + S.f_preds_1*0.0022739178966730833 + S.f_preds_2*0.000004044589331897441 + S.f_preds_3*0.002446443075314164 + S.f_preds_4*0.000007628525054315105 + S.f_preds_5*0.00001568856168887578 + S.f_preds_6*0.000006424799266824266 + S.f_preds_7*2.9643587140526506e-7 + S.f_preds_8*0.0000022504916614707327 + S.f_preds_9*0.00000209387690119911 + S.f_preds_10*0.00002307513750565704 + S.f_preds_11*0.001311492407694459 + S.f_preds_12*0.000015103913938219193 + S.f_preds_13*8.688106731824519e-7 + S.f_preds_14*0.00005533318835659884 + S.f_preds_15*0.000016482756109326147 + S.f_preds_16*0.0005267483065836132 + S.f_preds_17*0.0018814759096130729 + S.f_preds_18*0.0000017237175597983878 + S.f_preds_19*0.00039833024493418634 + S.f_preds_20*0.00000977200579654891 + S.f_preds_21*0.000006569051038241014 + S.f_preds_22*0.000012760150639223866 + S.f_preds_23*0.000009890945875667967 + S.f_preds_24*0.01804490014910698 + S.f_preds_25*0.004844821989536285 + S.f_preds_26*0.00023922703985590488 + S.f_preds_27*0.000015565292414976284 + S.f_preds_28*0.000010527898666623514 + S.f_preds_29*0.005573070142418146 + S.f_preds_30*0.000028921585908392444 + S.f_preds_31*0.0000033035873912012903 + S.f_preds_32*4.100716068933252e-7 + S.f_preds_33*6.601695758945425e-7 + S.f_preds_34*0.0004126813728362322 + S.f_preds_35*0.00003285566344857216 + S.f_preds_36*0.0018572922563180327 + S.f_preds_37*0.0006297582876868546 + S.f_preds_38*0.0001985664275707677 + S.f_preds_39*0.000013924099221185315 + S.f_preds_40*0.00005194657205720432 + S.f_preds_41*0.000042426461732247844 + S.f_preds_42*0.0005176834529265761 + S.f_preds_43*0.01681959442794323 + S.f_preds_44*0.00021062373707536608 + S.f_preds_45*8.841304293127905e-7 + S.f_preds_46*0.000049778231186792254 + S.f_preds_47*0.018970871344208717 + S.f_preds_48*0.0007048664847388864 + S.f_preds_49*0.000028281960112508386 + S.f_preds_50*0.0006428731721825898 + S.f_preds_51*0.0028357082046568394 + S.f_preds_52*0.0000034002564461843576 + S.f_preds_53*0.023662345483899117 + S.f_preds_54*0.00044131523463875055 + S.f_preds_55*0.00241877231746912 + S.f_preds_56*0.00003444610410952009 + S.f_preds_57*0.0014652458485215902 + S.f_preds_58*0.0020906205754727125 + S.f_preds_59*0.000013210412362241186 + S.f_preds_60*0.0001858155446825549 + S.f_preds_61*0.007895379327237606 + S.f_preds_62*0.0000024245182430604473 + S.f_preds_63*0.0000026513437205721857 + S.f_preds_64*0.0009304345585405827 + S.f_preds_65*0.00023366720415651798 + S.f_preds_66*0.005655152257531881 + S.f_preds_67*0.0009726776042953134 + S.f_preds_68*0.000012197562682558782 + S.f_preds_69*1.0011000739496012e-7 + S.f_preds_70*0.00020311045227572322 + S.f_preds_71*0.00025495662703178823 + S.f_preds_72*0.0008810111903585494 + S.f_preds_73*0.00011881050158990547 + S.f_preds_74*0.01844712905585766 + S.f_preds_75*0.0014010248705744743 + S.f_preds_76*6.89977426304722e-8 + S.f_preds_77*0.000001391107161907712 + S.f_preds_78*0.00006117242446634918 + S.f_preds_79*0.010232416912913322 + S.f_preds_80*0.000001088506337509898 + S.f_preds_81*0.023149210959672928 + S.f_preds_82*0.0218129251152277 + S.f_preds_83*0.00027983225299976766 + S.f_preds_84*0.000002296665115864016 + S.f_preds_85*0.00029460497898980975 + S.f_preds_86*0.0011194923426955938 + S.f_preds_87*0.01881752721965313 + S.f_preds_88*0.00012937233259435743 + S.f_preds_89*0.006397274788469076 + S.f_preds_90*0.000016658137610647827 + S.f_preds_91*0.00017631007358431816) + (S.f_preds_92*0.00012058904394507408 + S.f_preds_93*0.00005393704850575887 + S.f_preds_94*0.004301635082811117 + S.f_preds_95*0.0000014354379800352035 + S.f_preds_96*0.000005481480002345052 + S.f_preds_97*0.00009395895904162899 + S.f_preds_98*0.00007922713120933622 + S.f_preds_99*1.1477159489459154e-7 + S.f_preds_100*0.0003226808912586421 + S.f_preds_101*0.00006615019083255902 + S.f_preds_102*6.292503371696512e-7 + S.f_preds_103*0.00010970210860250518 + S.f_preds_104*0.0005167542258277535 + S.f_preds_105*0.0000012490669405451627 + S.f_preds_106*0.00039587795617990196 + S.f_preds_107*0.005248239729553461 + S.f_preds_108*1.9826785546683823e-7 + S.f_preds_109*0.00005959632107988 + S.f_preds_110*0.00014260836178436875 + S.f_preds_111*0.00010338414722355083 + S.f_preds_112*0.0000205065243790159 + S.f_preds_113*0.0000020813254195672926 + S.f_preds_114*0.00020459122606553137 + S.f_preds_115*0.002042416948825121 + S.f_preds_116*0.00006533438136102632 + S.f_preds_117*0.000005608823357761139 + S.f_preds_118*0.001865064725279808 + S.f_preds_119*0.00007586278661619872 + S.f_preds_120*0.0004040422209072858 + S.f_preds_121*0.0007578746299259365 + S.f_preds_122*0.0037188904825598 + S.f_preds_123*0.014093589968979359 + S.f_preds_124*0.0002631600946187973 + S.f_preds_125*0.00681808777153492 + S.f_preds_126*3.617186052906618e-7 + S.f_preds_127*0.002112553222104907 + S.f_preds_128*0.0025430587120354176 + S.f_preds_129*0.00016653123020660132 + S.f_preds_130*0.02345724031329155 + S.f_preds_131*0.002475341083481908 + S.f_preds_132*0.00014247142826206982 + S.f_preds_133*0.0005273931892588735 + S.f_preds_134*0.000013719729395234026 + S.f_preds_135*0.0035834198351949453 + S.f_preds_136*0.0000024723869955778355 + S.f_preds_137*0.017516305670142174 + S.f_preds_138*0.00004870363409281708 + S.f_preds_139*0.00013708391634281725 + S.f_preds_140*0.000009344155841972679 + S.f_preds_141*0.00001242540565726813 + S.f_preds_142*0.023233819752931595 + S.f_preds_143*0.000028271842893445864 + S.f_preds_144*0.013134469278156757 + S.f_preds_145*0.0007207064772956073 + S.f_preds_146*0.00003083339106524363 + S.f_preds_147*0.000008576449545216747 + S.f_preds_148*0.016772819682955742 + S.f_preds_149*0.0009123460622504354 + S.f_preds_150*0.012531428597867489 + S.f_preds_151*0.0002033248165389523 + S.f_preds_152*0.00003387439573998563 + S.f_preds_153*0.00012878944107796997 + S.f_preds_154*0.000011766812349378597 + S.f_preds_155*0.0035626592580229044 + S.f_preds_156*0.0000020471034076763317 + S.f_preds_157*0.0000035694906728167553 + S.f_preds_158*0.00015981854812707752 + S.f_preds_159*0.0011637889547273517 + S.f_preds_160*0.00387061876244843 + S.f_preds_161*0.00008862809045240283 + S.f_preds_162*0.00006104543717810884 + S.f_preds_163*0.00011629491928033531 + S.f_preds_164*0.00004587496732710861 + S.f_preds_165*0.0000650517686153762 + S.f_preds_166*0.0017447016434744 + S.f_preds_167*0.021712763234972954 + S.f_preds_168*0.0005890600150451064 + S.f_preds_169*6.537584908983263e-7 + S.f_preds_170*3.564814790024684e-7 + S.f_preds_171*0.0009296662174165249 + S.f_preds_172*0.00006215124449227005 + S.f_preds_173*0.012565955519676208 + S.f_preds_174*0.007263803388923407 + S.f_preds_175*0.0027377239894121885 + S.f_preds_176*0.000008244623131758999 + S.f_preds_177*0.0000023269387838809052 + S.f_preds_178*0.00022972709848545492 + S.f_preds_179*0.00002270613913424313 + S.f_preds_180*2.0221226293415384e-7 + S.f_preds_181*0.000009496605343883857 + S.f_preds_182*0.000005108888672111789 + S.f_preds_183*0.0005137779517099261)) + ((S.f_preds_184*0.000010669865332602058 + S.f_preds_185*0.0009854049421846867 + S.f_preds_186*0.0012607353273779154 + S.f_preds_187*0.0006667971611022949 + S.f_preds_188*0.000014654996448371094 + S.f_preds_189*2.450668716846849e-7 + S.f_preds_190*0.00021140761964488775 + S.f_preds_191*0.00004480480129132047 + S.f_preds_192*0.00010504756937734783 + S.f_preds_193*0.00002219480484200176 + S.f_preds_194*0.0014585570897907019 + S.f_preds_195*0.000023570381017634645 + S.f_preds_196*0.0035868771374225616 + S.f_preds_197*0.000026636558686732315 + S.f_preds_198*5.375615046432358e-7 + S.f_preds_199*0.004330820869654417 + S.f_preds_200*0.000013430114449874964 + S.f_preds_201*0.00009262594539904967 + S.f_preds_202*0.00008550980419386178 + S.f_preds_203*0.00005542610233533196 + S.f_preds_204*0.000022133155653136782 + S.f_preds_205*0.0016019666800275445 + S.f_preds_206*8.774132993494277e-7 + S.f_preds_207*8.674058449287259e-7 + S.f_preds_208*0.00004084217653144151 + S.f_preds_209*0.000011412805179134011 + S.f_preds_210*0.0005518625839613378 + S.f_preds_211*0.00008674130367580801 + S.f_preds_212*0.0000026616432933224132 + S.f_preds_213*0.000023289498130907305 + S.f_preds_214*0.00001417942166881403 + S.f_preds_215*0.00011560232087504119 + S.f_preds_216*0.000008560010428482201 + S.f_preds_217*0.0010935317259281874 + S.f_preds_218*0.00014956091763451695 + S.f_preds_219*0.0005274588475003839 + S.f_preds_220*0.0003844208549708128 + S.f_preds_221*0.008620514534413815 + S.f_preds_222*4.901575039184536e-7 + S.f_preds_223*0.000002932444431280601 + S.f_preds_224*0.00008497372618876398 + S.f_preds_225*0.000006627227776334621 + S.f_preds_226*0.00003267589272581972 + S.f_preds_227*3.307827682874631e-7 + S.f_preds_228*0.003242308972403407 + S.f_preds_229*0.00006843778101028875 + S.f_preds_230*0.000021847368770977482 + S.f_preds_231*0.0000022423150767281186 + S.f_preds_232*0.0026875378098338842 + S.f_preds_233*0.00005942605639575049 + S.f_preds_234*0.000003830415607808391 + S.f_preds_235*0.00003467744681984186 + S.f_preds_236*0.0003422269073780626 + S.f_preds_237*0.00003503493280732073 + S.f_preds_238*0.0000010439758852953673 + S.f_preds_239*0.0000018763837488222634 + S.f_preds_240*0.00012398957915138453 + S.f_preds_241*0.0011635670671239495 + S.f_preds_242*0.0000011317058579152217 + S.f_preds_243*0.015879152342677116 + S.f_preds_244*0.00003913496766472235 + S.f_preds_245*0.000037293462810339406 + S.f_preds_246*0.0000361335369234439 + S.f_preds_247*0.00005024169513490051 + S.f_preds_248*0.0013123423559591174 + S.f_preds_249*0.0000015925746765788062 + S.f_preds_250*0.0003526984073687345 + S.f_preds_251*0.0015677057672291994 + S.f_preds_252*0.000027051797587773763 + S.f_preds_253*0.00004910511779598892 + S.f_preds_254*0.0000018012084410656826 + S.f_preds_255*0.0000886311536305584 + S.f_preds_256*0.0005031937034800649 + S.f_preds_257*0.0002624462067615241 + S.f_preds_258*0.00016602191317360848 + S.f_preds_259*0.00024625565856695175 + S.f_preds_260*0.000034680469980230555 + S.f_preds_261*0.000011574317795748357 + S.f_preds_262*0.0006084603955969214 + S.f_preds_263*0.0000021471892068802845 + S.f_preds_264*0.00001021452590066474 + S.f_preds_265*0.00019133792375214398 + S.f_preds_266*0.000016060286725405604 + S.f_preds_267*0.00014434140757657588 + S.f_preds_268*0.00015234870079439133 + S.f_preds_269*0.000001094846425075957 + S.f_preds_270*0.000011648219697235618 + S.f_preds_271*0.0004886073293164372 + S.f_preds_272*0.0016627265140414238 + S.f_preds_273*0.01329568587243557 + S.f_preds_274*3.138311797101778e-7 + S.f_preds_275*0.0003020265430677682) + (S.f_preds_276*6.091020736676e-7 + S.f_preds_277*0.000008524235454387963 + S.f_preds_278*0.00020711110846605152 + S.f_preds_279*0.000003148366204186459 + S.f_preds_280*0.00014961144188418984 + S.f_preds_281*0.000024720802684896626 + S.f_preds_282*0.00008777557377470657 + S.f_preds_283*0.000008208664439735003 + S.f_preds_284*0.000012109705494367518 + S.f_preds_285*6.579759883607039e-7 + S.f_preds_286*0.000012348501513770316 + S.f_preds_287*0.0007436243467964232 + S.f_preds_288*0.000004955889380653389 + S.f_preds_289*8.595026201874134e-7 + S.f_preds_290*0.007464772555977106 + S.f_preds_291*0.0001946113770827651 + S.f_preds_292*0.00007105382974259555 + S.f_preds_293*0.0033582847099751234 + S.f_preds_294*0.0006477180868387222 + S.f_preds_295*3.6187074670124275e-7 + S.f_preds_296*0.0000693765832693316 + S.f_preds_297*0.0000033425619676563656 + S.f_preds_298*0.0000029267844183777925 + S.f_preds_299*0.0000039293249756156 + S.f_preds_300*0.000038189460610738024 + S.f_preds_301*5.310744768394215e-7 + S.f_preds_302*0.0000017443657043259009 + S.f_preds_303*0.000011264039130765013 + S.f_preds_304*0.000007809535418346059 + S.f_preds_305*0.00004670179259846918 + S.f_preds_306*0.00004879724292550236 + S.f_preds_307*0.0015165400691330433 + S.f_preds_308*0.000002572912762843771 + S.f_preds_309*0.0009610670967958868 + S.f_preds_310*0.000015019456441223156 + S.f_preds_311*0.000018154683857574128 + S.f_preds_312*0.00042174625559709966 + S.f_preds_313*0.00000472815827379236 + S.f_preds_314*0.022676438093185425 + S.f_preds_315*5.847223860655504e-7 + S.f_preds_316*0.000004039436589664547 + S.f_preds_317*0.000008301932211907115 + S.f_preds_318*0.000013070617569610476 + S.f_preds_319*0.00009437942935619503 + S.f_preds_320*9.450283044998287e-8 + S.f_preds_321*0.00013990449951961637 + S.f_preds_322*0.00000192821448763425 + S.f_preds_323*0.0000013222486359154573 + S.f_preds_324*0.000052494549890980124 + S.f_preds_325*0.000007503287179133622 + S.f_preds_326*0.0000017911829672812019 + S.f_preds_327*0.00011045519931940362 + S.f_preds_328*0.0022690247278660536 + S.f_preds_329*0.0001165569992735982 + S.f_preds_330*0.0000018839571112039266 + S.f_preds_331*0.000032350140827475116 + S.f_preds_332*0.00002531428799557034 + S.f_preds_333*0.000050617811211850494 + S.f_preds_334*0.0007663340657018125 + S.f_preds_335*0.00013900290650781244 + S.f_preds_336*0.000049367572501068935 + S.f_preds_337*0.000042445550207048655 + S.f_preds_338*0.0001376005238853395 + S.f_preds_339*0.00004797595465788618 + S.f_preds_340*0.000022352229279931635 + S.f_preds_341*0.0002962714934255928 + S.f_preds_342*0.00004398026067065075 + S.f_preds_343*0.01037600077688694 + S.f_preds_344*0.00004516577610047534 + S.f_preds_345*0.0006199416238814592 + S.f_preds_346*0.0000012476348274503835 + S.f_preds_347*0.000004629550403478788 + S.f_preds_348*0.023418789729475975 + S.f_preds_349*0.00000777882360125659 + S.f_preds_350*0.000029099664970999584 + S.f_preds_351*0.000014523630852636416 + S.f_preds_352*0.0000023763323042658158 + S.f_preds_353*0.0005294025759212673 + S.f_preds_354*0.000005048519142292207 + S.f_preds_355*0.0000032741434097260935 + S.f_preds_356*0.000007565846317447722 + S.f_preds_357*0.00004547514981823042 + S.f_preds_358*0.0002636595454532653 + S.f_preds_359*1.800865874201918e-7 + S.f_preds_360*0.00001865157719294075 + S.f_preds_361*0.000376529002096504 + S.f_preds_362*0.0000027501305339683313 + S.f_preds_363*0.00030277753830887377 + S.f_preds_364*0.00005115754538564943 + S.f_preds_365*0.0012324352283030748 + S.f_preds_366*0.00018544473277870566 + S.f_preds_367*0.00005313345172908157 + S.f_preds_368*0.000027433607101556845))) + (((S.f_preds_369*0.00020436079648789018 + S.f_preds_370*0.0006155724404379725 + S.f_preds_371*0.0000035860705338564003 + S.f_preds_372*0.000004185253146715695 + S.f_preds_373*0.00005271961708785966 + S.f_preds_374*0.000002029272991421749 + S.f_preds_375*0.00018188006652053446 + S.f_preds_376*0.000012025258911307901 + S.f_preds_377*0.0014208576176315546 + S.f_preds_378*9.447777671311997e-8 + S.f_preds_379*0.00006892470992170274 + S.f_preds_380*0.00003242859020247124 + S.f_preds_381*0.0007282765000127256 + S.f_preds_382*0.00375916319899261 + S.f_preds_383*0.0000028596371066669235 + S.f_preds_384*0.000009051633242052048 + S.f_preds_385*0.0011416266206651926 + S.f_preds_386*0.0000043961154005955905 + S.f_preds_387*0.000008625760528957471 + S.f_preds_388*0.00009348194726044312 + S.f_preds_389*0.000010609912351355888 + S.f_preds_390*0.0005257150623947382 + S.f_preds_391*0.00009466418850934133 + S.f_preds_392*0.000017193688108818606 + S.f_preds_393*0.007605379913002253 + S.f_preds_394*0.0004980259109288454 + S.f_preds_395*0.0007890399429015815 + S.f_preds_396*2.0968185410197293e-8 + S.f_preds_397*8.087992569016933e-7 + S.f_preds_398*6.560824772350315e-7 + S.f_preds_399*0.00014008089783601463 + S.f_preds_400*0.0006039145519025624 + S.f_preds_401*0.0000071473286880063824 + S.f_preds_402*0.00004870027623837814 + S.f_preds_403*0.000056747336202533916 + S.f_preds_404*0.0000023145769318944076 + S.f_preds_405*0.0008232579566538334 + S.f_preds_406*9.881814548862167e-7 + S.f_preds_407*0.0000017835916423791787 + S.f_preds_408*0.004612771328538656 + S.f_preds_409*0.000830142991617322 + S.f_preds_410*7.730937312544484e-8 + S.f_preds_411*0.012341471388936043 + S.f_preds_412*0.002177329733967781 + S.f_preds_413*0.00037074810825288296 + S.f_preds_414*0.0017332970164716244 + S.f_preds_415*0.0013398604933172464 + S.f_preds_416*0.0000012682459100687993 + S.f_preds_417*3.9440632804144116e-7 + S.f_preds_418*0.0000511453217768576 + S.f_preds_419*0.0000014611218830395956 + S.f_preds_420*1.3488853767285036e-7 + S.f_preds_421*0.0011070219334214926 + S.f_preds_422*0.000172565778484568 + S.f_preds_423*0.00000847523278935114 + S.f_preds_424*0.0000021629989532812033 + S.f_preds_425*0.00015388638712465763 + S.f_preds_426*1.244669363131834e-7 + S.f_preds_427*5.292966989145498e-7 + S.f_preds_428*0.0026793526485562325 + S.f_preds_429*0.00000242397072724998 + S.f_preds_430*0.0000017537361145514296 + S.f_preds_431*0.00007314266986213624 + S.f_preds_432*0.015580660663545132 + S.f_preds_433*0.00003077931614825502 + S.f_preds_434*0.001671489211730659 + S.f_preds_435*0.0051424503326416016 + S.f_preds_436*0.0000012573142385008396 + S.f_preds_437*0.0000017696989971227595 + S.f_preds_438*0.00003416148683754727 + S.f_preds_439*0.0008575073443353176 + S.f_preds_440*0.0004498166963458061 + S.f_preds_441*0.0014703682390972972 + S.f_preds_442*0.000014719891623826697 + S.f_preds_443*0.00017386121908202767 + S.f_preds_444*0.000073970302764792 + S.f_preds_445*0.00011574722157092765 + S.f_preds_446*0.005045986268669367 + S.f_preds_447*0.004470773506909609 + S.f_preds_448*0.000002035713578152354 + S.f_preds_449*0.000043419364374130964 + S.f_preds_450*0.012585037387907505 + S.f_preds_451*0.000001728835741232615 + S.f_preds_452*0.00000100265071978356 + S.f_preds_453*0.0015500244917348027 + S.f_preds_454*0.0013294772943481803 + S.f_preds_455*0.0010233717039227486 + S.f_preds_456*0.0010060714557766914 + S.f_preds_457*0.000006806942565162899 + S.f_preds_458*6.035304522811202e-7 + S.f_preds_459*0.00020313254208303988 + S.f_preds_460*0.003389031160622835) + (S.f_preds_461*0.0008531991043128073 + S.f_preds_462*0.00033116189297288656 + S.f_preds_463*0.000012784821592504159 + S.f_preds_464*0.0000283551707980223 + S.f_preds_465*0.0006966462824493647 + S.f_preds_466*0.00002789941027003806 + S.f_preds_467*0.0005911355256102979 + S.f_preds_468*0.002994079142808914 + S.f_preds_469*0.0000017838807480075047 + S.f_preds_470*2.4400554821113474e-7 + S.f_preds_471*5.014719022256031e-7 + S.f_preds_472*0.000011036318028345704 + S.f_preds_473*3.725356805261981e-7 + S.f_preds_474*1.266375591058022e-7 + S.f_preds_475*0.000018449984054313973 + S.f_preds_476*0.00003187067704857327 + S.f_preds_477*0.005098472349345684 + S.f_preds_478*0.0035877684131264687 + S.f_preds_479*0.00013991973537486047 + S.f_preds_480*0.00002065905755443964 + S.f_preds_481*9.214570582116721e-7 + S.f_preds_482*0.00005737582250731066 + S.f_preds_483*1.1891671647390467e-7 + S.f_preds_484*5.138089136380586e-7 + S.f_preds_485*0.00003446516711846925 + S.f_preds_486*0.00003241106605855748 + S.f_preds_487*0.00007031692803138867 + S.f_preds_488*0.00004199549584882334 + S.f_preds_489*0.00007063855446176603 + S.f_preds_490*1.3241979957001604e-7 + S.f_preds_491*0.0014177433913573623 + S.f_preds_492*0.000021480864234035835 + S.f_preds_493*0.00009588408283889294 + S.f_preds_494*0.0008136858814395964 + S.f_preds_495*0.00000723950142855756 + S.f_preds_496*0.000041920222429325804 + S.f_preds_497*0.000004934684966428904 + S.f_preds_498*0.01999528706073761 + S.f_preds_499*0.000009792875971470494 + S.f_preds_500*0.006254177540540695 + S.f_preds_501*0.0019237757660448551 + S.f_preds_502*0.0000336199882440269 + S.f_preds_503*0.00011587099288590252 + S.f_preds_504*5.696571747648704e-7 + S.f_preds_505*0.0000030567177873308538 + S.f_preds_506*0.000029180990168242715 + S.f_preds_507*0.000054005424317438155 + S.f_preds_508*0.0003109595272690058 + S.f_preds_509*0.000013896006748836953 + S.f_preds_510*0.00003571807610569522 + S.f_preds_511*0.002394929062575102 + S.f_preds_512*0.0001247953186975792 + S.f_preds_513*0.00014427122368942946 + S.f_preds_514*0.00009447086631553248 + S.f_preds_515*0.000035395132726989686 + S.f_preds_516*0.0030862544663250446 + S.f_preds_517*0.0020114730577915907 + S.f_preds_518*0.0015699266223236918 + S.f_preds_519*0.000022298700059764087 + S.f_preds_520*0.017703445628285408 + S.f_preds_521*0.0014080592663958669 + S.f_preds_522*0.005447646602988243 + S.f_preds_523*0.01645824871957302 + S.f_preds_524*0.000027788570150732994 + S.f_preds_525*0.00001340962899121223 + S.f_preds_526*0.000019065997548750602 + S.f_preds_527*0.000009252670679416042 + S.f_preds_528*0.0000025471656499576056 + S.f_preds_529*7.461294160293619e-8 + S.f_preds_530*0.00017533496429678053 + S.f_preds_531*0.000027902546207769774 + S.f_preds_532*0.000016202477127080783 + S.f_preds_533*0.00020475717610679567 + S.f_preds_534*0.0004499423084780574 + S.f_preds_535*0.0006966591463424265 + S.f_preds_536*0.000550146447494626 + S.f_preds_537*0.00016486116510350257 + S.f_preds_538*0.000007280455974978395 + S.f_preds_539*0.000010827132427948527 + S.f_preds_540*0.0000876995109138079 + S.f_preds_541*0.0003642363881226629 + S.f_preds_542*0.0000012616019375855103 + S.f_preds_543*0.0000019034998786082724 + S.f_preds_544*0.00005276263254927471 + S.f_preds_545*0.00005496077210409567 + S.f_preds_546*0.00027376782963983715 + S.f_preds_547*0.000005710881850973237 + S.f_preds_548*8.988230604245473e-9 + S.f_preds_549*0.0000016730429024391924 + S.f_preds_550*0.00007679287227801979 + S.f_preds_551*0.0000016976399592749658 + S.f_preds_552*0.000004411753707245225 + S.f_preds_553*0.000005111335212859558)) + ((S.f_preds_554*0.00016011434490792453 + S.f_preds_555*0.0000029296325010363944 + S.f_preds_556*0.00025140767684206367 + S.f_preds_557*0.00005884619167773053 + S.f_preds_558*0.0015645562671124935 + S.f_preds_559*0.00020754926663357764 + S.f_preds_560*0.00025478313909843564 + S.f_preds_561*0.002211499260738492 + S.f_preds_562*6.609280376324023e-7 + S.f_preds_563*0.0010862791677936912 + S.f_preds_564*0.000012614359548024368 + S.f_preds_565*0.00017907589790411294 + S.f_preds_566*0.0000037052925563330064 + S.f_preds_567*0.00010445003135828301 + S.f_preds_568*0.017669612541794777 + S.f_preds_569*0.00000553270638192771 + S.f_preds_570*0.00006034673788235523 + S.f_preds_571*0.00039125533658079803 + S.f_preds_572*0.000007338504929066403 + S.f_preds_573*0.000122875499073416 + S.f_preds_574*2.4581188995398406e-7 + S.f_preds_575*0.000011374868336133659 + S.f_preds_576*0.00008426068961853161 + S.f_preds_577*6.751050705133821e-7 + S.f_preds_578*0.0000064212285906251054 + S.f_preds_579*0.00005932396015850827 + S.f_preds_580*0.0004973491886630654 + S.f_preds_581*5.367184030546923e-7 + S.f_preds_582*0.000743536336813122 + S.f_preds_583*0.0000017930664171217359 + S.f_preds_584*0.0006554555729962885 + S.f_preds_585*0.0000037011327549407724 + S.f_preds_586*0.0002179535513278097 + S.f_preds_587*0.00003099400782957673 + S.f_preds_588*0.000004918352260574466 + S.f_preds_589*0.000009350197615276556 + S.f_preds_590*0.000016639018213027157 + S.f_preds_591*0.00010717164695961401 + S.f_preds_592*0.000023528449673904106 + S.f_preds_593*0.000001007815853881766 + S.f_preds_594*0.000005481845619215164 + S.f_preds_595*0.00015956266724970192 + S.f_preds_596*0.0002082450082525611 + S.f_preds_597*0.00020021837553940713 + S.f_preds_598*0.0010172325419262052 + S.f_preds_599*0.0000010955390052913572 + S.f_preds_600*0.0000026854265797737753 + S.f_preds_601*0.0015208498807623982 + S.f_preds_602*0.000006445457074732985 + S.f_preds_603*0.000020037914509885013 + S.f_preds_604*1.9325770495015604e-7 + S.f_preds_605*0.0000010705190334192594 + S.f_preds_606*0.000028866608772659674 + S.f_preds_607*0.00010479458433110267 + S.f_preds_608*0.010287586599588394 + S.f_preds_609*0.0012989990646019578 + S.f_preds_610*0.000003944566287827911 + S.f_preds_611*2.8879350466581855e-8 + S.f_preds_612*0.0000069853654167673085 + S.f_preds_613*0.000021991474568494596 + S.f_preds_614*0.00007703194569330662 + S.f_preds_615*0.00023750816762913018 + S.f_preds_616*0.00944328773766756 + S.f_preds_617*0.0000010323501555831172 + S.f_preds_618*1.501768736034137e-7 + S.f_preds_619*0.00007150989404181018 + S.f_preds_620*0.0000061345799622358754 + S.f_preds_621*0.0004368036170490086 + S.f_preds_622*8.976332992460812e-7 + S.f_preds_623*0.001628484926186502 + S.f_preds_624*0.000003928264959540684 + S.f_preds_625*0.0009367550374008715 + S.f_preds_626*0.0000026312400223105215 + S.f_preds_627*0.00006375551311066374 + S.f_preds_628*5.137369498697808e-7 + S.f_preds_629*0.00001137478193413699 + S.f_preds_630*0.000017766524251783267 + S.f_preds_631*0.003897836897522211 + S.f_preds_632*0.008055249229073524 + S.f_preds_633*0.00241384981200099 + S.f_preds_634*0.00003595540329115465 + S.f_preds_635*0.00005645097917295061 + S.f_preds_636*0.00006621936336159706 + S.f_preds_637*0.014510611072182655 + S.f_preds_638*1.050159710302978e-7 + S.f_preds_639*0.019783876836299896 + S.f_preds_640*0.008949201554059982 + S.f_preds_641*0.00024250347632914782 + S.f_preds_642*0.00010105391265824437 + S.f_preds_643*0.00004060524588567205 + S.f_preds_644*0.0001331030362052843 + S.f_preds_645*8.045903427955636e-7) + (S.f_preds_646*0.00001027736743708374 + S.f_preds_647*1.7172181898672534e-8 + S.f_preds_648*0.0033125479239970446 + S.f_preds_649*0.0000016601031802565558 + S.f_preds_650*0.00005077445530332625 + S.f_preds_651*0.0002987897605635226 + S.f_preds_652*0.002370136557146907 + S.f_preds_653*0.0005649944650940597 + S.f_preds_654*0.000028247362934052944 + S.f_preds_655*0.000047565263230353594 + S.f_preds_656*0.00001847093335527461 + S.f_preds_657*0.000002787693574646255 + S.f_preds_658*0.000005108099685458001 + S.f_preds_659*0.0010631985496729612 + S.f_preds_660*0.00004261540743755177 + S.f_preds_661*0.0002864064881578088 + S.f_preds_662*0.00009470334771322086 + S.f_preds_663*0.00006371989729814231 + S.f_preds_664*4.832561444345629e-7 + S.f_preds_665*0.0000020821610178245464 + S.f_preds_666*0.0002292780700372532 + S.f_preds_667*3.029888162586758e-8 + S.f_preds_668*0.00005065908044343814 + S.f_preds_669*0.00020125712035223842 + S.f_preds_670*0.00047489258577115834 + S.f_preds_671*3.227335909627982e-8 + S.f_preds_672*0.0001816168805817142 + S.f_preds_673*0.0000014918402939656517 + S.f_preds_674*0.000007308628937607864 + S.f_preds_675*0.00004637502206605859 + S.f_preds_676*0.00005088275429443456 + S.f_preds_677*0.0004129812878090888 + S.f_preds_678*0.02267596498131752 + S.f_preds_679*0.00004278955384506844 + S.f_preds_680*0.0005632776883430779 + S.f_preds_681*0.0000029753814487776253 + S.f_preds_682*0.0010443839710205793 + S.f_preds_683*0.000003015262109329342 + S.f_preds_684*0.0000015277154261639225 + S.f_preds_685*0.00008885315764928237 + S.f_preds_686*0.0001388078526360914 + S.f_preds_687*0.00009291117748944089 + S.f_preds_688*0.00005038944073021412 + S.f_preds_689*4.75471949812345e-7 + S.f_preds_690*1.248018861588207e-7 + S.f_preds_691*0.001185665954835713 + S.f_preds_692*0.00003074324922636151 + S.f_preds_693*9.277101753468742e-7 + S.f_preds_694*0.00022056890884414315 + S.f_preds_695*0.007895344868302345 + S.f_preds_696*0.013903992250561714 + S.f_preds_697*0.00014689314411953092 + S.f_preds_698*0.000002063769670712645 + S.f_preds_699*0.0009233330492861569 + S.f_preds_700*0.000016178952137124725 + S.f_preds_701*0.0010267875622957945 + S.f_preds_702*0.00005498975224327296 + S.f_preds_703*0.00013355587725527585 + S.f_preds_704*0.00002518714427424129 + S.f_preds_705*0.0001367626537103206 + S.f_preds_706*0.0002352102310396731 + S.f_preds_707*0.00003683730392367579 + S.f_preds_708*0.00019189013983123004 + S.f_preds_709*0.00040489446837455034 + S.f_preds_710*0.000008120999154925812 + S.f_preds_711*0.000005265692379907705 + S.f_preds_712*0.000013629485692945309 + S.f_preds_713*0.00010411084804218262 + S.f_preds_714*0.00010439017933094874 + S.f_preds_715*0.00011334530427120626 + S.f_preds_716*0.0004283014277461916 + S.f_preds_717*0.0003968038654420525 + S.f_preds_718*0.00012781038822140545 + S.f_preds_719*0.010870814323425293 + S.f_preds_720*0.00004498447378864512 + S.f_preds_721*0.00006518877489725128 + S.f_preds_722*0.000029586220989585854 + S.f_preds_723*0.0000569157928111963 + S.f_preds_724*0.000020265539205865934 + S.f_preds_725*0.00003896027192240581 + S.f_preds_726*0.000001130614123212581 + S.f_preds_727*0.0017423968529328704 + S.f_preds_728*0.0009984647622331977 + S.f_preds_729*0.00020527135347947478 + S.f_preds_730*0.00010141624807147309 + S.f_preds_731*0.0000013939235259385896 + S.f_preds_732*0.0017145039746537805 + S.f_preds_733*0.00048482915735803545 + S.f_preds_734*0.0003125153889413923 + S.f_preds_735*0.000006966789896978298 + S.f_preds_736*0.00024413960636593401 + S.f_preds_737*0.000002054856395261595 + S.f_preds_738*0.00002746743120951578)))) / S.norm_f_preds / 0.11221887324529495
            as d_pc
        from search_recs S
        where true
            --# NOTE This filter is the key to making it fast (vs. 1/n which is slow)
            and S.species in (select species from slp)
    ),

    S_filter_dist_spi as (
        select
            S.*,
            row_number() over (partition by S.species order by S.d_pc) as sp_d_pc_i
        from
            S_filter_dist S
    ),

    S_filter_dist_spi_slp_top as (
        select
            S.species, S.source_id, --# ...
            sp_d_pc_i,
            coalesce(slp.slp, 1e38) as slp,
            S.d_pc
        from slp
            join S_filter_dist_spi S using (species)
        where true
            and sp_d_pc_i <= 2
            --# Exclude query_rec from results
            and S.source_id != 'xc:370896'
            --# Filters
            and S.species in ('RODO', 'BTPI', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'ALHU', 'WEGU', 'CAGU', 'CATE', 'DCCO', 'BRPE', 'GBHE', 'TUVU', 'OSPR', 'NOHA', 'SSHA', 'COHA', 'RSHA', 'RTHA', 'BANO', 'GHOW', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'RMPA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'HUVI', 'CAVI', 'WAVI', 'REVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'TRES', 'VGSW', 'BARS', 'CBCH', 'BUSH', 'RBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'CEDW', 'HOFI', 'PUFI', 'PISI', 'LEGO', 'AMGO', 'CHSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'CALT', 'SPTO', 'HOOR', 'BUOR', 'BHCO', 'BRBL', 'OCWA', 'NAWA', 'COYE', 'AMRE', 'NOPA', 'YEWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'SUTA', 'WETA', 'RBGR', 'BHGR', 'LAZB', 'INBU', 'HOSP')
            and S.quality in ('A', 'B')
            and S.species not in ()
            and (false
                or S.species_species_group not in ()
                or S.species in ()
            )
            and S.source_id not in ()
    )

select *
from S_filter_dist_spi_slp_top
order by
    slp asc, d_pc asc
limit 31

In [ ]:
%%sqla -ceng -Iplan -p3
select distinct species
from search_recs S
where true
    -- Filters duplicated below (in final query)
    and S.species in ('RODO', 'BTPI', 'EUCD', 'MODO', 'WTSW', 'ANHU', 'ALHU', 'WEGU', 'CAGU', 'CATE', 'DCCO', 'BRPE', 'GBHE', 'TUVU', 'OSPR', 'NOHA', 'SSHA', 'COHA', 'RSHA', 'RTHA', 'BANO', 'GHOW', 'ACWO', 'DOWO', 'NUWO', 'HAWO', 'NOFL', 'AMKE', 'MERL', 'PEFA', 'RMPA', 'OSFL', 'WEWP', 'WIFL', 'PSFL', 'BLPH', 'SAPH', 'ATFL', 'HUVI', 'CAVI', 'WAVI', 'REVI', 'STJA', 'CASJ', 'AMCR', 'CORA', 'TRES', 'VGSW', 'BARS', 'CBCH', 'BUSH', 'RBNU', 'PYNU', 'BRCR', 'ROWR', 'HOWR', 'PAWR', 'MAWR', 'BEWR', 'BGGN', 'GCKI', 'RCKI', 'WREN', 'WEBL', 'VATH', 'SWTH', 'HETH', 'AMRO', 'NOMO', 'EUST', 'CEDW', 'HOFI', 'PUFI', 'PISI', 'LEGO', 'AMGO', 'CHSP', 'FOSP', 'DEJU', 'WCSP', 'GCSP', 'WTSP', 'SAVS', 'SOSP', 'LISP', 'CALT', 'SPTO', 'HOOR', 'BUOR', 'BHCO', 'BRBL', 'OCWA', 'NAWA', 'COYE', 'AMRE', 'NOPA', 'YEWA', 'YRWA', 'BTYW', 'TOWA', 'HEWA', 'WIWA', 'SUTA', 'WETA', 'RBGR', 'BHGR', 'LAZB', 'INBU', 'HOSP')        -- Safe for covering index (species)
    and S.quality in ('A', 'B')      -- Safe for covering index (quality)
    and S.species not in ()      -- Safe for covering index (species)
    and (false
      or S.species_species_group not in ()
      or S.species in ()
    ) -- Safe for covering index (species_species_group)
    and S.source_id not in ()          -- Safe for covering index (source_id)